<a href="https://colab.research.google.com/github/braltoids0089/BIO_APP/blob/main/(%2B%2B)Digital_Twin_Workflow_(APP_CONVERT)__From_Gene_Expression_to_Pathway_Analysis_Guided_Drug_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Digital Twin Workflow: From Gene Expression to Pathway Analysis-Guided Drug Selection**

**A. Introduction**

This notebook demonstrates a workflow for identifying potential drug therapies for cancer patients based on their gene expression data. The approach involves:
- Loading and processing gene expression data (TPM values) from the Toil database.
- Focusing on a subset of genes relevant to predefined signaling pathways.
- Performing Principal Component Analysis (PCA) for dimensionality reduction and visualization of samples (Tumor vs Normal).
- Scoring the activity of selected signaling pathways for each patient.
- Formulating a Quadratic Unconstrained Binary Optimization (QUBO) problem to select a combination of drugs from a predefined panel, considering both potential benefit (based on pathway activity) and penalties for overlapping mechanisms.
- Solving the QUBO problem using either an exact solver (for small problems) or a quantum Approximate Optimization Algorithm (QAOA) implemented with CUDA-Q (if available).
- Analyzing the frequency and co-selection patterns of the selected drugs across multiple patients.
- Generating a patient-level report summarizing selected drugs and pathway context.
- Performing a synthetic classification validation to assess the predictive potential of the selected drug features.

The goal is to provide a computational approach for personalized therapy selection by mapping biological pathway activity to a discrete optimization problem and exploring the potential for predicting outcomes based on the selected therapies.

**B. Scope and Limitations**

- **Data:** The model is designed to handle gene expression data, specifically TPM values, and relies on a predefined set of genes and pathways (`SIGS`). The current implementation uses a small subset of samples (`N_SAMPLES = 40`) for demonstration purposes.
- **Pathway Definitions:** The drug panel (`example_drug_panel`) and the mapping of drugs to pathways are predefined and simplified.
- **QUBO Formulation:** The QUBO formulation is a specific model that balances pathway activation (benefit) with drug overlap (penalty). Other formulations are possible and might yield different results. The penalty matrix (`build_penalty_matrix`) uses a fixed `base_overlap` and `sparsity` parameter.
- **Solver:** The exact QUBO solver is limited to small numbers of drugs (K ≤ 20) due to the exponential complexity. The CUDA-Q QAOA solver's performance and availability depend on the CUDA-Q installation and the size of the problem.
- **Multi-patient Analysis:** The multi-patient analysis is performed on a limited number of samples (`N=25`) and provides a basic assessment of drug selection frequency and co-selection.
- **Patient-Level Reporting:** The patient-level report is a basic summary and does not include detailed clinical information or visualizations.
- **Synthetic Classification:** The classification validation uses synthetic labels based on mean pathway activity and is intended as a demonstration of how selected drug features *could* potentially be used for prediction. It does not represent a real-world clinical prediction task and has significant limitations regarding biological complexity and external validity. The high performance metrics (ROC-AUC, PR-AUC) are expected due to the synthetic nature of the labels and should not be interpreted as indicative of real-world predictive accuracy.
- **Biological Complexity:** The model simplifies complex biological interactions and drug mechanisms. It does not account for all potential factors influencing drug response, such as pharmacokinetics, pharmacodynamics, patient history, or other genetic variations.
- **Clinical Validation:** The results from this model are computational predictions and require rigorous clinical validation before being used for actual treatment decisions.

**C. Model Modules and Implementation**

The model is implemented in Python and utilizes several key libraries:
- **`pandas`:** For data loading, manipulation, and structuring (DataFrames).
- **`numpy`:** For numerical operations, including matrix calculations for PCA (SVD) and QUBO formulation.
- **`gzip` and `re`:** For reading and processing the compressed data file.
- **`collections.Counter`:** For summarizing drug selection frequencies.
- **`matplotlib.pyplot`:** For generating visualizations (PCA scatter plot, heatmap, bar plot, dendrogram).
- **`scipy.cluster.hierarchy`:** For hierarchical clustering and dendrogram generation.
- **`cudaq` (optional):** For attempting to solve the QUBO problem using QAOA on quantum hardware or simulators.
- **`itertools`:** For generating combinations of drugs for co-selection analysis.
- **`sklearn.linear_model.LogisticRegression` and `sklearn.metrics`:** For the synthetic classification validation.

***The code is organized into distinct sections:***

- **Setup:** Installs necessary libraries and handles environment setup.
- **Parameters & data fetch:** Defines parameters and downloads the input data.
- **Unified ingest → small expression matrix:** Reads and processes the raw data to create a small expression matrix (`expr_sym_small`).
- **Baseline PCA:** Performs PCA on the expression data.
- **Pathway scoring:** Implements functions for z-scoring and calculating pathway activity scores (`P`).
- **QUBO → CUDA-Q QAOA:** Defines the drug panel, utility functions for QUBO formulation, and the QAOA solver (with an exact fallback).
- **Multi-patient + Stability:** Runs the drug selection process for multiple patients and summarizes the frequency and size distribution of drug selections.
- **Visualization:** Generates plots to visualize the results (drug frequency barplot, co-selection heatmap, dendrogram, clustered heatmap, selection size histogram, classification coefficients).
- **Top co-selected pairs:** Analyzes and ranks the most frequently co-selected drug pairs.
- **Patient-level report:** Generates a summary table for each patient, including selected drugs and pathway context.
- **Synthetic Classification Validation:** Performs a classification task using selected drugs as features and synthetic labels to explore predictive potential.


**D. Key Points**

- The notebook provides a complete workflow from raw gene expression data to personalized drug therapy selection using a computational approach.
- It demonstrates how to integrate biological pathway knowledge into a quantitative model.
- The use of a QUBO formulation allows the problem to be potentially addressed by quantum computing approaches like QAOA, while also providing a classical exact solver fallback.
- The analysis of drug selection frequency and co-selection patterns across patients provides insights into the potential stability and commonalities of the predicted therapies.
- The notebook includes a patient-level report for summarizing individual therapy selections and their biological context.
- A synthetic classification validation is performed to demonstrate how selected drug features could potentially be used for predicting outcomes.
- The approach is modular, with distinct steps for data processing, analysis, optimization, reporting, and validation.

# **0) Setup (lean pins + CUDA-Q; single restart)**

In [ ]:
#@title 🔧 Hotfix: align NumPy/Matplotlib/SciPy/Sklearn (auto-restart)
import os, sys, subprocess

pkgs = [
    "numpy<2",              # 1.26.x
    "scipy<1.12",           # built against numpy 1.26
    "scikit-learn==1.3.2",  # compatible with numpy 1.26
    "matplotlib==3.8.4",    # compatible with numpy 1.26
    "pandas==2.2.2",
]
cmd=[sys.executable,"-m","pip","install","-q","--no-cache-dir","--force-reinstall"]+pkgs
print("Installing:", pkgs)
subprocess.check_call(cmd)

# optional: (re)install CUDA-Q after aligning stack
try:
    subprocess.check_call([sys.executable,"-m","pip","install","-q","--no-cache-dir","cudaq"])
    print("CUDA-Q OK")
except subprocess.CalledProcessError:
    print("CUDA-Q install skipped (you can try again later)")

# one-time restart to load the new ABI
flag="/content/.abi_aligned_np126"
if not os.path.exists(flag):
    open(flag,"w").close()
    print("🔁 Restarting runtime to finalize ABI alignment…")
    os._exit(0)


Installing: ['numpy<2', 'scipy<1.12', 'scikit-learn==1.3.2', 'matplotlib==3.8.4', 'pandas==2.2.2']


# **1) Parameters & data fetch (Toil TPM; tiny sample count)**

In [ ]:
#@title 1) Params & download Toil TPM
import os, requests

# keep small to stay light
N_SAMPLES = 40  # increase to 80–120 later if you want

TPM_URL  = "https://toil.xenahubs.net/download/TcgaTargetGtex_rsem_gene_tpm.gz"
TPM_PATH = "/content/TcgaTargetGtex_rsem_gene_tpm.gz"

if not os.path.exists(TPM_PATH):
    print("Downloading Toil TPM…")
    r = requests.get(TPM_URL, stream=True, timeout=300); r.raise_for_status()
    with open(TPM_PATH,"wb") as f:
        for ch in r.iter_content(1<<20):
            if ch: f.write(ch)
    print("✅ Saved:", TPM_PATH)
else:
    print("Found:", TPM_PATH)


# **2–6) Unified ingest → small expression matrix (expr_sym_small)**

In [ ]:
#@title 2–6) One-shot ingest → expr_sym_small (streamed, robust)
import re, gzip, pandas as pd
from collections import Counter

# A) read header → gene_col + sample subset
with gzip.open(TPM_PATH, "rt", encoding="utf-8", errors="replace") as f:
    header = f.readline().rstrip("\n")
cols = header.split("\t")
gene_col = cols[0]
sample_cols = [c for c in cols[1:] if isinstance(c,str) and c.startswith("TCGA-")]
if not sample_cols: sample_cols = cols[1:]
SELECTED_SAMPLES = sample_cols[:N_SAMPLES]
print(f"gene_col={gene_col}  |  selected_samples={len(SELECTED_SAMPLES)}")

# B) compact pathway signatures (editable)
SIGS = {
    "REACTOME_SIGNALING_BY_EGFR": [
        "EGFR","ERBB2","ERBB3","GRB2","SOS1","SHC1","PTPN11","KRAS","NRAS","HRAS",
        "BRAF","MAP2K1","MAP2K2","MAPK1","MAPK3","PLCG1","PIK3CA","PIK3R1","AKT1","AKT2","AKT3","GAB1"
    ],
    "REACTOME_SIGNALING_BY_ALK": [
        "ALK","EML4","GRB2","SHC1","PIK3CA","PIK3R1","AKT1","AKT2","AKT3","STAT3","MAP2K1","MAPK1","MAPK3"
    ],
    "REACTOME_MAPK1_MAPK3_SIGNALING": [
        "BRAF","RAF1","MAP2K1","MAP2K2","MAPK1","MAPK3","DUSP6","DUSP4","FOS","JUN","EGFR"
    ],
    "REACTOME_PI3K_AKT_SIGNALING": [
        "PIK3CA","PIK3CB","PIK3CD","PIK3R1","PIK3R2","AKT1","AKT2","AKT3","PTEN","MTOR","RHEB"
    ],
    "REACTOME_MTORC1_MEDIATED_SIGNALLING": [
        "MTOR","RPTOR","MLST8","RHEB","TSC1","TSC2","EIF4EBP1","RPS6KB1","RPS6"
    ],
    "REACTOME_PD1_SIGNALING": [
        "PDCD1","CD274","PDCD1LG2","JAK1","JAK2","STAT1","IFNG","GZMB","LAG3","TIGIT","CXCL9","CXCL10"
    ],
    "REACTOME_VEGFA_VEGFR2_SIGNALING_PATHWAY": [
        "VEGFA","KDR","FLT1","FLT4","PTPRB","PLCG1","MAP2K1","MAPK1","NOS3"
    ],
    "REACTOME_SIGNALING_BY_FGFR": [
        "FGFR1","FGFR2","FGFR3","FGFR4","FRS2","PLCG1","PIK3CA","PIK3R1","MAP2K1","MAPK1"
    ]
}
SIG_GENES = sorted({g for gs in SIGS.values() for g in gs})

# C) built-in symbol→Ensembl map (no network calls)
SYM2ENSG = {
    "EGFR":"ENSG00000146648","ERBB2":"ENSG00000141736","ERBB3":"ENSG00000065361","GRB2":"ENSG00000177885",
    "SOS1":"ENSG00000115904","SHC1":"ENSG00000154639","PTPN11":"ENSG00000179295","KRAS":"ENSG00000133703",
    "NRAS":"ENSG00000213281","HRAS":"ENSG00000174775","BRAF":"ENSG00000157764","MAP2K1":"ENSG00000169032",
    "MAP2K2":"ENSG00000126934","MAPK1":"ENSG00000100030","MAPK3":"ENSG00000102882","PLCG1":"ENSG00000124181",
    "PIK3CA":"ENSG00000121879","PIK3R1":"ENSG00000145675","AKT1":"ENSG00000142208","AKT2":"ENSG00000105221",
    "AKT3":"ENSG00000117020","GAB1":"ENSG00000117676",
    "ALK":"ENSG00000171094","EML4":"ENSG00000143924","STAT3":"ENSG00000168610",
    "DUSP6":"ENSG00000139318","DUSP4":"ENSG00000120875","FOS":"ENSG00000170345","JUN":"ENSG00000177606","RAF1":"ENSG00000132155",
    "PIK3CB":"ENSG00000119402","PIK3CD":"ENSG00000171608","PIK3R2":"ENSG00000189403","PTEN":"ENSG00000171862",
    "MTOR":"ENSG00000198793","RHEB":"ENSG00000106615",
    "RPTOR":"ENSG00000141564","MLST8":"ENSG00000105705","TSC1":"ENSG00000165699","TSC2":"ENSG00000103197",
    "EIF4EBP1":"ENSG00000187840","RPS6KB1":"ENSG00000108443","RPS6":"ENSG00000137154",
    "PDCD1":"ENSG00000276977","CD274":"ENSG00000120217","PDCD1LG2":"ENSG00000197646","JAK1":"ENSG00000162434",
    "JAK2":"ENSG00000096968","STAT1":"ENSG00000115415","IFNG":"ENSG00000111537","GZMB":"ENSG00000100453",
    "LAG3":"ENSG00000089692","TIGIT":"ENSG00000181847","CXCL9":"ENSG00000138755","CXCL10":"ENSG00000169245",
    "VEGFA":"ENSG00000112715","KDR":"ENSG00000128052","FLT1":"ENSG00000102755","FLT4":"ENSG00000037280",
    "PTPRB":"ENSG00000160593","NOS3":"ENSG00000164867",
    "FGFR1":"ENSG00000077782","FGFR2":"ENSG00000066468","FGFR3":"ENSG00000068078","FGFR4":"ENSG00000069535",
    "FRS2":"ENSG00000181873"
}
ENSG_SET = set(SYM2ENSG.get(g) for g in SIG_GENES if g in SYM2ENSG)

# D) detect row ID type (Ensembl vs symbol)
def detect_row_mode(path, scan_rows=50000):
    seen = Counter(); total = 0
    with gzip.open(path, "rt", encoding="utf-8", errors="replace") as f:
        reader = pd.read_csv(f, sep="\t", chunksize=200_000, usecols=[0], dtype=str, header=0)
        for ch in reader:
            v = ch.iloc[:,0].astype(str)
            vals = v.head(min(len(v), scan_rows-total)).tolist()
            total += len(vals)
            seen.update('ENSG' if x.startswith('ENSG') else 'OTHER' for x in vals)
            if total >= scan_rows: break
    ratio = seen['ENSG']/max(1,(seen['ENSG']+seen['OTHER']))
    mode = 'ensembl' if ratio >= 0.6 else 'symbol'
    print(f"Row mode: {mode.upper()} (ENSG ratio={ratio:.2f})")
    return mode

row_mode = detect_row_mode(TPM_PATH)

# E) stream-select rows + columns (light)
def norm_ensembl(x): return x.split('.',1)[0]
def norm_symbol(x):  return re.sub(r'[^A-Za-z0-9_-]+','', x)

TARGET_ROWS = ENSG_SET if row_mode=='ensembl' else set(SIG_GENES)
normalize   = norm_ensembl if row_mode=='ensembl' else norm_symbol

usecols = [gene_col] + SELECTED_SAMPLES
kept, n_hits = [], 0
with gzip.open(TPM_PATH, "rt", encoding="utf-8", errors="replace") as f:
    reader = pd.read_csv(f, sep="\t", chunksize=50_000, dtype=str,
                         usecols=lambda c: (c in usecols) or (c == gene_col))
    for ch in reader:
        ch = ch.rename(columns={gene_col: "row_id"})
        ids = ch["row_id"].astype(str).map(normalize)
        mask = ids.isin(TARGET_ROWS)
        if mask.any():
            out = ch.loc[mask].copy()
            out["row_id"] = out["row_id"].map(normalize)
            kept.append(out); n_hits += int(mask.sum())

if n_hits == 0:
    raise RuntimeError("No signature rows matched. Try smaller N_SAMPLES or a tinier SIGS.")

expr_small = pd.concat(kept, axis=0, ignore_index=False).drop_duplicates(subset=["row_id"]).set_index("row_id")

# F) convert to symbols if needed
if row_mode == 'ensembl':
    ENSG2SYM = {v:k for k,v in SYM2ENSG.items()}
    expr_sym_small = expr_small.copy()
    expr_sym_small.index = [ENSG2SYM.get(e, e) for e in expr_small.index]
else:
    expr_sym_small = expr_small.copy()

expr_sym_small = expr_sym_small[~expr_sym_small.index.duplicated(keep="first")]
print(f"✅ expr_sym_small ready: {expr_sym_small.shape} (genes × samples)")
display(expr_sym_small.iloc[:5, :5])


# **7) Baseline PCA (TCGA Tumor vs Normal coloring; no phenotype needed)**

In [ ]:
#@title 7) Clean → PCA (robust to string/NaN; uses NumPy SVD; optional plot)
import numpy as np, pandas as pd

# 7a) Sanitize expression matrix (genes x samples, numeric float)
E = expr_sym_small.copy()

# force numeric for every cell (non-numeric -> NaN)
E = E.apply(pd.to_numeric, errors="coerce")

# drop genes that are entirely NaN across samples
E = E.loc[~E.isna().all(axis=1)]

# if any sample (column) is entirely NaN, drop it
E = E.loc[:, ~E.isna().all(axis=0)]

# per-gene mean imputation for remaining NaNs (keeps z-scoring stable)
gene_means = E.mean(axis=1)
E = E.apply(lambda col: col.fillna(gene_means), axis=0)

print("Cleaned expr shape (genes x samples):", E.shape)

# 7b) Build groups from TCGA barcode (Tumor vs Normal)
def groups_from_barcode(ids):
    def code(s):
        p = s.split('-')
        return int(p[3][:2]) if (len(p)>=4 and len(p[3])>=2 and p[3][:2].isdigit()) else None
    def bucket(c):
        if c is None: return "Other"
        if 1 <= c <= 9:   return "Tumor"
        if 10<= c <= 19:  return "Normal"
        return "Other"
    return pd.Series([bucket(code(s)) for s in ids], index=ids)

X = E.T  # samples x genes
groups = groups_from_barcode(list(X.index))

# drop zero-variance genes (after imputation some can still be flat)
std = X.std(axis=0)
X = X.loc[:, (std > 0).values]

# standardize features (genes)
mu = X.mean(axis=0).to_numpy(na_value=0.0)
sd = X.std(axis=0).to_numpy(na_value=0.0) + 1e-8
Xz = (X.to_numpy() - mu) / sd

# PCA via NumPy SVD (no sklearn needed)
U, S, VT = np.linalg.svd(Xz, full_matrices=False)
scores = U * S
var = S**2
explained = (var / var.sum()) * 100.0

print("Explained variance (%):", np.round(explained[:10], 2))

# Optional scatter plot (PC1 vs PC2). If matplotlib missing, we just print EVR.
try:
    import matplotlib.pyplot as plt
    plt.figure(figsize=(7,6))
    labels = groups.values
    for g in pd.unique(labels):
        idx = (labels == g)
        plt.scatter(scores[idx,0], scores[idx,1], label=g, alpha=0.75)
    plt.xlabel(f"PC1 ({explained[0]:.1f}% var)")
    plt.ylabel(f"PC2 ({explained[1]:.1f}% var)")
    plt.title("PCA (NumPy SVD) — TCGA Tumor vs Normal")
    plt.legend()
    plt.show()
except Exception as e:
    print("Plot skipped (matplotlib issue):", repr(e))


# **8) Pathway scoring (lightweight, no extra deps)**

In [ ]:
#@title 8a) Helpers: z-scoring & pathway scoring (mean of member z's)
import numpy as np, pandas as pd

def zscore_by_gene(expr_symbols: pd.DataFrame) -> pd.DataFrame:
    E = expr_symbols.copy()
    E = E.apply(pd.to_numeric, errors="coerce")
    # drop all-NaN genes, then impute per-gene mean for remaining NaNs
    E = E.loc[~E.isna().all(axis=1)]
    gene_means = E.mean(axis=1)
    E = E.apply(lambda col: col.fillna(gene_means), axis=0)
    mu = E.mean(axis=1)
    sd = E.std(axis=1) + 1e-8
    return (E.sub(mu, axis=0)).div(sd, axis=0)

def pathway_scores(expr_symbols: pd.DataFrame, signatures: dict) -> pd.DataFrame:
    """Return pathways x samples (mean z across member genes present)."""
    Z = zscore_by_gene(expr_symbols)
    rows = []
    for pw, genes in signatures.items():
        present = [g for g in genes if g in Z.index]
        if present:
            s = Z.loc[present].mean(axis=0)
        else:
            s = pd.Series([np.nan]*Z.shape[1], index=Z.columns)
        s.name = pw
        rows.append(s)
    return pd.DataFrame(rows)


In [ ]:
#@title 8b) Compute pathway matrix P (pathways × samples)
# Uses SIGS defined earlier in your notebook (from Section 2–6)
P = pathway_scores(expr_sym_small, SIGS)
print("Pathways x samples:", P.shape)
display(P.iloc[:5, :5])


In [ ]:
#@title 8c) (Optional) Quick visualization: small heatmap (no seaborn)
import numpy as np

# pick at most 12 pathways with highest variance to keep it light
var_by_pw = P.var(axis=1).sort_values(ascending=False)
pw_keep = list(var_by_pw.index[:12])
Ps = P.loc[pw_keep]

# z-score per pathway for viz (center rows)
Ps_vis = (Ps - Ps.mean(axis=1).values.reshape(-1,1)) / (Ps.std(axis=1).values.reshape(-1,1) + 1e-8)

try:
    import matplotlib.pyplot as plt
    plt.figure(figsize=(min(12, 0.3*Ps_vis.shape[1]+3), 6))
    plt.imshow(Ps_vis.values, aspect='auto', interpolation='nearest')
    plt.xticks(range(Ps_vis.shape[1]), Ps_vis.columns.str.slice(0,12), rotation=90)
    plt.yticks(range(Ps_vis.shape[0]), Ps_vis.index)
    plt.colorbar(label="z (within pathway)")
    plt.title("Pathway activity (subset)")
    plt.tight_layout()
    plt.show()
except Exception as e:
    print("Heatmap skipped:", repr(e))


# **9) QUBO → CUDA-Q QAOA (discrete therapy selection)**

In [ ]:
#@title 9a) Drug panel & QUBO utilities (light)
import numpy as np, pandas as pd

def example_drug_panel():
    return {
        "EGFRi": ["REACTOME_SIGNALING_BY_EGFR"],
        "ALKi":  ["REACTOME_SIGNALING_BY_ALK"],
        "MEKi":  ["REACTOME_MAPK1_MAPK3_SIGNALING"],
        "PI3Ki": ["REACTOME_PI3K_AKT_SIGNALING"],
        "mTORi": ["REACTOME_MTORC1_MEDIATED_SIGNALLING"],
        "PD1i":  ["REACTOME_PD1_SIGNALING"],
        "VEGFi": ["REACTOME_VEGFA_VEGFR2_SIGNALING_PATHWAY"],
        "FGFRi": ["REACTOME_SIGNALING_BY_FGFR"],
    }

def patient_vector(P: pd.DataFrame, sample_id: str) -> pd.Series:
    """z-normalize pathways across samples; return vector for one patient."""
    z = (P - P.mean(axis=1).values.reshape(-1,1)) / (P.std(axis=1).values.reshape(-1,1) + 1e-8)
    return z[sample_id].fillna(0.0)

def drug_benefit_prior(z_path: pd.Series, panel: dict) -> pd.Series:
    """Aggregate pathway z's per drug (ReLU to emphasize upregulated pathways)."""
    s = pd.Series({d: float(np.sum([max(z_path.get(p, 0.0), 0.0) for p in pws])) for d, pws in panel.items()})
    return s / s.max() if s.max() > 0 else s

def build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10):
    """Pairwise penalties for overlapping mechanisms; diagonal = sparsity."""
    K = len(drugs); R = np.zeros((K, K), dtype=float)
    for i in range(K):
        for j in range(i+1, K):
            overlap = len(set(panel[drugs[i]]) & set(panel[drugs[j]]))
            if overlap > 0:
                R[i, j] = R[j, i] = base_overlap * overlap
    for i in range(K):
        R[i, i] += sparsity
    return R

def build_qubo(b_hat, R, lam=1.0):
    """QUBO: minimize x^T (lam R) x + q^T x  where q = -b̂ + diag(lam R)."""
    Q = lam * R.copy()
    q = -b_hat.copy()
    diag = np.diag(Q).copy()
    np.fill_diagonal(Q, 0.0)  # keep off-diagonal in Q
    q += diag
    return q, np.triu(Q, 1)

def qubo_to_ising(q, Q):
    """Map QUBO to Ising E(z)=h·z + z^T J z  (z ∈ {±1})."""
    K = len(q); Qf = Q + Q.T
    h = np.zeros(K); J = np.zeros((K, K)); const = 0.0
    for i in range(K):
        const += 0.5 * q[i]
        h[i] += -0.5 * q[i]
    for i in range(K):
        for j in range(i+1, K):
            Qij = Qf[i, j]
            const += 0.25 * Qij
            h[i]  += -0.25 * Qij
            h[j]  += -0.25 * Qij
            J[i, j] += 0.25 * Qij
    return h, J, const


In [ ]:
#@title 9b) CUDA-Q QAOA solver (p small; safe fallback built-in)
# Will attempt to import cudaq; if not present, sets CUDAQ_OK=False
CUDAQ_OK = True
try:
    import cudaq  # type: ignore
    from cudaq import spin  # type: ignore
except Exception:
    CUDAQ_OK = False
    print("⚠️ CUDA-Q not available; you can still inspect QUBO or plug a classical solver.")

def qaoa_solve(h, J, p=2, shots=2048, max_iters=60, seed=7):
    """Return dict with best_state bitstring and counts; None if CUDA-Q missing."""
    if not CUDAQ_OK:
        return None
    import numpy as np, cudaq
    from cudaq import spin

    def build_H(h,J):
        H = 0.0 * spin.z(0); H = 0.0 * H
        K=len(h)
        for i in range(K):
            if abs(h[i])>0: H += h[i]*spin.z(i)
        for i in range(K):
            for j in range(i+1,K):
                if abs(J[i,j])>1e-12: H += J[i,j]*spin.z(i)*spin.z(j)
        return H

    H = build_H(h, J); K=len(h)

    @cudaq.kernel
    def ansatz(params: list[float]):
        q = cudaq.qvector(K)
        for i in range(K): cudaq.h(q[i])
        for layer in range(p):
            gamma = params[layer]
            for i in range(K):
                if abs(h[i])>0: cudaq.rz(2.0*gamma*h[i], q[i])
            for i in range(K):
                for j in range(i+1,K):
                    if abs(J[i,j])>1e-12:
                        cudaq.cx(q[i], q[j]); cudaq.rz(2.0*gamma*J[i,j], q[j]); cudaq.cx(q[i], q[j])
            beta = params[p+layer]
            for i in range(K): cudaq.rx(2.0*beta, q[i])

    def objective(params):
        return cudaq.observe(ansatz, H, params, shots_count=shots).expectation()

    np.random.seed(seed)
    x0 = np.random.uniform(0,1,2*p).tolist()
    try:
        res = cudaq.optimize(objective, x0=x0, max_eval=max_iters)
        params_opt = res.optimal_parameters
    except Exception:
        # tiny random search fallback
        grid=[np.random.uniform(0,1,2*p) for _ in range(48)]
        vals=[objective(g.tolist()) for g in grid]
        params_opt = grid[int(np.argmin(vals))].tolist()

    counts = cudaq.sample(ansatz, params_opt, shots_count=shots)
    # choose lowest-energy state from samples
    best_state=None; best_energy=float("inf")
    for bitstring, c in counts.items():
        z = np.array([1 if b=='0' else -1 for b in bitstring[::-1]])
        e = float(np.dot(h, z) + sum(J[i,j]*z[i]*z[j] for i in range(len(z)) for j in range(i+1,len(z))))
        if e < best_energy:
            best_energy, best_state = e, bitstring
    return {"best_state": best_state, "counts": dict(counts)}


In [ ]:
#@title 9b) Robust solvers: exact QUBO (fast for K≤20) + optional CUDA-Q
import numpy as np, pandas as pd

def exact_qubo_solve(b_hat: np.ndarray, R: np.ndarray, lam: float = 1.0):
    """
    Exact minimization of QUBO:
      E(x) = x^T (lam R) x + q^T x, with q = -b̂ + diag(lam R)
    We enumerate all bitstrings (2^K). Returns best bitstring (as 0/1 np array).
    """
    K = len(b_hat)
    Q = lam * R.copy()
    q = -b_hat.copy() + np.diag(Q)
    np.fill_diagonal(Q, 0.0)  # keep only off-diagonal in Q

    best_e = np.inf
    best_x = None
    # vectorize partial precomputations
    upper_idx = np.triu_indices(K, 1)
    for mask in range(1 << K):
        # build x from bits
        x = np.fromiter(((mask >> i) & 1 for i in range(K)), dtype=np.int8)
        # E = x^T Q x + q^T x, where Q is strictly upper-triangular mirrored
        e = np.dot(q, x) + 2.0 * np.sum(Q[upper_idx] * (x[upper_idx[0]] * x[upper_idx[1]]))
        if e < best_e:
            best_e, best_x = e, x
    return best_x, float(best_e)

# Optional: CUDA-Q runner (only used if your install exposes expected API)
def try_cudaq_qaoa(h, J, p=2, shots=2048, max_iters=60, seed=7):
    try:
        import cudaq  # type: ignore
        from cudaq import spin  # type: ignore
    except Exception:
        return None  # CUDA-Q not available

    # Different wheels expose different APIs; we wrap cautiously
    K = len(h)
    try:
        # Build Ising H = h·Z + sum J_ij Z_i Z_j
        H = 0.0 * spin.z(0); H = 0.0 * H
        for i in range(K):
            if abs(h[i]) > 0: H += h[i] * spin.z(i)
        for i in range(K):
            for j in range(i+1, K):
                if abs(J[i, j]) > 0: H += J[i, j] * spin.z(i) * spin.z(j)

        # If your wheel has a high-level QAOA interface, try to use it:
        try:
            from cudaq.algorithms import QAOA  # type: ignore
            qaoa = QAOA(H, steps=p)
            res = qaoa.minimize()  # may not exist in some builds
            # res may return bitstring directly in some builds; we normalize
            bitstring = getattr(res, "bitstring", None)
            if bitstring is None and isinstance(res, dict):
                bitstring = res.get("bitstring")
            if bitstring:
                return {"best_state": bitstring, "counts": {}}
        except Exception:
            pass

        # Fallback: simple parameter sweep (small) via observe/sample if present
        import numpy as np
        @cudaq.kernel
        def ansatz(params: list[float]):
            q = cudaq.qvector(K)
            # Some builds don’t expose `cudaq.h`; try ry(π/2) as H-equivalent
            for i in range(K):
                try:
                    cudaq.h(q[i])          # if available
                except Exception:
                    cudaq.ry(np.pi/2, q[i])
            for layer in range(p):
                gamma = params[layer]
                for i in range(K):
                    # phase separation via rz on Z terms
                    cudaq.rz(2.0*gamma*h[i], q[i])
                for i in range(K):
                    for j in range(i+1, K):
                        if abs(J[i,j]) > 0:
                            cudaq.cx(q[i], q[j]); cudaq.rz(2.0*gamma*J[i,j], q[j]); cudaq.cx(q[i], q[j])
                beta = params[p+layer]
                for i in range(K):
                    cudaq.rx(2.0*beta, q[i])

        def energy_from_bitstring(bits):
            z = np.array([1 if b=='0' else -1 for b in bits[::-1]])
            return float(np.dot(h, z) + sum(J[i,j]*z[i]*z[j] for i in range(K) for j in range(i+1,K)))

        # tiny random search since `cudaq.optimize` may not exist
        rng = np.random.default_rng(seed)
        best_state, best_e = None, np.inf
        for _ in range(64):
            params = rng.random(2*p).tolist()
            try:
                counts = cudaq.sample(ansatz, params, shots_count=shots)
            except Exception:
                return None  # give up gracefully
            for bitstring, c in counts.items():
                e = energy_from_bitstring(bitstring)
                if e < best_e:
                    best_e, best_state = e, bitstring
        if best_state:
            return {"best_state": best_state, "counts": {}}
        return None
    except Exception:
        return None


In [ ]:
#@title 9c) Build QUBO → run (CUDA-Q if available else exact)
import numpy as np, pandas as pd

panel = example_drug_panel()
drugs = list(panel.keys())

# pick a patient (first column for demo)
sample_id = P.columns[0]
z_path = patient_vector(P, sample_id)

b_series = drug_benefit_prior(z_path, panel).reindex(drugs).fillna(0.0)
R = build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10)

b = b_series.to_numpy(float)
q, Q = build_qubo(b, R, lam=1.0)

# Ising mapping (for CUDA-Q path)
h, J, const = qubo_to_ising(q, Q)

# Try CUDA-Q first
res = try_cudaq_qaoa(h, J, p=2, shots=2048, max_iters=60, seed=7)
if res and res.get("best_state"):
    bitstring = res["best_state"]
    sel = [drugs[i] for i, bit in enumerate(bitstring[::-1]) if bit == '1']
    solver_used = "CUDA-Q QAOA"
else:
    # exact QUBO fallback (guaranteed optimum for K≤20)
    x_star, e_star = exact_qubo_solve(b, R, lam=1.0)
    sel = [drugs[i] for i, xi in enumerate(x_star) if xi == 1]
    solver_used = "Exact QUBO (enumeration)"

summary = pd.DataFrame({
    "drug": drugs,
    "b_hat": b_series.values,
    "selected": [d in sel for d in drugs],
    "targets": [", ".join(panel[d]) for d in drugs]
}).sort_values("b_hat", ascending=False).reset_index(drop=True)

print("Patient:", sample_id)
print("Solver:", solver_used)
print("Selected therapies:", sel if sel else "(none)")
display(summary)


# **10. Multi-patient + Stability******

In [ ]:
#@title 🔄 Multi-patient QUBO run + stability summary
import pandas as pd, numpy as np
from collections import Counter

panel = example_drug_panel()
drugs = list(panel.keys())

def run_for_patient(sample_id):
    z_path = patient_vector(P, sample_id)
    b_series = drug_benefit_prior(z_path, panel).reindex(drugs).fillna(0.0)
    R = build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10)
    b = b_series.to_numpy(float)
    q, Q = build_qubo(b, R, lam=1.0)
    # exact solve (stable & fast)
    x_star, e_star = exact_qubo_solve(b, R, lam=1.0)
    sel = [drugs[i] for i, xi in enumerate(x_star) if xi == 1]
    return sel, b_series

# --- run over first N patients
N = 25
all_sels = []
all_bhats = []

for sid in P.columns[:N]:
    sel, b_series = run_for_patient(sid)
    all_sels.append(sel)
    all_bhats.append(b_series)

# --- frequency summary
flat = [d for sel in all_sels for d in sel]
freq = Counter(flat)
freq_df = pd.DataFrame({"drug": drugs, "frequency": [freq[d] for d in drugs]})
freq_df["frequency_pct"] = 100 * freq_df["frequency"] / N
freq_df = freq_df.sort_values("frequency_pct", ascending=False).reset_index(drop=True)

print(f"Ran {N} patients")
display(freq_df)


# **11. Visualization**

In [ ]:
#@title 📊 Drug selection frequency barplot
try:
    import matplotlib.pyplot as plt
    plt.figure(figsize=(7,5))
    plt.bar(freq_df["drug"], freq_df["frequency_pct"], color="steelblue")
    plt.ylabel("% patients selected")
    plt.title("Drug selection stability across patients")
    plt.xticks(rotation=45)
    plt.show()
except Exception as e:
    print("Plot skipped:", repr(e))


# **12. Hierarchical Clustering: Heatmap, Dendrogram, and Clustered Heatmap**

In [ ]:
#@title 🔗 Co-selection analysis (rebuild if needed) + heatmap + dendrogram
import numpy as np, pandas as pd

# --- prerequisites: panel, P, and the helper funcs from Section 9a must exist ---
# If all_sels/drugs are missing, rebuild them quickly.
needs_run = ("all_sels" not in globals()) or ("drugs" not in globals())
if needs_run:
    panel = example_drug_panel()
    drugs = list(panel.keys())

    def run_for_patient(sample_id):
        z_path = patient_vector(P, sample_id)
        b_series = drug_benefit_prior(z_path, panel).reindex(drugs).fillna(0.0)
        R = build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10)
        b = b_series.to_numpy(float)
        x_star, e_star = exact_qubo_solve(b, R, lam=1.0)
        sel = [drugs[i] for i, xi in enumerate(x_star) if xi == 1]
        return sel

    N = min(25, P.shape[1])
    all_sels = [run_for_patient(sid) for sid in P.columns[:N]]

# --- build co-selection counts ---
co_mat = pd.DataFrame(0, index=drugs, columns=drugs, dtype=int)
for sel in all_sels:
    # ensure unique set per patient (avoid double counting same drug)
    uniq = list(dict.fromkeys(sel))
    for i in range(len(uniq)):
        for j in range(i, len(uniq)):
            di, dj = uniq[i], uniq[j]
            co_mat.loc[di, dj] += 1
            if i != j:
                co_mat.loc[dj, di] += 1

# normalize to % of patients
n_pat = max(1, len(all_sels))
co_pct = co_mat / n_pat * 100.0

print("Co-selection matrix (% of patients):")
display(co_pct.round(1))

# --- heatmap ---
try:
    import matplotlib.pyplot as plt
    plt.figure(figsize=(7,6))
    im = plt.imshow(co_pct.values, cmap="Blues", interpolation="nearest")
    plt.xticks(range(len(drugs)), drugs, rotation=45, ha="right")
    plt.yticks(range(len(drugs)), drugs)
    plt.colorbar(im, label="% patients co-selected")
    plt.title("Drug co-selection heatmap")
    plt.tight_layout()
    plt.show()
except Exception as e:
    print("Heatmap skipped:", repr(e))

# --- dendrogram + clustered heatmap (requires scipy) ---
try:
    from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list

    # distance = 1 - correlation on rows
    corr = np.corrcoef(co_pct.values)
    # safety: numeric issues -> clip to [-1,1]
    corr = np.clip(corr, -1.0, 1.0)
    dist = 1.0 - corr

    Z = linkage(dist, method="average")
    # dendrogram
    plt.figure(figsize=(8,5))
    dendrogram(Z, labels=drugs, leaf_rotation=90, leaf_font_size=10,
               color_threshold=0.7 * np.max(Z[:,2]))
    plt.title("Clustered dendrogram of drug co-selection")
    plt.ylabel("Distance (1 - correlation)")
    plt.tight_layout()
    plt.show()

    # clustered heatmap
    order = leaves_list(Z)
    co_pct_ordered = co_pct.iloc[order, order]

    plt.figure(figsize=(7,6))
    im = plt.imshow(co_pct_ordered.values, cmap="Blues", interpolation="nearest")
    plt.xticks(range(len(order)), co_pct_ordered.columns, rotation=45, ha="right")
    plt.yticks(range(len(order)), co_pct_ordered.index)
    plt.colorbar(im, label="% patients co-selected")
    plt.title("Clustered heatmap of drug co-selection")
    plt.tight_layout()
    plt.show()

    display(co_pct_ordered.round(1))
except Exception as e:
    print("Clustering skipped (scipy missing or version issue):", repr(e))


# **13. Top co-selected pairs (ranked)**

In [ ]:
#@title 📈 Top co-selected drug pairs (ranked table)
import itertools, pandas as pd, numpy as np

# rebuild pair counts from all_sels to be safe
pair_counts = {}
N_pat = max(1, len(all_sels))
for sel in all_sels:
    uniq = sorted(set(sel))
    for (a,b) in itertools.combinations(uniq, 2):
        pair_counts[(a,b)] = pair_counts.get((a,b), 0) + 1

pairs_df = pd.DataFrame(
    [(a,b,c, 100.0*c/N_pat) for (a,b),c in pair_counts.items()],
    columns=["drug_a","drug_b","count","pct_patients"]
).sort_values(["pct_patients","count"], ascending=False, ignore_index=True)

print(f"Pairs observed: {len(pairs_df)}  (N patients={N_pat})")
display(pairs_df.head(15))


# **14. Patient-level report (what each patient got + pathway context)**

In [ ]:
#@title 🧾 Patient-level report (selected drugs + pathway context)
import pandas as pd, numpy as np

panel = example_drug_panel()
drugs = list(panel.keys())

def run_patient_once(sample_id):
    z_path = patient_vector(P, sample_id)
    b_series = drug_benefit_prior(z_path, panel).reindex(drugs).fillna(0.0)
    R = build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10)
    x_star, e_star = exact_qubo_solve(b_series.to_numpy(float), R, lam=1.0)
    sel = [drugs[i] for i, xi in enumerate(x_star) if xi == 1]
    return sel, b_series, z_path

rows = []
for sid in P.columns[:len(all_sels)]:  # align with prior run size
    sel, b_hat, z_path = run_patient_once(sid)
    # top pathways (absolute z) for context
    top_pw = z_path.abs().sort_values(ascending=False).head(5)
    rows.append({
        "patient": sid,
        "selected_drugs": ", ".join(sel) if sel else "(none)",
        "top_pathways": "; ".join([f"{p}:{z_path[p]:+.2f}" for p in top_pw.index]),
        **{f"b̂.{d}": float(b_hat.get(d,0.0)) for d in drugs}
    })

patient_report = pd.DataFrame(rows)
display(patient_report.head(10))

# optional export
out_csv = "/content/patient_report.csv"
patient_report.to_csv(out_csv, index=False)
print("Saved:", out_csv)


# **15. Selection stability summary (frequencies, co-selection, size distribution)**

In [ ]:
#@title 🧮 Stability summary (freqs + selection size histogram)
import pandas as pd, numpy as np
from collections import Counter

# frequencies (reuse or recompute)
flat = [d for sel in all_sels for d in sel]
freq = Counter(flat)
freq_df = pd.DataFrame({"drug": drugs, "freq": [freq[d] for d in drugs]})
freq_df["pct"] = 100.0 * freq_df["freq"] / max(1,len(all_sels))
freq_df = freq_df.sort_values("pct", ascending=False).reset_index(drop=True)
display(freq_df)

# selection size distribution
sizes = [len(set(sel)) for sel in all_sels]
size_hist = pd.Series(sizes).value_counts().sort_index()
print("Selection size distribution (unique drugs per patient):")
display(pd.DataFrame({"k_drugs": size_hist.index, "n_patients": size_hist.values}))

# quick plots (skip if matplotlib missing)
try:
    import matplotlib.pyplot as plt
    plt.figure(figsize=(7,4))
    plt.bar(freq_df["drug"], freq_df["pct"])
    plt.ylabel("% patients selected")
    plt.title("Drug selection frequency")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout(); plt.show()

    plt.figure(figsize=(5,4))
    plt.bar(size_hist.index, size_hist.values)
    plt.xlabel("# drugs selected")
    plt.ylabel("# patients")
    plt.title("Selection size distribution")
    plt.tight_layout(); plt.show()
except Exception as e:
    print("Plots skipped:", repr(e))


# **16. Code-cell: Synthetic Classification Validation**


The classification-style validation on top of the survival proxy has been added.
Because the notebook does not have a ground-truth patient response labels (as of the moment of the analysis),  we simulated binary risk classes (high vs low) from pathway signals, then test whether drug selections can predict them.

In [ ]:
#@title 🧪 Classification validation (synthetic labels → ROC/PR metrics)
import numpy as np, pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

panel = example_drug_panel()
drugs = list(panel.keys())

# 1) Construct patient-drug feature matrix
N = len(all_sels)
X = np.zeros((N, len(drugs)))
for i, sel in enumerate(all_sels):
    for d in sel:
        if d in drugs:
            X[i, drugs.index(d)] = 1

# 2) Synthetic binary labels
# Risk score = mean pathway z → label = high (1) if above median
risk_scores = []
for sid in P.columns[:N]:
    z_path = patient_vector(P, sid)
    risk_scores.append(z_path.mean())
risk_scores = np.array(risk_scores)
y = (risk_scores > np.median(risk_scores)).astype(int)

print("Synthetic labels (0=low risk, 1=high risk):")
print(np.bincount(y))

# 3) Train simple logistic regression
clf = LogisticRegression(max_iter=200)
clf.fit(X, y)
y_pred = clf.predict_proba(X)[:,1]

# 4) Metrics
roc_auc = roc_auc_score(y, y_pred)
pr_auc = average_precision_score(y, y_pred)

print(f"ROC-AUC: {roc_auc:.3f}")
print(f"PR-AUC: {pr_auc:.3f}")

# Optional barplot of learned coefficients
try:
    import matplotlib.pyplot as plt
    coefs = pd.Series(clf.coef_[0], index=drugs).sort_values()
    plt.figure(figsize=(7,4))
    coefs.plot(kind="barh", color=["steelblue" if v>0 else "salmon" for v in coefs])
    plt.title("Drug selection coefficients (synthetic label prediction)")
    plt.xlabel("Weight")
    plt.tight_layout()
    plt.show()
except Exception as e:
    print("Plot skipped:", repr(e))


# **Summary:**

This notebook presented a comprehensive workflow demonstrating the application of the "Medical-Patient Digital Twin" concept to personalized cancer therapy selection. Starting with raw gene expression data, we showed how to build a computational representation of a patient's biological state, specifically focusing on key signaling pathways.

The core of the approach involved formulating the drug selection problem as a Quadratic Unconstrained Binary Optimization (QUBO) problem, balancing the potential therapeutic benefit (derived from pathway activity scores) with penalties for overlapping drug mechanisms. We explored solving this optimization problem using both an exact classical method and an optional quantum Approximate Optimization Algorithm (QAOA) using CUDA-Q, highlighting the potential for leveraging advanced computing paradigms.

Beyond individual patient selection, the notebook included analysis of drug selection patterns across a cohort of patients, examining the frequency and co-selection of different therapies. We also generated a patient-level report to summarize individual therapy recommendations and their pathway context. Finally, a synthetic classification validation was performed to illustrate how the features derived from the selected drug profiles could potentially be used for predictive modeling, although the inherent limitations of this synthetic approach were emphasized.

In essence, this notebook serves as a proof-of-concept, illustrating how integrating biological data, pathway analysis, and optimization techniques within a digital twin framework can provide a powerful computational tool to inform personalized cancer therapy. While acknowledging the necessary simplifications and the critical need for rigorous clinical validation, this work demonstrates the potential of such computational approaches to contribute to the future of precision medicine.

# Task
Convert the provided Jupyter notebook into a web application.

## Identify core logic

### Subtask:
Extract the key Python functions and classes from the notebook that perform the data processing, pathway scoring, QUBO formulation, and drug selection.


## Choose a web framework

### Subtask:
Select a suitable Python web framework (e.g., Flask, Django, or Streamlit) to build the web application. Streamlit is often a good choice for data science applications due to its ease of use.


## Create a user interface

### Subtask:
Design and implement the front-end of the web application using the chosen framework (Streamlit). This will involve creating input fields for uploading gene expression data, selecting parameters (e.g., number of samples, lambda value), and displaying the results (e.g., selected drugs, pathway scores, visualizations).


**Reasoning**:
Create a new Streamlit script `app.py` and add the basic structure including title, description, file uploader, parameter inputs for N_SAMPLES and lambda, and placeholders for results.



In [ ]:
%pip install streamlit

**Reasoning**:
Now that Streamlit is installed, recreate the Streamlit application script with the basic UI elements as described in the instructions.



In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
# Assume the core logic functions from the notebook are available,
# either by importing them from a separate file or including them here.
# For this step, we only set up the UI elements.

st.set_page_config(layout="wide")

st.title("Digital Twin Workflow: Gene Expression to Drug Selection")

st.markdown("""
This application demonstrates a workflow for identifying potential drug therapies
for cancer patients based on their gene expression data, using pathway analysis
and QUBO optimization.
""")

st.sidebar.header("Input Data")
uploaded_file = st.sidebar.file_uploader("Upload gene expression data (CSV or TSV)", type=["csv", "tsv", "gz"])

st.sidebar.header("Parameters")
n_samples = st.sidebar.slider("Number of samples to process", min_value=10, max_value=100, value=40, step=10)
lam_value = st.sidebar.slider("Lambda (penalty) value for QUBO", min_value=0.1, max_value=5.0, value=1.0, step=0.1)

st.header("Analysis Results")

# Placeholders for results
results_placeholder = st.empty()
pathway_scores_placeholder = st.empty()
selected_drugs_placeholder = st.empty()
visualizations_placeholder = st.empty()

if uploaded_file is not None:
    try:
        # In a real app, you would load and process the data here
        # For now, just acknowledge the upload
        st.sidebar.success("File uploaded successfully!")
        st.write("Click 'Run Analysis' to process the data and get results.")

        # Add a button to trigger analysis
        if st.button("Run Analysis"):
            # This is where the data processing and analysis functions would be called
            # For this step, we just show a message
            with st.spinner("Running analysis..."):
                # Simulate analysis time
                import time
                time.sleep(3)
                results_placeholder.write("Analysis complete! Results will appear below.")
                # In a real app, populate the placeholders with actual results
                # pathway_scores_placeholder.dataframe(...)
                # selected_drugs_placeholder.write(...)
                # visualizations_placeholder.pyplot(...)

    except Exception as e:
        st.error(f"Error loading or processing file: {e}")
else:
    st.info("Please upload a gene expression data file to begin.")


## Connect backend logic

### Subtask:
Integrate the extracted Python functions into the web framework (Streamlit) to handle user requests. This will involve setting up routes or endpoints to receive data from the front-end, process it using the core logic, and return the results.


**Reasoning**:
Define a function `run_analysis` that encapsulates the core workflow, including data loading, processing, pathway scoring, QUBO formulation, and drug selection. This function will take the uploaded file, number of samples, and lambda value as input.



In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import gzip
import re
from collections import Counter
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
import matplotlib.pyplot as plt
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
import io

# Assume the core logic functions from the notebook are available here.
# For demonstration, I'll include them directly. In a larger project,
# these would be in a separate module.

def norm_ensembl(x): return x.split('.',1)[0]
def norm_symbol(x):  return re.sub(r'[^A-Za-z0-9_-]+','', x)

def detect_row_mode(file_object, scan_rows=50000):
    """Detect row ID type (Ensembl vs symbol) from file-like object."""
    seen = Counter(); total = 0
    # Use a TextIOWrapper to treat the bytes stream as text
    with io.TextIOWrapper(file_object, encoding="utf-8", errors="replace") as f:
        # Read the header line to skip it for row detection
        header = f.readline()
        reader = pd.read_csv(f, sep="\t", chunksize=200_000, usecols=[0], dtype=str, header=None) # header=None because we already read it
        for ch in reader:
            v = ch.iloc[:,0].astype(str)
            vals = v.head(min(len(v), scan_rows-total)).tolist()
            total += len(vals)
            seen.update('ENSG' if x.startswith('ENSG') else 'OTHER' for x in vals)
            if total >= scan_rows: break
    ratio = seen['ENSG']/max(1,(seen['ENSG']+seen['OTHER']))
    mode = 'ensembl' if ratio >= 0.6 else 'symbol'
    return mode

def stream_select_rows_columns(file_object, selected_samples, sig_genes, sym2ensg, row_mode, gene_col_name="Name"):
    """Stream-select rows and columns from file-like object."""
    if row_mode == 'ensembl':
        target_rows = set(sym2ensg.get(g) for g in sig_genes if g in sym2ensg)
        normalize = norm_ensembl
    else:
        target_rows = set(sig_genes)
        normalize = norm_symbol

    usecols_filter = [gene_col_name] + selected_samples
    kept = []

    file_object.seek(0) # Reset file pointer to the beginning

    with io.TextIOWrapper(file_object, encoding="utf-8", errors="replace") as f:
        # Read header to find the actual gene_col_name if it's not "Name"
        header = f.readline().rstrip("\n").split("\t")
        try:
            gene_col_idx = header.index(gene_col_name)
        except ValueError:
             # If "Name" not found, assume the first column is the gene column
            gene_col_name = header[0]
            gene_col_idx = 0

        reader = pd.read_csv(f, sep="\t", chunksize=50_000, dtype=str, header=None) # header=None because we already read it
        for i, ch in enumerate(reader):
            # Assign correct column names based on header
            if i == 0:
                 ch.columns = header
            else:
                 # For subsequent chunks, pandas might not automatically use the header,
                 # so we need to ensure we are selecting the correct columns by index
                 ch.columns = header # Assign header to the chunk
                 ch = ch[usecols_filter] # Select only necessary columns

            ch = ch.rename(columns={gene_col_name: "row_id"})
            # Ensure 'row_id' column exists after renaming
            if 'row_id' not in ch.columns:
                 st.error(f"Error: Could not find gene identifier column '{gene_col_name}' in the uploaded file.")
                 return pd.DataFrame() # Return empty DataFrame on error

            ids = ch["row_id"].astype(str).map(normalize)
            mask = ids.isin(target_rows)

            if mask.any():
                out = ch.loc[mask].copy()
                out["row_id"] = out["row_id"].map(normalize)
                kept.append(out)

    if not kept:
        st.warning("No signature rows matched the provided gene list.")
        return pd.DataFrame()

    expr_small = pd.concat(kept, axis=0, ignore_index=False).drop_duplicates(subset=["row_id"]).set_index("row_id")

    if row_mode == 'ensembl':
        ensg2sym = {v:k for k,v in sym2ensg.items()}
        expr_sym_small = expr_small.copy()
        expr_sym_small.index = [ensg2sym.get(e, e) for e in expr_small.index]
    else:
        expr_sym_small = expr_small.copy()

    expr_sym_small = expr_sym_small[~expr_sym_small.index.duplicated(keep="first")]
    return expr_sym_small


def zscore_by_gene(expr_symbols: pd.DataFrame) -> pd.DataFrame:
    E = expr_symbols.copy()
    E = E.apply(pd.to_numeric, errors="coerce")
    E = E.loc[~E.isna().all(axis=1)]
    gene_means = E.mean(axis=1)
    E = E.apply(lambda col: col.fillna(gene_means), axis=0)
    mu = E.mean(axis=1)
    sd = E.std(axis=1) + 1e-8
    return (E.sub(mu, axis=0)).div(sd, axis=0)

def pathway_scores(expr_symbols: pd.DataFrame, signatures: dict) -> pd.DataFrame:
    """Return pathways x samples (mean z across member genes present)."""
    Z = zscore_by_gene(expr_symbols)
    rows = []
    for pw, genes in signatures.items():
        present = [g for g in genes if g in Z.index]
        if present:
            s = Z.loc[present].mean(axis=0)
        else:
            s = pd.Series([np.nan]*Z.shape[1], index=Z.columns)
        s.name = pw
        rows.append(s)
    return pd.DataFrame(rows)

def example_drug_panel():
    return {
        "EGFRi": ["REACTOME_SIGNALING_BY_EGFR"],
        "ALKi":  ["REACTOME_SIGNALING_BY_ALK"],
        "MEKi":  ["REACTOME_MAPK1_MAPK3_SIGNALING"],
        "PI3Ki": ["REACTOME_PI3K_AKT_SIGNALING"],
        "mTORi": ["REACTOME_MTORC1_MEDIATED_SIGNALLING"],
        "PD1i":  ["REACTOME_PD1_SIGNALING"],
        "VEGFi": ["REACTOME_VEGFA_VEGFR2_SIGNALING_PATHWAY"],
        "FGFRi": ["REACTOME_SIGNALING_BY_FGFR"],
    }

def patient_vector(P: pd.DataFrame, sample_id: str) -> pd.Series:
    """z-normalize pathways across samples; return vector for one patient."""
    z = (P - P.mean(axis=1).values.reshape(-1,1)) / (P.std(axis=1).values.reshape(-1,1) + 1e-8)
    return z[sample_id].fillna(0.0)

def drug_benefit_prior(z_path: pd.Series, panel: dict) -> pd.Series:
    """Aggregate pathway z's per drug (ReLU to emphasize upregulated pathways)."""
    s = pd.Series({d: float(np.sum([max(z_path.get(p, 0.0), 0.0) for p in pws])) for d, pws in panel.items()})
    return s / s.max() if s.max() > 0 else s

def build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10):
    """Pairwise penalties for overlapping mechanisms; diagonal = sparsity."""
    K = len(drugs); R = np.zeros((K, K), dtype=float)
    for i in range(K):
        for j in range(i+1, K):
            overlap = len(set(panel[drugs[i]]) & set(panel[drugs[j]]))
            if overlap > 0:
                R[i, j] = R[j, i] = base_overlap * overlap
    for i in range(K):
        R[i, i] += sparsity
    return R

def build_qubo(b_hat, R, lam=1.0):
    """QUBO: minimize x^T (lam R) x + q^T x  where q = -b̂ + diag(lam R)."""
    Q = lam * R.copy()
    q = -b_hat.copy()
    diag = np.diag(Q).copy()
    np.fill_diagonal(Q, 0.0)  # keep off-diagonal in Q
    q += diag
    return q, np.triu(Q, 1)

def exact_qubo_solve(b_hat: np.ndarray, R: np.ndarray, lam: float = 1.0):
    """
    Exact minimization of QUBO:
      E(x) = x^T (lam R) x + q^T x, with q = -b̂ + diag(lam R)
    We enumerate all bitstrings (2^K). Returns best bitstring (as 0/1 np array).
    """
    K = len(b_hat)
    Q = lam * R.copy()
    q = -b_hat.copy() + np.diag(Q)
    np.fill_diagonal(Q, 0.0)  # keep only off-diagonal in Q

    best_e = np.inf
    best_x = None
    # vectorize partial precomputations
    upper_idx = np.triu_indices(K, 1)
    for mask in range(1 << K):
        # build x from bits
        x = np.fromiter(((mask >> i) & 1 for i in range(K)), dtype=np.int8)
        # E = x^T Q x + q^T x, where Q is strictly upper-triangular mirrored
        e = np.dot(q, x) + 2.0 * np.sum(Q[upper_idx] * (x[upper_idx[0]] * x[upper_idx[1]]))
        if e < best_e:
            best_e, best_x = e, x
    return best_x, float(best_e)


# Placeholder for CUDA-Q function if needed later, currently uses exact solve
def try_cudaq_qaoa(h, J, p=2, shots=2048, max_iters=60, seed=7):
    return None # Not implemented in this web app version

# --- Main analysis function ---
def run_analysis(uploaded_file, n_samples, lam_value):
    """
    Runs the full analysis workflow for the uploaded data.
    Returns pathway scores, drug selection summary, and patient report.
    """
    SIGS = {
        "REACTOME_SIGNALING_BY_EGFR": [
            "EGFR","ERBB2","ERBB3","GRB2","SOS1","SHC1","PTPN11","KRAS","NRAS","HRAS",
            "BRAF","MAP2K1","MAP2K2","MAPK1","MAPK3","PLCG1","PIK3CA","PIK3R1","AKT1","AKT2","AKT3","GAB1"
        ],
        "REACTOME_SIGNALING_BY_ALK": [
            "ALK","EML4","GRB2","SHC1","PIK3CA","PIK3R1","AKT1","AKT2","AKT3","STAT3","MAP2K1","MAPK1","MAPK3"
        ],
        "REACTOME_MAPK1_MAPK3_SIGNALING": [
            "BRAF","RAF1","MAP2K1","MAP2K2","MAPK1","MAPK3","DUSP6","DUSP4","FOS","JUN","EGFR"
        ],
        "REACTOME_PI3K_AKT_SIGNALING": [
            "PIK3CA","PIK3CB","PIK3CD","PIK3R1","PIK3R2","AKT1","AKT2","AKT3","PTEN","MTOR","RHEB"
        ],
        "REACTOME_MTORC1_MEDIATED_SIGNALLING": [
            "MTOR","RPTOR","MLST8","RHEB","TSC1","TSC2","EIF4EBP1","RPS6KB1","RPS6"
        ],
        "REACTOME_PD1_SIGNALING": [
            "PDCD1","CD274","PDCD1LG2","PDCD1LG2","JAK1","JAK2","STAT1","IFNG","GZMB","LAG3","TIGIT","CXCL9","CXCL10"
        ],
        "REACTOME_VEGFA_VEGFR2_SIGNALING_PATHWAY": [
            "VEGFA","KDR","FLT1","FLT4","PTPRB","PLCG1","MAP2K1","MAPK1","NOS3"
        ],
        "REACTOME_SIGNALING_BY_FGFR": [
            "FGFR1","FGFR2","FGFR3","FGFR4","FRS2","PLCG1","PIK3CA","PIK3R1","MAP2K1","MAPK1"
        ]
    }
    SIG_GENES = sorted({g for gs in SIGS.values() for g in gs})
    SYM2ENSG = {
        "EGFR":"ENSG00000146648","ERBB2":"ENSG00000141736","ERBB3":"ENSG00000065361","GRB2":"ENSG00000177885",
        "SOS1":"ENSG00000115904","SHC1":"ENSG00000154639","PTPN11":"ENSG00000179295","KRAS":"ENSG00000133703",
        "NRAS":"ENSG00000213281","HRAS":"ENSG00000174775","BRAF":"ENSG00000157764","MAP2K1":"ENSG00000169032",
        "MAP2K2":"ENSG00000126934","MAPK1":"ENSG00000100030","MAPK3":"ENSG00000102882","PLCG1":"ENSG00000124181",
        "PIK3CA":"ENSG00000121879","PIK3R1":"ENSG00000145675","AKT1":"ENSG00000142208","AKT2":"ENSG00000105221",
        "AKT3":"ENSG00000117020","GAB1":"ENSG00000117676",
        "ALK":"ENSG00000171094","EML4":"ENSG00000143924","STAT3":"ENSG00000168610",
        "DUSP6":"ENSG00000139318","DUSP4":"ENSG00000120875","FOS":"ENSG00000170345","JUN":"ENSG00000177606","RAF1":"ENSG00000132155",
        "PIK3CB":"ENSG00000119402","PIK3CD":"ENSG00000171608","PIK3R2":"ENSG00000189403","PTEN":"ENSG00000171862",
        "MTOR":"ENSG00000198793","RHEB":"ENSG00000106615",
        "RPTOR":"ENSG00000141564","MLST8":"ENSG00000105705","TSC1":"ENSG00000165699","TSC2":"ENSG00000103197",
        "EIF4EBP1":"ENSG00000187840","RPS6KB1":"ENSG00000108443","RPS6":"ENSG00000137154",
        "PDCD1":"ENSG00000276977","CD274":"ENSG00000120217","PDCD1LG2":"ENSG00000197646","JAK1":"ENSG00000162434",
        "JAK2":"ENSG00000096968","STAT1":"ENSG00000115415","IFNG":"ENSG00000111537","GZMB":"ENSG00000100453",
        "LAG3":"ENSG00000089692","TIGIT":"ENSG00000181847","CXCL9":"ENSG00000138755","CXCL10":"ENSG00000169245",
        "VEGFA":"ENSG00000112715","KDR":"ENSG00000128052","FLT1":"ENSG00000102755","FLT4":"ENSG00000037280",
        "PTPRB":"ENSG00000160593","NOS3":"ENSG00000164867",
        "FGFR1":"ENSG00000077782","FGFR2":"ENSG00000066468","FGFR3":"ENSG00000068078","FGFR4":"ENSG00000069535",
        "FRS2":"ENSG00000181873"
    }


    # Data Loading and Initial Processing
    # Handle potential gzip compression
    if uploaded_file.name.endswith('.gz'):
        gz_file = gzip.GzipFile(fileobj=uploaded_file)
        # Need to read into memory or save to a temp file to use seek(0)
        # Reading into memory for simplicity here, but be mindful of large files
        file_content = io.BytesIO(gz_file.read())
    else:
        file_content = io.BytesIO(uploaded_file.getvalue())


    # Detect row mode
    row_mode = detect_row_mode(file_content)
    st.write(f"Detected row mode: {row_mode.upper()}")

    # Get header to identify sample columns and gene column
    file_content.seek(0)
    header_line = io.TextIOWrapper(file_content, encoding="utf-8", errors="replace").readline().rstrip("\n")
    cols = header_line.split("\t")
    # Assuming the first column is the gene ID/symbol column
    gene_col = cols[0]
    # Assuming sample columns start from the second column
    sample_cols_full = cols[1:]

    # Select samples based on n_samples parameter
    if n_samples > len(sample_cols_full):
        st.warning(f"Requested {n_samples} samples, but only {len(sample_cols_full)} available. Using all available samples.")
        selected_samples = sample_cols_full
    else:
        selected_samples = sample_cols_full[:n_samples]

    st.write(f"Processing {len(selected_samples)} samples.")
    st.write(f"Using gene column: {gene_col}")

    # Stream-select rows and columns
    file_content.seek(0) # Reset file pointer before passing to stream_select
    expr_sym_small = stream_select_rows_columns(file_content, selected_samples, SIG_GENES, SYM2ENSG, row_mode, gene_col_name=gene_col)

    if expr_sym_small.empty:
        return None, None, None

    st.write("Expression matrix shape (genes x samples):", expr_sym_small.shape)
    st.dataframe(expr_sym_small.head())

    # Pathway Scoring
    st.subheader("Pathway Activity Scores")
    P = pathway_scores(expr_sym_small, SIGS)
    st.write("Pathway scores matrix shape (pathways x samples):", P.shape)
    st.dataframe(P)

    # QUBO Formulation and Drug Selection
    st.subheader("Drug Selection (QUBO)")
    panel = example_drug_panel()
    drugs = list(panel.keys())

    all_sels = []
    all_bhats = []
    patient_reports_data = []

    # Run analysis for each selected sample
    for sample_id in P.columns:
        z_path = patient_vector(P, sample_id)
        b_series = drug_benefit_prior(z_path, panel).reindex(drugs).fillna(0.0)
        R = build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10)
        b = b_series.to_numpy(float)
        q, Q = build_qubo(b, R, lam=lam_value)

        # Use exact solver for now
        x_star, e_star = exact_qubo_solve(b, R, lam=lam_value)
        sel = [drugs[i] for i, xi in enumerate(x_star) if xi == 1]
        all_sels.append(sel)
        all_bhats.append(b_series)

        # Prepare data for patient report
        top_pw = z_path.abs().sort_values(ascending=False).head(5)
        patient_reports_data.append({
            "patient": sample_id,
            "selected_drugs": ", ".join(sel) if sel else "(none)",
            "top_pathways": "; ".join([f"{p}:{z_path[p]:+.2f}" for p in top_pw.index]),
            **{f"b̂.{d}": float(b_series.get(d,0.0)) for d in drugs}
        })

    # Drug Selection Summary
    st.subheader("Drug Selection Summary")
    flat = [d for sel in all_sels for d in sel]
    freq = Counter(flat)
    freq_df = pd.DataFrame({"drug": drugs, "frequency": [freq[d] for d in drugs]})
    freq_df["frequency_pct"] = 100 * freq_df["frequency"] / len(selected_samples)
    freq_df = freq_df.sort_values("frequency_pct", ascending=False).reset_index(drop=True)
    st.dataframe(freq_df)

    # Patient-level Report
    st.subheader("Patient-level Report")
    patient_report_df = pd.DataFrame(patient_reports_data)
    st.dataframe(patient_report_df)

    return P, freq_df, patient_report_df, all_sels, drugs, panel

# --- Streamlit UI ---
st.set_page_config(layout="wide")

st.title("Digital Twin Workflow: Gene Expression to Drug Selection")

st.markdown("""
This application demonstrates a workflow for identifying potential drug therapies
for cancer patients based on their gene expression data, using pathway analysis
and QUBO optimization.
""")

st.sidebar.header("Input Data")
uploaded_file = st.sidebar.file_uploader("Upload gene expression data (CSV or TSV, can be gzipped)", type=["csv", "tsv", "gz"])

st.sidebar.header("Parameters")
n_samples = st.sidebar.slider("Number of samples to process", min_value=1, max_value=100, value=40, step=1)
lam_value = st.sidebar.slider("Lambda (penalty) value for QUBO", min_value=0.1, max_value=5.0, value=1.0, step=0.1)

st.header("Analysis Results")

# Placeholders for results
# results_placeholder = st.empty() # Not strictly needed with direct writes
# pathway_scores_placeholder = st.empty() # Displayed directly in run_analysis
# selected_drugs_placeholder = st.empty() # Displayed directly in run_analysis
visualizations_placeholder = st.empty()

if uploaded_file is not None:
    # Reset file pointer after file_uploader reads it initially
    uploaded_file.seek(0)
    file_extension = uploaded_file.name.split('.')[-1]
    if file_extension == 'gz':
         # Check the extension before .gz
         inner_extension = uploaded_file.name.split('.')[-2]
         sep = ',' if inner_extension == 'csv' else '\t'
    else:
        sep = ',' if file_extension == 'csv' else '\t'

    st.sidebar.success("File uploaded successfully!")
    st.write("Click 'Run Analysis' to process the data and get results.")

    if st.button("Run Analysis"):
        with st.spinner(f"Running analysis for {n_samples} samples with lambda={lam_value}..."):
            pathway_scores_df, freq_df, patient_report_df, all_sels, drugs, panel = run_analysis(uploaded_file, n_samples, lam_value)

            if pathway_scores_df is not None:
                visualizations_placeholder.subheader("Visualizations")

                # Drug selection frequency barplot
                try:
                    fig1, ax1 = plt.subplots(figsize=(7,5))
                    ax1.bar(freq_df["drug"], freq_df["frequency_pct"], color="steelblue")
                    ax1.set_ylabel("% patients selected")
                    ax1.set_title("Drug selection stability across patients")
                    plt.xticks(rotation=45, ha="right")
                    plt.tight_layout()
                    visualizations_placeholder.pyplot(fig1)
                except Exception as e:
                    visualizations_placeholder.write(f"Could not generate frequency barplot: {e}")

                # Co-selection heatmap and dendrogram
                try:
                    st.subheader("Co-selection Analysis")
                     # --- build co-selection counts ---
                    co_mat = pd.DataFrame(0, index=drugs, columns=drugs, dtype=int)
                    n_pat = max(1, len(all_sels))
                    for sel in all_sels:
                        uniq = list(dict.fromkeys(sel))
                        for i in range(len(uniq)):
                            for j in range(i, len(uniq)):
                                di, dj = uniq[i], uniq[j]
                                co_mat.loc[di, dj] += 1
                                if i != j:
                                    co_mat.loc[dj, di] += 1

                    # normalize to % of patients
                    co_pct = co_mat / n_pat * 100.0
                    st.write("Co-selection matrix (% of patients):")
                    st.dataframe(co_pct.round(1))

                    # heatmap
                    fig2, ax2 = plt.subplots(figsize=(7,6))
                    im = ax2.imshow(co_pct.values, cmap="Blues", interpolation="nearest")
                    ax2.set_xticks(range(len(drugs)), drugs, rotation=45, ha="right")
                    ax2.set_yticks(range(len(drugs)), drugs)
                    plt.colorbar(im, ax=ax2, label="% patients co-selected")
                    ax2.set_title("Drug co-selection heatmap")
                    plt.tight_layout()
                    visualizations_placeholder.pyplot(fig2)

                    # dendrogram
                    corr = np.corrcoef(co_pct.values)
                    corr = np.clip(corr, -1.0, 1.0)
                    dist = 1.0 - corr
                    Z = linkage(dist, method="average")

                    fig3, ax3 = plt.subplots(figsize=(8,5))
                    dendrogram(Z, labels=drugs, leaf_rotation=90, leaf_font_size=10,
                               color_threshold=0.7 * np.max(Z[:,2]), ax=ax3)
                    ax3.set_title("Clustered dendrogram of drug co-selection")
                    ax3.set_ylabel("Distance (1 - correlation)")
                    plt.tight_layout()
                    visualizations_placeholder.pyplot(fig3)

                    # clustered heatmap
                    order = leaves_list(Z)
                    co_pct_ordered = co_pct.iloc[order, order]

                    fig4, ax4 = plt.subplots(figsize=(7,6))
                    im = ax4.imshow(co_pct_ordered.values, cmap="Blues", interpolation="nearest")
                    ax4.set_xticks(range(len(order)), co_pct_ordered.columns, rotation=45, ha="right")
                    ax4.set_yticks(range(len(order)), co_pct_ordered.index)
                    plt.colorbar(im, ax=ax4, label="% patients co-selected")
                    ax4.set_title("Clustered heatmap of drug co-selection")
                    plt.tight_layout()
                    visualizations_placeholder.pyplot(fig4)
                    st.write("Clustered co-selection matrix:")
                    st.dataframe(co_pct_ordered.round(1))

                except Exception as e:
                    visualizations_placeholder.write(f"Could not generate co-selection plots: {e}")

                # Selection size distribution histogram
                try:
                    sizes = [len(set(sel)) for sel in all_sels]
                    size_hist = pd.Series(sizes).value_counts().sort_index()
                    fig5, ax5 = plt.subplots(figsize=(5,4))
                    ax5.bar(size_hist.index, size_hist.values)
                    ax5.set_xlabel("# drugs selected")
                    ax5.set_ylabel("# patients")
                    ax5.set_title("Selection size distribution")
                    plt.tight_layout()
                    visualizations_placeholder.pyplot(fig5)
                except Exception as e:
                    visualizations_placeholder.write(f"Could not generate selection size histogram: {e}")

                # Synthetic Classification Validation
                try:
                    st.subheader("Synthetic Classification Validation")
                    # 1) Construct patient-drug feature matrix
                    N = len(all_sels)
                    X = np.zeros((N, len(drugs)))
                    for i, sel in enumerate(all_sels):
                        for d in sel:
                            if d in drugs:
                                X[i, drugs.index(d)] = 1

                    # 2) Synthetic binary labels
                    risk_scores = []
                    for sid in P.columns[:N]:
                         z_path = patient_vector(P, sid)
                         risk_scores.append(z_path.mean())
                    risk_scores = np.array(risk_scores)
                    # Handle case where all risk scores are the same
                    if np.std(risk_scores) == 0:
                         st.warning("Cannot generate synthetic labels: all pathway mean scores are the same.")
                         y = np.zeros(N) # Or handle as appropriate
                         st.write("Synthetic labels (all 0):")
                         st.write(np.bincount(y))
                    else:
                         y = (risk_scores > np.median(risk_scores)).astype(int)
                         st.write("Synthetic labels (0=low risk, 1=high risk):")
                         st.write(np.bincount(y))

                    # 3) Train simple logistic regression
                    if len(np.unique(y)) > 1: # Only train if there's more than one class
                        clf = LogisticRegression(max_iter=200)
                        clf.fit(X, y)
                        y_pred = clf.predict_proba(X)[:,1]

                        # 4) Metrics
                        roc_auc = roc_auc_score(y, y_pred)
                        pr_auc = average_precision_score(y, y_pred)

                        st.write(f"ROC-AUC: {roc_auc:.3f}")
                        st.write(f"PR-AUC: {pr_auc:.3f}")

                        # Optional barplot of learned coefficients
                        coefs = pd.Series(clf.coef_[0], index=drugs).sort_values()
                        fig6, ax6 = plt.subplots(figsize=(7,4))
                        coefs.plot(kind="barh", ax=ax6, color=["steelblue" if v>0 else "salmon" for v in coefs])
                        ax6.set_title("Drug selection coefficients (synthetic label prediction)")
                        ax6.set_xlabel("Weight")
                        plt.tight_layout()
                        visualizations_placeholder.pyplot(fig6)
                    else:
                         st.warning("Classification skipped: only one unique synthetic label class found.")

                except Exception as e:
                    visualizations_placeholder.write(f"Could not perform synthetic classification validation: {e}")


            st.success("Analysis complete!")

else:
    st.info("Please upload a gene expression data file to begin.")

## Handle data input and output

### Subtask:
Implement mechanisms for users to upload their gene expression data (e.g., CSV file) through the web interface and for the application to output the results in a user-friendly format (e.g., tables, plots, downloadable reports).


**Reasoning**:
Implement file upload, data loading, and display results using Streamlit widgets and the core logic functions.



In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import gzip
import re
from collections import Counter
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
import matplotlib.pyplot as plt
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
import io
import base64

# Assume the core logic functions from the notebook are available here.
# For demonstration, I'll include them directly. In a larger project,
# these would be in a separate module.

def norm_ensembl(x): return x.split('.',1)[0]
def norm_symbol(x):  return re.sub(r'[^A-Za-z0-9_-]+','', x)

def detect_row_mode(file_object, scan_rows=50000):
    """Detect row ID type (Ensembl vs symbol) from file-like object."""
    seen = Counter(); total = 0
    # Use a TextIOWrapper to treat the bytes stream as text
    # Ensure file_object is at the beginning
    file_object.seek(0)
    with io.TextIOWrapper(file_object, encoding="utf-8", errors="replace") as f:
        # Read the header line to skip it for row detection
        header = f.readline()
        # Use header=None because we already read the header
        reader = pd.read_csv(f, sep="\t", chunksize=200_000, usecols=[0], dtype=str, header=None)
        for ch in reader:
            v = ch.iloc[:,0].astype(str)
            vals = v.head(min(len(v), scan_rows-total)).tolist()
            total += len(vals)
            seen.update('ENSG' if x.startswith('ENSG') else 'OTHER' for x in vals)
            if total >= scan_rows: break
    ratio = seen['ENSG']/max(1,(seen['ENSG']+seen['OTHER']))
    mode = 'ensembl' if ratio >= 0.6 else 'symbol'
    return mode

def stream_select_rows_columns(file_object, selected_samples, sig_genes, sym2ensg, row_mode, gene_col_name="Name", sep='\t'):
    """Stream-select rows and columns from file-like object."""
    if row_mode == 'ensembl':
        target_rows = set(sym2ensg.get(g) for g in sig_genes if g in sym2ensg)
        normalize = norm_ensembl
    else:
        target_rows = set(sig_genes)
        normalize = norm_symbol

    # Ensure file_object is at the beginning
    file_object.seek(0)

    with io.TextIOWrapper(file_object, encoding="utf-8", errors="replace") as f:
        # Read header to find the actual gene_col_name and sample columns
        header = f.readline().rstrip("\n").split(sep)
        try:
            gene_col_idx = header.index(gene_col_name)
        except ValueError:
             # If gene_col_name not found, assume the first column is the gene column
            gene_col_name = header[0]
            gene_col_idx = 0

        # Filter header to include only the gene column and selected sample columns
        usecols_filter = [gene_col_name] + selected_samples

        # Read data in chunks, selecting only necessary columns
        # Use header=None as we've already read the header
        reader = pd.read_csv(f, sep=sep, chunksize=50_000, dtype=str, header=None)
        kept = []
        for i, ch in enumerate(reader):
            # Assign original header to the chunk
            ch.columns = header
            # Select only the columns we need
            ch = ch[usecols_filter]

            ch = ch.rename(columns={gene_col_name: "row_id"})

            # Ensure 'row_id' column exists after renaming
            if 'row_id' not in ch.columns:
                 st.error(f"Error: Could not find gene identifier column '{gene_col_name}' in the uploaded file.")
                 return pd.DataFrame() # Return empty DataFrame on error

            ids = ch["row_id"].astype(str).map(normalize)
            mask = ids.isin(target_rows)

            if mask.any():
                out = ch.loc[mask].copy()
                out["row_id"] = out["row_id"].map(normalize)
                kept.append(out)

    if not kept:
        st.warning("No signature rows matched the provided gene list.")
        return pd.DataFrame()

    expr_small = pd.concat(kept, axis=0, ignore_index=False).drop_duplicates(subset=["row_id"]).set_index("row_id")

    if row_mode == 'ensembl':
        ensg2sym = {v:k for k,v in sym2ensg.items()}
        expr_sym_small = expr_small.copy()
        expr_sym_small.index = [ensg2sym.get(e, e) for e in expr_small.index]
    else:
        expr_sym_small = expr_small.copy()

    expr_sym_small = expr_sym_small[~expr_sym_small.index.duplicated(keep="first")]
    return expr_sym_small


def zscore_by_gene(expr_symbols: pd.DataFrame) -> pd.DataFrame:
    E = expr_symbols.copy()
    E = E.apply(pd.to_numeric, errors="coerce")
    E = E.loc[~E.isna().all(axis=1)]
    gene_means = E.mean(axis=1)
    E = E.apply(lambda col: col.fillna(gene_means), axis=0)
    mu = E.mean(axis=1)
    sd = E.std(axis=1) + 1e-8
    return (E.sub(mu, axis=0)).div(sd, axis=0)

def pathway_scores(expr_symbols: pd.DataFrame, signatures: dict) -> pd.DataFrame:
    """Return pathways x samples (mean z across member genes present)."""
    Z = zscore_by_gene(expr_symbols)
    rows = []
    for pw, genes in signatures.items():
        present = [g for g in genes if g in Z.index]
        if present:
            s = Z.loc[present].mean(axis=0)
        else:
            s = pd.Series([np.nan]*Z.shape[1], index=Z.columns)
        s.name = pw
        rows.append(s)
    return pd.DataFrame(rows)

def example_drug_panel():
    return {
        "EGFRi": ["REACTOME_SIGNALING_BY_EGFR"],
        "ALKi":  ["REACTOME_SIGNALING_BY_ALK"],
        "MEKi":  ["REACTOME_MAPK1_MAPK3_SIGNALING"],
        "PI3Ki": ["REACTOME_PI3K_AKT_SIGNALING"],
        "mTORi": ["REACTOME_MTORC1_MEDIATED_SIGNALLING"],
        "PD1i":  ["REACTOME_PD1_SIGNALING"],
        "VEGFi": ["REACTOME_VEGFA_VEGFR2_SIGNALING_PATHWAY"],
        "FGFRi": ["REACTOME_SIGNALING_BY_FGFR"],
    }

def patient_vector(P: pd.DataFrame, sample_id: str) -> pd.Series:
    """z-normalize pathways across samples; return vector for one patient."""
    z = (P - P.mean(axis=1).values.reshape(-1,1)) / (P.std(axis=1).values.reshape(-1,1) + 1e-8)
    return z[sample_id].fillna(0.0)

def drug_benefit_prior(z_path: pd.Series, panel: dict) -> pd.Series:
    """Aggregate pathway z's per drug (ReLU to emphasize upregulated pathways)."""
    s = pd.Series({d: float(np.sum([max(z_path.get(p, 0.0), 0.0) for p in pws])) for d, pws in panel.items()})
    return s / s.max() if s.max() > 0 else s

def build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10):
    """Pairwise penalties for overlapping mechanisms; diagonal = sparsity."""
    K = len(drugs); R = np.zeros((K, K), dtype=float)
    for i in range(K):
        for j in range(i+1, K):
            overlap = len(set(panel[drugs[i]]) & set(panel[drugs[j]]))
            if overlap > 0:
                R[i, j] = R[j, i] = base_overlap * overlap
    for i in range(K):
        R[i, i] += sparsity
    return R

def build_qubo(b_hat, R, lam=1.0):
    """QUBO: minimize x^T (lam R) x + q^T x  where q = -b̂ + diag(lam R)."""
    Q = lam * R.copy()
    q = -b_hat.copy()
    diag = np.diag(Q).copy()
    np.fill_diagonal(Q, 0.0)  # keep off-diagonal in Q
    q += diag
    return q, np.triu(Q, 1)

def exact_qubo_solve(b_hat: np.ndarray, R: np.ndarray, lam: float = 1.0):
    """
    Exact minimization of QUBO:
      E(x) = x^T (lam R) x + q^T x, with q = -b̂ + diag(lam R)
    We enumerate all bitstrings (2^K). Returns best bitstring (as 0/1 np array).
    """
    K = len(b_hat)
    Q = lam * R.copy()
    q = -b_hat.copy() + np.diag(Q)
    np.fill_diagonal(Q, 0.0)  # keep only off-diagonal in Q

    best_e = np.inf
    best_x = None
    # vectorize partial precomputations
    upper_idx = np.triu_indices(K, 1)
    for mask in range(1 << K):
        # build x from bits
        x = np.fromiter(((mask >> i) & 1 for i in range(K)), dtype=np.int8)
        # E = x^T Q x + q^T x, where Q is strictly upper-triangular mirrored
        e = np.dot(q, x) + 2.0 * np.sum(Q[upper_idx] * (x[upper_idx[0]] * x[upper_idx[1]]))
        if e < best_e:
            best_e, best_x = e, x
    return best_x, float(best_e)


# Placeholder for CUDA-Q function if needed later, currently uses exact solve
def try_cudaq_qaoa(h, J, p=2, shots=2048, max_iters=60, seed=7):
    return None # Not implemented in this web app version

# --- Main analysis function ---
def run_analysis(uploaded_file, n_samples, lam_value):
    """
    Runs the full analysis workflow for the uploaded data.
    Returns pathway scores, drug selection summary, and patient report.
    """
    SIGS = {
        "REACTOME_SIGNALING_BY_EGFR": [
            "EGFR","ERBB2","ERBB3","GRB2","SOS1","SHC1","PTPN11","KRAS","NRAS","HRAS",
            "BRAF","MAP2K1","MAP2K2","MAPK1","MAPK3","PLCG1","PIK3CA","PIK3R1","AKT1","AKT2","AKT3","GAB1"
        ],
        "REACTOME_SIGNALING_BY_ALK": [
            "ALK","EML4","GRB2","SHC1","PIK3CA","PIK3R1","AKT1","AKT2","AKT3","STAT3","MAP2K1","MAPK1","MAPK3"
        ],
        "REACTOME_MAPK1_MAPK3_SIGNALING": [
            "BRAF","RAF1","MAP2K1","MAP2K2","MAPK1","MAPK3","DUSP6","DUSP4","FOS","JUN","EGFR"
        ],
        "REACTOME_PI3K_AKT_SIGNALING": [
            "PIK3CA","PIK3CB","PIK3CD","PIK3R1","PIK3R2","AKT1","AKT2","AKT3","PTEN","MTOR","RHEB"
        ],
        "REACTOME_MTORC1_MEDIATED_SIGNALLING": [
            "MTOR","RPTOR","MLST8","RHEB","TSC1","TSC2","EIF4EBP1","RPS6KB1","RPS6"
        ],
        "REACTOME_PD1_SIGNALING": [
            "PDCD1","CD274","PDCD1LG2","PDCD1LG2","JAK1","JAK2","STAT1","IFNG","GZMB","LAG3","TIGIT","CXCL9","CXCL10"
        ],
        "REACTOME_VEGFA_VEGFR2_SIGNALING_PATHWAY": [
            "VEGFA","KDR","FLT1","FLT4","PTPRB","PLCG1","MAP2K1","MAPK1","NOS3"
        ],
        "REACTOME_SIGNALING_BY_FGFR": [
            "FGFR1","FGFR2","FGFR3","FGFR4","FRS2","PLCG1","PIK3CA","PIK3R1","MAP2K1","MAPK1"
        ]
    }
    SIG_GENES = sorted({g for gs in SIGS.values() for g in gs})
    SYM2ENSG = {
        "EGFR":"ENSG00000146648","ERBB2":"ENSG00000141736","ERBB3":"ENSG00000065361","GRB2":"ENSG00000177885",
        "SOS1":"ENSG00000115904","SHC1":"ENSG00000154639","PTPN11":"ENSG00000179295","KRAS":"ENSG00000133703",
        "NRAS":"ENSG00000213281","HRAS":"ENSG00000174775","BRAF":"ENSG00000157764","MAP2K1":"ENSG00000169032",
        "MAP2K2":"ENSG00000126934","MAPK1":"ENSG00000100030","MAPK3":"ENSG00000102882","PLCG1":"ENSG00000124181",
        "PIK3CA":"ENSG00000121879","PIK3R1":"ENSG00000145675","AKT1":"ENSG00000142208","AKT2":"ENSG00000105221",
        "AKT3":"ENSG00000117020","GAB1":"ENSG00000117676",
        "ALK":"ENSG00000171094","EML4":"ENSG00000143924","STAT3":"ENSG00000168610",
        "DUSP6":"ENSG00000139318","DUSP4":"ENSG00000120875","FOS":"ENSG00000170345","JUN":"ENSG00000177606","RAF1":"ENSG00000132155",
        "PIK3CB":"ENSG00000119402","PIK3CD":"ENSG00000171608","PIK3R2":"ENSG00000189403","PTEN":"ENSG00000171862",
        "MTOR":"ENSG00000198793","RHEB":"ENSG00000106615",
        "RPTOR":"ENSG00000141564","MLST8":"ENSG00000105705","TSC1":"ENSG00000165699","TSC2":"ENSG00000103197",
        "EIF4EBP1":"ENSG00000187840","RPS6KB1":"ENSG00000108443","RPS6":"ENSG00000137154",
        "PDCD1":"ENSG00000276977","CD274":"ENSG00000120217","PDCD1LG2":"ENSG00000197646","JAK1":"ENSG00000162434",
        "JAK2":"ENSG00000096968","STAT1":"ENSG00000115415","IFNG":"ENSG00000111537","GZMB":"ENSG00000100453",
        "LAG3":"ENSG00000089692","TIGIT":"ENSG00000181847","CXCL9":"ENSG00000138755","CXCL10":"ENSG00000169245",
        "VEGFA":"ENSG00000112715","KDR":"ENSG00000128052","FLT1":"ENSG00000102755","FLT4":"ENSG00000037280",
        "PTPRB":"ENSG00000160593","NOS3":"ENSG00000164867",
        "FGFR1":"ENSG00000077782","FGFR2":"ENSG00000066468","FGFR3":"ENSG00000068078","FGFR4":"ENSG00000069535",
        "FRS2":"ENSG00000181873"
    }

    # Data Loading and Initial Processing
    # Handle potential gzip compression and determine delimiter
    file_content = uploaded_file.getvalue()
    if uploaded_file.name.endswith('.gz'):
        try:
            gz_file = gzip.GzipFile(fileobj=io.BytesIO(file_content))
            # Peek at the first line after decompression to determine delimiter
            with io.TextIOWrapper(gz_file, encoding="utf-8", errors="replace") as f:
                header_peek = f.readline().rstrip("\n")
                sep = ',' if ',' in header_peek.split() else '\t'
            # Reset gz_file for actual reading
            gz_file.seek(0)
            file_stream = io.BytesIO(gz_file.read())
        except Exception as e:
            st.error(f"Error reading gzipped file: {e}")
            return None, None, None, None, None, None
    else:
        # Determine delimiter for non-gzipped files
        try:
            header_peek = io.BytesIO(file_content).readline().decode('utf-8').rstrip("\n")
            sep = ',' if ',' in header_peek.split() else '\t'
        except Exception as e:
             st.error(f"Error reading file header: {e}")
             return None, None, None, None, None, None
        file_stream = io.BytesIO(file_content)

    st.write(f"Detected delimiter: '{sep}'")

    # Detect row mode
    # Need a fresh file object for detect_row_mode
    file_stream_for_detect = io.BytesIO(file_stream.getvalue())
    row_mode = detect_row_mode(file_stream_for_detect)
    st.write(f"Detected row mode: {row_mode.upper()}")

    # Get header to identify sample columns and gene column
    file_stream.seek(0) # Ensure stream is at the beginning
    header_line = io.TextIOWrapper(file_stream, encoding="utf-8", errors="replace").readline().rstrip("\n")
    cols = header_line.split(sep)
    # Assuming the first column is the gene ID/symbol column
    gene_col = cols[0]
    # Assuming sample columns start from the second column
    sample_cols_full = cols[1:]

    # Select samples based on n_samples parameter
    if n_samples > len(sample_cols_full):
        st.warning(f"Requested {n_samples} samples, but only {len(sample_cols_full)} available. Using all available samples.")
        selected_samples = sample_cols_full
    else:
        selected_samples = sample_cols_full[:n_samples]

    st.write(f"Processing {len(selected_samples)} samples.")
    st.write(f"Using gene column: '{gene_col}'")

    # Stream-select rows and columns
    # Need a fresh file object for stream_select_rows_columns
    file_stream_for_select = io.BytesIO(file_stream.getvalue())
    expr_sym_small = stream_select_rows_columns(file_stream_for_select, selected_samples, SIG_GENES, SYM2ENSG, row_mode, gene_col_name=gene_col, sep=sep)

    if expr_sym_small.empty:
        st.error("Failed to load expression data. Please check file format and contents.")
        return None, None, None, None, None, None

    st.subheader("Expression Matrix (Subset)")
    st.write("Shape (genes x samples):", expr_sym_small.shape)
    st.dataframe(expr_sym_small.head())

    # Pathway Scoring
    st.subheader("Pathway Activity Scores")
    P = pathway_scores(expr_sym_small, SIGS)
    st.write("Shape (pathways x samples):", P.shape)
    st.dataframe(P)

    # QUBO Formulation and Drug Selection
    st.subheader("Drug Selection Results")
    panel = example_drug_panel()
    drugs = list(panel.keys())

    all_sels = []
    all_bhats = []
    patient_reports_data = []

    # Run analysis for each selected sample
    if P.empty:
        st.warning("No pathway scores computed. Cannot perform drug selection.")
        return P, pd.DataFrame(), pd.DataFrame(), [], drugs, panel

    for sample_id in P.columns:
        z_path = patient_vector(P, sample_id)
        b_series = drug_benefit_prior(z_path, panel).reindex(drugs).fillna(0.0)
        R = build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10)
        b = b_series.to_numpy(float)
        q, Q = build_qubo(b, R, lam=lam_value)

        # Use exact solver for now
        x_star, e_star = exact_qubo_solve(b, R, lam=lam_value)
        sel = [drugs[i] for i, xi in enumerate(x_star) if xi == 1]
        all_sels.append(sel)
        all_bhats.append(b_series)

        # Prepare data for patient report
        top_pw = z_path.abs().sort_values(ascending=False).head(5)
        patient_reports_data.append({
            "patient": sample_id,
            "selected_drugs": ", ".join(sel) if sel else "(none)",
            "top_pathways": "; ".join([f"{p}:{z_path[p]:+.2f}" for p in top_pw.index]),
            **{f"b̂.{d}": float(b_series.get(d,0.0)) for d in drugs}
        })

    # Drug Selection Summary
    st.subheader("Drug Selection Frequency")
    if not all_sels:
        st.info("No drugs were selected for any patient.")
        freq_df = pd.DataFrame({"drug": drugs, "frequency": 0, "frequency_pct": 0.0})
    else:
        flat = [d for sel in all_sels for d in sel]
        freq = Counter(flat)
        freq_df = pd.DataFrame({"drug": drugs, "frequency": [freq[d] for d in drugs]})
        freq_df["frequency_pct"] = 100 * freq_df["frequency"] / len(selected_samples)
        freq_df = freq_df.sort_values("frequency_pct", ascending=False).reset_index(drop=True)
    st.dataframe(freq_df)

    # Patient-level Report
    st.subheader("Patient-level Report")
    patient_report_df = pd.DataFrame(patient_reports_data)
    st.dataframe(patient_report_df)

    return P, freq_df, patient_report_df, all_sels, drugs, panel

# Function to create a download link
def create_download_link(df, filename, text):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    href = f'<a href="data:file/csv;base64,{b64}" download="{filename}">{text}</a>'
    return href


# --- Streamlit UI ---
st.set_page_config(layout="wide")

st.title("Digital Twin Workflow: Gene Expression to Drug Selection")

st.markdown("""
This application demonstrates a workflow for identifying potential drug therapies
for cancer patients based on their gene expression data, using pathway analysis
and QUBO optimization.
""")

st.sidebar.header("Input Data")
uploaded_file = st.sidebar.file_uploader("Upload gene expression data (CSV, TSV, or gzipped)", type=["csv", "tsv", "gz"])

st.sidebar.header("Parameters")
n_samples = st.sidebar.slider("Number of samples to process", min_value=1, max_value=100, value=40, step=1)
lam_value = st.sidebar.slider("Lambda (penalty) value for QUBO", min_value=0.1, max_value=5.0, value=1.0, step=0.1)

st.header("Analysis Results")

if uploaded_file is not None:
    if st.button("Run Analysis"):
        with st.spinner(f"Running analysis for {n_samples} samples with lambda={lam_value}..."):
            pathway_scores_df, freq_df, patient_report_df, all_sels, drugs, panel = run_analysis(uploaded_file, n_samples, lam_value)

            if pathway_scores_df is not None:
                st.subheader("Visualizations")

                # Drug selection frequency barplot
                if freq_df is not None and not freq_df.empty:
                    try:
                        fig1, ax1 = plt.subplots(figsize=(7,5))
                        ax1.bar(freq_df["drug"], freq_df["frequency_pct"], color="steelblue")
                        ax1.set_ylabel("% patients selected")
                        ax1.set_title("Drug selection stability across patients")
                        plt.xticks(rotation=45, ha="right")
                        plt.tight_layout()
                        st.pyplot(fig1)
                        plt.close(fig1) # Close figure to free memory
                    except Exception as e:
                        st.write(f"Could not generate frequency barplot: {e}")

                # Co-selection heatmap and dendrogram
                if all_sels and drugs and panel:
                    try:
                        st.subheader("Co-selection Analysis")
                         # --- build co-selection counts ---
                        co_mat = pd.DataFrame(0, index=drugs, columns=drugs, dtype=int)
                        n_pat = max(1, len(all_sels))
                        for sel in all_sels:
                            uniq = list(dict.fromkeys(sel))
                            for i in range(len(uniq)):
                                for j in range(i, len(uniq)):
                                    di, dj = uniq[i], uniq[j]
                                    co_mat.loc[di, dj] += 1
                                    if i != j:
                                        co_mat.loc[dj, di] += 1

                        # normalize to % of patients
                        co_pct = co_mat / n_pat * 100.0
                        st.write("Co-selection matrix (% of patients):")
                        st.dataframe(co_pct.round(1))

                        # heatmap
                        fig2, ax2 = plt.subplots(figsize=(7,6))
                        im = ax2.imshow(co_pct.values, cmap="Blues", interpolation="nearest")
                        ax2.set_xticks(range(len(drugs)), drugs, rotation=45, ha="right")
                        ax2.set_yticks(range(len(drugs)), drugs)
                        plt.colorbar(im, ax=ax2, label="% patients co-selected")
                        ax2.set_title("Drug co-selection heatmap")
                        plt.tight_layout()
                        st.pyplot(fig2)
                        plt.close(fig2) # Close figure

                        # dendrogram
                        corr = np.corrcoef(co_pct.values)
                        corr = np.clip(corr, -1.0, 1.0)
                        dist = 1.0 - corr
                        Z = linkage(dist, method="average")

                        fig3, ax3 = plt.subplots(figsize=(8,5))
                        dendrogram(Z, labels=drugs, leaf_rotation=90, leaf_font_size=10,
                                color_threshold=0.7 * np.max(Z[:,2]), ax=ax3)
                        ax3.set_title("Clustered dendrogram of drug co-selection")
                        ax3.set_ylabel("Distance (1 - correlation)")
                        plt.tight_layout()
                        st.pyplot(fig3)
                        plt.close(fig3) # Close figure

                        # clustered heatmap
                        order = leaves_list(Z)
                        co_pct_ordered = co_pct.iloc[order, order]

                        fig4, ax4 = plt.subplots(figsize=(7,6))
                        im = ax4.imshow(co_pct_ordered.values, cmap="Blues", interpolation="nearest")
                        ax4.set_xticks(range(len(order)), co_pct_ordered.columns, rotation=45, ha="right")
                        ax4.set_yticks(range(len(order)), co_pct_ordered.index)
                        plt.colorbar(im, ax=ax4, label="% patients co-selected")
                        ax4.set_title("Clustered heatmap of drug co-selection")
                        plt.tight_layout()
                        st.pyplot(fig4)
                        plt.close(fig4) # Close figure
                        st.write("Clustered co-selection matrix:")
                        st.dataframe(co_pct_ordered.round(1))

                    except Exception as e:
                        st.write(f"Could not generate co-selection plots: {e}")

                # Selection size distribution histogram
                if all_sels:
                    try:
                        sizes = [len(set(sel)) for sel in all_sels]
                        size_hist = pd.Series(sizes).value_counts().sort_index()
                        fig5, ax5 = plt.subplots(figsize=(5,4))
                        ax5.bar(size_hist.index, size_hist.values)
                        ax5.set_xlabel("# drugs selected")
                        ax5.set_ylabel("# patients")
                        ax5.set_title("Selection size distribution")
                        plt.tight_layout()
                        st.pyplot(fig5)
                        plt.close(fig5) # Close figure
                    except Exception as e:
                        st.write(f"Could not generate selection size histogram: {e}")

                # Synthetic Classification Validation
                if all_sels and drugs and pathway_scores_df is not None and not pathway_scores_df.empty:
                    try:
                        st.subheader("Synthetic Classification Validation")
                        # 1) Construct patient-drug feature matrix
                        N = len(all_sels)
                        X = np.zeros((N, len(drugs)))
                        for i, sel in enumerate(all_sels):
                            for d in drugs: # Iterate through drugs to ensure correct indexing
                                if d in sel:
                                    X[i, drugs.index(d)] = 1

                        # 2) Synthetic binary labels
                        risk_scores = []
                        # Ensure we only process samples that were successfully processed in run_analysis
                        processed_sample_ids = pathway_scores_df.columns
                        if len(processed_sample_ids) < N:
                             st.warning(f"Only {len(processed_sample_ids)} samples processed for classification.")
                             # Adjust N and X to match processed samples if necessary, or skip classification
                             # For simplicity, skipping if mismatch or no processed samples
                             if len(processed_sample_ids) == 0:
                                  st.warning("No samples processed for synthetic classification.")
                                  raise ValueError("No processed samples") # Trigger exception to skip
                             N = len(processed_sample_ids)
                             X = np.zeros((N, len(drugs))) # Rebuild X with correct size
                             # Need to re-run selection logic for the processed samples to rebuild X correctly
                             # This is getting complex; a simpler approach is to ensure run_analysis
                             # returns data consistently sized or handle size mismatches more robustly.
                             # For now, rely on the check and skip if needed.
                             st.warning("Skipping synthetic classification due to sample count mismatch.")
                             raise ValueError("Sample count mismatch") # Trigger exception to skip

                        for sid in processed_sample_ids:
                             z_path = patient_vector(pathway_scores_df, sid)
                             risk_scores.append(z_path.mean())
                        risk_scores = np.array(risk_scores)

                        # Handle case where all risk scores are the same
                        if np.std(risk_scores) == 0:
                             st.warning("Cannot generate synthetic labels: all pathway mean scores are the same.")
                             st.write("Synthetic labels (all 0):")
                             st.write(np.zeros(N, dtype=int))
                        else:
                             y = (risk_scores > np.median(risk_scores)).astype(int)
                             st.write("Synthetic labels (0=low risk, 1=high risk):")
                             st.write(np.bincount(y))

                             # 3) Train simple logistic regression
                             if len(np.unique(y)) > 1: # Only train if there's more than one class
                                 clf = LogisticRegression(max_iter=200)
                                 clf.fit(X[:N], y) # Use adjusted N
                                 y_pred = clf.predict_proba(X[:N])[:,1] # Use adjusted N

                                 # 4) Metrics
                                 roc_auc = roc_auc_score(y, y_pred)
                                 pr_auc = average_precision_score(y, y_pred)

                                 st.write(f"ROC-AUC: {roc_auc:.3f}")
                                 st.write(f"PR-AUC: {pr_auc:.3f}")

                                 # Optional barplot of learned coefficients
                                 coefs = pd.Series(clf.coef_[0], index=drugs).sort_values()
                                 fig6, ax6 = plt.subplots(figsize=(7,4))
                                 coefs.plot(kind="barh", ax=ax6, color=["steelblue" if v>0 else "salmon" for v in coefs])
                                 ax6.set_title("Drug selection coefficients (synthetic label prediction)")
                                 ax6.set_xlabel("Weight")
                                 plt.tight_layout()
                                 st.pyplot(fig6)
                                 plt.close(fig6) # Close figure
                             else:
                                 st.warning("Classification skipped: only one unique synthetic label class found.")

                    except Exception as e:
                        st.write(f"Could not perform synthetic classification validation: {e}")

                # Downloadable reports
                if patient_report_df is not None and not patient_report_df.empty:
                    st.subheader("Download Reports")
                    st.markdown(create_download_link(patient_report_df, "patient_report.csv", "Download Patient Report (CSV)"), unsafe_allow_html=True)

                if freq_df is not None and not freq_df.empty:
                     st.markdown(create_download_link(freq_df, "drug_frequency_report.csv", "Download Drug Frequency Report (CSV)"), unsafe_allow_html=True)

                st.success("Analysis complete!")

else:
    st.info("Please upload a gene expression data file to begin.")

## Add visualizations

### Subtask:
Convert the matplotlib visualizations from the notebook into interactive plots for the web application using libraries like Plotly or Bokeh, or by embedding the matplotlib plots in the web framework.


**Reasoning**:
Convert the Matplotlib plots generated in the Streamlit application to interactive Plotly plots for enhanced user interaction.



In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import gzip
import re
from collections import Counter
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
import matplotlib.pyplot as plt
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
import io
import base64
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assume the core logic functions from the notebook are available here.
# For demonstration, I'll include them directly. In a larger project,
# these would be in a separate module.

def norm_ensembl(x): return x.split('.',1)[0]
def norm_symbol(x):  return re.sub(r'[^A-Za-z0-9_-]+','', x)

def detect_row_mode(file_object, scan_rows=50000):
    """Detect row ID type (Ensembl vs symbol) from file-like object."""
    seen = Counter(); total = 0
    # Use a TextIOWrapper to treat the bytes stream as text
    # Ensure file_object is at the beginning
    file_object.seek(0)
    with io.TextIOWrapper(file_object, encoding="utf-8", errors="replace") as f:
        # Read the header line to skip it for row detection
        header = f.readline()
        # Use header=None because we already read the header
        reader = pd.read_csv(f, sep="\t", chunksize=200_000, usecols=[0], dtype=str, header=None)
        for ch in reader:
            v = ch.iloc[:,0].astype(str)
            vals = v.head(min(len(v), scan_rows-total)).tolist()
            total += len(vals)
            seen.update('ENSG' if x.startswith('ENSG') else 'OTHER' for x in vals)
            if total >= scan_rows: break
    ratio = seen['ENSG']/max(1,(seen['ENSG']+seen['OTHER']))
    mode = 'ensembl' if ratio >= 0.6 else 'symbol'
    return mode

def stream_select_rows_columns(file_object, selected_samples, sig_genes, sym2ensg, row_mode, gene_col_name="Name", sep='\t'):
    """Stream-select rows and columns from file-like object."""
    if row_mode == 'ensembl':
        target_rows = set(sym2ensg.get(g) for g in sig_genes if g in sym2ensg)
        normalize = norm_ensembl
    else:
        target_rows = set(sig_genes)
        normalize = norm_symbol

    # Ensure file_object is at the beginning
    file_object.seek(0)

    with io.TextIOWrapper(file_object, encoding="utf-8", errors="replace") as f:
        # Read header to find the actual gene_col_name and sample columns
        header = f.readline().rstrip("\n").split(sep)
        try:
            gene_col_idx = header.index(gene_col_name)
        except ValueError:
             # If gene_col_name not found, assume the first column is the gene column
            gene_col_name = header[0]
            gene_col_idx = 0

        # Filter header to include only the gene column and selected sample columns
        usecols_filter = [gene_col_name] + selected_samples

        # Read data in chunks, selecting only necessary columns
        # Use header=None as we've already read the header
        reader = pd.read_csv(f, sep=sep, chunksize=50_000, dtype=str, header=None)
        kept = []
        for i, ch in enumerate(reader):
            # Assign original header to the chunk
            ch.columns = header
            # Select only the columns we need
            ch = ch[usecols_filter]

            ch = ch.rename(columns={gene_col_name: "row_id"})

            # Ensure 'row_id' column exists after renaming
            if 'row_id' not in ch.columns:
                 st.error(f"Error: Could not find gene identifier column '{gene_col_name}' in the uploaded file.")
                 return pd.DataFrame() # Return empty DataFrame on error

            ids = ch["row_id"].astype(str).map(normalize)
            mask = ids.isin(target_rows)

            if mask.any():
                out = ch.loc[mask].copy()
                out["row_id"] = out["row_id"].map(normalize)
                kept.append(out)

    if not kept:
        st.warning("No signature rows matched the provided gene list.")
        return pd.DataFrame()

    expr_small = pd.concat(kept, axis=0, ignore_index=False).drop_duplicates(subset=["row_id"]).set_index("row_id")

    if row_mode == 'ensembl':
        ensg2sym = {v:k for k,v in SYM2ENSG.items()}
        expr_sym_small = expr_small.copy()
        expr_sym_small.index = [ensg2sym.get(e, e) for e in expr_small.index]
    else:
        expr_sym_small = expr_small.copy()

    expr_sym_small = expr_sym_small[~expr_sym_small.index.duplicated(keep="first")]
    return expr_sym_small


def zscore_by_gene(expr_symbols: pd.DataFrame) -> pd.DataFrame:
    E = expr_symbols.copy()
    E = E.apply(pd.to_numeric, errors="coerce")
    E = E.loc[~E.isna().all(axis=1)]
    gene_means = E.mean(axis=1)
    E = E.apply(lambda col: col.fillna(gene_means), axis=0)
    mu = E.mean(axis=1)
    sd = E.std(axis=1) + 1e-8
    return (E.sub(mu, axis=0)).div(sd, axis=0)

def pathway_scores(expr_symbols: pd.DataFrame, signatures: dict) -> pd.DataFrame:
    """Return pathways x samples (mean z across member genes present)."""
    Z = zscore_by_gene(expr_symbols)
    rows = []
    for pw, genes in signatures.items():
        present = [g for g in genes if g in Z.index]
        if present:
            s = Z.loc[present].mean(axis=0)
        else:
            s = pd.Series([np.nan]*Z.shape[1], index=Z.columns)
        s.name = pw
        rows.append(s)
    return pd.DataFrame(rows)

def example_drug_panel():
    return {
        "EGFRi": ["REACTOME_SIGNALING_BY_EGFR"],
        "ALKi":  ["REACTOME_SIGNALING_BY_ALK"],
        "MEKi":  ["REACTOME_MAPK1_MAPK3_SIGNALING"],
        "PI3Ki": ["REACTOME_PI3K_AKT_SIGNALING"],
        "mTORi": ["REACTOME_MTORC1_MEDIATED_SIGNALLING"],
        "PD1i":  ["REACTOME_PD1_SIGNALING"],
        "VEGFi": ["REACTOME_VEGFA_VEGFR2_SIGNALING_PATHWAY"],
        "FGFRi": ["REACTOME_SIGNALING_BY_FGFR"],
    }

def patient_vector(P: pd.DataFrame, sample_id: str) -> pd.Series:
    """z-normalize pathways across samples; return vector for one patient."""
    z = (P - P.mean(axis=1).values.reshape(-1,1)) / (P.std(axis=1).values.reshape(-1,1) + 1e-8)
    return z[sample_id].fillna(0.0)

def drug_benefit_prior(z_path: pd.Series, panel: dict) -> pd.Series:
    """Aggregate pathway z's per drug (ReLU to emphasize upregulated pathways)."""
    s = pd.Series({d: float(np.sum([max(z_path.get(p, 0.0), 0.0) for p in pws])) for d, pws in panel.items()})
    return s / s.max() if s.max() > 0 else s

def build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10):
    """Pairwise penalties for overlapping mechanisms; diagonal = sparsity."""
    K = len(drugs); R = np.zeros((K, K), dtype=float)
    for i in range(K):
        for j in range(i+1, K):
            overlap = len(set(panel[drugs[i]]) & set(panel[drugs[j]]))
            if overlap > 0:
                R[i, j] = R[j, i] = base_overlap * overlap
    for i in range(K):
        R[i, i] += sparsity
    return R

def build_qubo(b_hat, R, lam=1.0):
    """QUBO: minimize x^T (lam R) x + q^T x  where q = -b̂ + diag(lam R)."""
    Q = lam * R.copy()
    q = -b_hat.copy()
    diag = np.diag(Q).copy()
    np.fill_diagonal(Q, 0.0)  # keep off-diagonal in Q
    q += diag
    return q, np.triu(Q, 1)

def exact_qubo_solve(b_hat: np.ndarray, R: np.ndarray, lam: float = 1.0):
    """
    Exact minimization of QUBO:
      E(x) = x^T (lam R) x + q^T x, with q = -b̂ + diag(lam R)
    We enumerate all bitstrings (2^K). Returns best bitstring (as 0/1 np array).
    """
    K = len(b_hat)
    Q = lam * R.copy()
    q = -b_hat.copy() + np.diag(Q)
    np.fill_diagonal(Q, 0.0)  # keep only off-diagonal in Q

    best_e = np.inf
    best_x = None
    # vectorize partial precomputations
    upper_idx = np.triu_indices(K, 1)
    for mask in range(1 << K):
        # build x from bits
        x = np.fromiter(((mask >> i) & 1 for i in range(K)), dtype=np.int8)
        # E = x^T Q x + q^T x, where Q is strictly upper-triangular mirrored
        e = np.dot(q, x) + 2.0 * np.sum(Q[upper_idx] * (x[upper_idx[0]] * x[upper_idx[1]]))
        if e < best_e:
            best_e, best_x = e, x
    return best_x, float(best_e)


# Placeholder for CUDA-Q function if needed later, currently uses exact solve
def try_cudaq_qaoa(h, J, p=2, shots=2048, max_iters=60, seed=7):
    return None # Not implemented in this web app version

# --- Main analysis function ---
def run_analysis(uploaded_file, n_samples, lam_value):
    """
    Runs the full analysis workflow for the uploaded data.
    Returns pathway scores, drug selection summary, and patient report.
    """
    SIGS = {
        "REACTOME_SIGNALING_BY_EGFR": [
            "EGFR","ERBB2","ERBB3","GRB2","SOS1","SHC1","PTPN11","KRAS","NRAS","HRAS",
            "BRAF","MAP2K1","MAP2K2","MAPK1","MAPK3","PLCG1","PIK3CA","PIK3R1","AKT1","AKT2","AKT3","GAB1"
        ],
        "REACTOME_SIGNALING_BY_ALK": [
            "ALK","EML4","GRB2","SHC1","PIK3CA","PIK3R1","AKT1","AKT2","AKT3","STAT3","MAP2K1","MAPK1","MAPK3"
        ],
        "REACTOME_MAPK1_MAPK3_SIGNALING": [
            "BRAF","RAF1","MAP2K1","MAP2K2","MAPK1","MAPK3","DUSP6","DUSP4","FOS","JUN","EGFR"
        ],
        "REACTOME_PI3K_AKT_SIGNALING": [
            "PIK3CA","PIK3CB","PIK3CD","PIK3R1","PIK3R2","AKT1","AKT2","AKT3","PTEN","MTOR","RHEB"
        ],
        "REACTOME_MTORC1_MEDIATED_SIGNALLING": [
            "MTOR","RPTOR","MLST8","RHEB","TSC1","TSC2","EIF4EBP1","RPS6KB1","RPS6"
        ],
        "REACTOME_PD1_SIGNALING": [
            "PDCD1","CD274","PDCD1LG2","PDCD1LG2","JAK1","JAK2","STAT1","IFNG","GZMB","LAG3","TIGIT","CXCL9","CXCL10"
        ],
        "REACTOME_VEGFA_VEGFR2_SIGNALING_PATHWAY": [
            "VEGFA","KDR","FLT1","FLT4","PTPRB","PLCG1","MAP2K1","MAPK1","NOS3"
        ],
        "REACTOME_SIGNALING_BY_FGFR": [
            "FGFR1","FGFR2","FGFR3","FGFR4","FRS2","PLCG1","PIK3CA","PIK3R1","MAP2K1","MAPK1"
        ]
    }
    SIG_GENES = sorted({g for gs in SIGS.values() for g in gs})
    SYM2ENSG = {
        "EGFR":"ENSG00000146648","ERBB2":"ENSG00000141736","ERBB3":"ENSG00000065361","GRB2":"ENSG00000177885",
        "SOS1":"ENSG00000115904","SHC1":"ENSG00000154639","PTPN11":"ENSG00000179295","KRAS":"ENSG00000133703",
        "NRAS":"ENSG00000213281","HRAS":"ENSG00000174775","BRAF":"ENSG00000157764","MAP2K1":"ENSG00000169032",
        "MAP2K2":"ENSG00000126934","MAPK1":"ENSG00000100030","MAPK3":"ENSG00000102882","PLCG1":"ENSG00000124181",
        "PIK3CA":"ENSG00000121879","PIK3R1":"ENSG00000145675","AKT1":"ENSG00000142208","AKT2":"ENSG00000105221",
        "AKT3":"ENSG00000117020","GAB1":"ENSG00000117676",
        "ALK":"ENSG00000171094","EML4":"ENSG00000143924","STAT3":"ENSG00000168610",
        "DUSP6":"ENSG00000139318","DUSP4":"ENSG00000120875","FOS":"ENSG00000170345","JUN":"ENSG00000177606","RAF1":"ENSG00000132155",
        "PIK3CB":"ENSG00000119402","PIK3CD":"ENSG00000171608","PIK3R2":"ENSG00000189403","PTEN":"ENSG00000171862",
        "MTOR":"ENSG00000198793","RHEB":"ENSG00000106615",
        "RPTOR":"ENSG00000141564","MLST8":"ENSG00000105705","TSC1":"ENSG00000165699","TSC2":"ENSG00000103197",
        "EIF4EBP1":"ENSG00000187840","RPS6KB1":"ENSG00000108443","RPS6":"ENSG00000137154",
        "PDCD1":"ENSG00000276977","CD274":"ENSG00000120217","PDCD1LG2":"ENSG00000197646","JAK1":"ENSG00000162434",
        "JAK2":"ENSG00000096968","STAT1":"ENSG00000115415","IFNG":"ENSG00000111537","GZMB":"ENSG00000100453",
        "LAG3":"ENSG00000089692","TIGIT":"ENSG00000181847","CXCL9":"ENSG00000138755","CXCL10":"ENSG00000169245",
        "VEGFA":"ENSG00000112715","KDR":"ENSG00000128052","FLT1":"ENSG00000102755","FLT4":"ENSG00000037280",
        "PTPRB":"ENSG00000160593","NOS3":"ENSG00000164867",
        "FGFR1":"ENSG00000077782","FGFR2":"ENSG00000066468","FGFR3":"ENSG00000068078","FGFR4":"ENSG00000069535",
        "FRS2":"ENSG00000181873"
    }

    # Data Loading and Initial Processing
    # Handle potential gzip compression and determine delimiter
    file_content = uploaded_file.getvalue()
    if uploaded_file.name.endswith('.gz'):
        try:
            gz_file = gzip.GzipFile(fileobj=io.BytesIO(file_content))
            # Peek at the first line after decompression to determine delimiter
            with io.TextIOWrapper(gz_file, encoding="utf-8", errors="replace") as f:
                header_peek = f.readline().rstrip("\n")
                sep = ',' if ',' in header_peek.split() else '\t'
            # Reset gz_file for actual reading
            gz_file.seek(0)
            file_stream = io.BytesIO(gz_file.read())
        except Exception as e:
            st.error(f"Error reading gzipped file: {e}")
            return None, None, None, None, None, None
    else:
        # Determine delimiter for non-gzipped files
        try:
            header_peek = io.BytesIO(file_content).readline().decode('utf-8').rstrip("\n")
            sep = ',' if ',' in header_peek.split() else '\t'
        except Exception as e:
             st.error(f"Error reading file header: {e}")
             return None, None, None, None, None, None
        file_stream = io.BytesIO(file_content)

    st.write(f"Detected delimiter: '{sep}'")

    # Detect row mode
    # Need a fresh file object for detect_row_mode
    file_stream_for_detect = io.BytesIO(file_stream.getvalue())
    row_mode = detect_row_mode(file_stream_for_detect)
    st.write(f"Detected row mode: {row_mode.upper()}")

    # Get header to identify sample columns and gene column
    file_stream.seek(0) # Ensure stream is at the beginning
    header_line = io.TextIOWrapper(file_stream, encoding="utf-8", errors="replace").readline().rstrip("\n")
    cols = header_line.split(sep)
    # Assuming the first column is the gene ID/symbol column
    gene_col = cols[0]
    # Assuming sample columns start from the second column
    sample_cols_full = cols[1:]

    # Select samples based on n_samples parameter
    if n_samples > len(sample_cols_full):
        st.warning(f"Requested {n_samples} samples, but only {len(sample_cols_full)} available. Using all available samples.")
        selected_samples = sample_cols_full
    else:
        selected_samples = sample_cols_full[:n_samples]

    st.write(f"Processing {len(selected_samples)} samples.")
    st.write(f"Using gene column: '{gene_col}'")

    # Stream-select rows and columns
    # Need a fresh file object for stream_select_rows_columns
    file_stream_for_select = io.BytesIO(file_stream.getvalue())
    expr_sym_small = stream_select_rows_columns(file_stream_for_select, selected_samples, SIG_GENES, SYM2ENSG, row_mode, gene_col_name=gene_col, sep=sep)

    if expr_sym_small.empty:
        st.error("Failed to load expression data. Please check file format and contents.")
        return None, None, None, None, None, None

    st.subheader("Expression Matrix (Subset)")
    st.write("Shape (genes x samples):", expr_sym_small.shape)
    st.dataframe(expr_sym_small.head())

    # Pathway Scoring
    st.subheader("Pathway Activity Scores")
    P = pathway_scores(expr_sym_small, SIGS)
    st.write("Shape (pathways x samples):", P.shape)
    st.dataframe(P)

    # QUBO Formulation and Drug Selection
    st.subheader("Drug Selection Results")
    panel = example_drug_panel()
    drugs = list(panel.keys())

    all_sels = []
    all_bhats = []
    patient_reports_data = []

    # Run analysis for each selected sample
    if P.empty:
        st.warning("No pathway scores computed. Cannot perform drug selection.")
        return P, pd.DataFrame(), pd.DataFrame(), [], drugs, panel

    for sample_id in P.columns:
        z_path = patient_vector(P, sample_id)
        b_series = drug_benefit_prior(z_path, panel).reindex(drugs).fillna(0.0)
        R = build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10)
        b = b_series.to_numpy(float)
        q, Q = build_qubo(b, R, lam=lam_value)

        # Use exact solver for now
        x_star, e_star = exact_qubo_solve(b, R, lam=lam_value)
        sel = [drugs[i] for i, xi in enumerate(x_star) if xi == 1]
        all_sels.append(sel)
        all_bhats.append(b_series)

        # Prepare data for patient report
        top_pw = z_path.abs().sort_values(ascending=False).head(5)
        patient_reports_data.append({
            "patient": sample_id,
            "selected_drugs": ", ".join(sel) if sel else "(none)",
            "top_pathways": "; ".join([f"{p}:{z_path[p]:+.2f}" for p in top_pw.index]),
            **{f"b̂.{d}": float(b_series.get(d,0.0)) for d in drugs}
        })

    # Drug Selection Summary
    st.subheader("Drug Selection Frequency")
    if not all_sels:
        st.info("No drugs were selected for any patient.")
        freq_df = pd.DataFrame({"drug": drugs, "frequency": 0, "frequency_pct": 0.0})
    else:
        flat = [d for sel in all_sels for d in sel]
        freq = Counter(flat)
        freq_df = pd.DataFrame({"drug": drugs, "frequency": [freq[d] for d in drugs]})
        freq_df["frequency_pct"] = 100 * freq_df["frequency"] / len(selected_samples)
        freq_df = freq_df.sort_values("frequency_pct", ascending=False).reset_index(drop=True)
    st.dataframe(freq_df)

    # Patient-level Report
    st.subheader("Patient-level Report")
    patient_report_df = pd.DataFrame(patient_reports_data)
    st.dataframe(patient_report_df)

    return P, freq_df, patient_report_df, all_sels, drugs, panel

# Function to create a download link
def create_download_link(df, filename, text):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    href = f'<a href="data:file/csv;base64,{b64}" download="{filename}">{text}</a>'
    return href


# --- Streamlit UI ---
st.set_page_config(layout="wide")

st.title("Digital Twin Workflow: Gene Expression to Drug Selection")

st.markdown("""
This application demonstrates a workflow for identifying potential drug therapies
for cancer patients based on their gene expression data, using pathway analysis
and QUBO optimization.
""")

st.sidebar.header("Input Data")
uploaded_file = st.sidebar.file_uploader("Upload gene expression data (CSV, TSV, or gzipped)", type=["csv", "tsv", "gz"])

st.sidebar.header("Parameters")
n_samples = st.sidebar.slider("Number of samples to process", min_value=1, max_value=100, value=40, step=1)
lam_value = st.sidebar.slider("Lambda (penalty) value for QUBO", min_value=0.1, max_value=5.0, value=1.0, step=0.1)

st.header("Analysis Results")

if uploaded_file is not None:
    if st.button("Run Analysis"):
        with st.spinner(f"Running analysis for {n_samples} samples with lambda={lam_value}..."):
            pathway_scores_df, freq_df, patient_report_df, all_sels, drugs, panel = run_analysis(uploaded_file, n_samples, lam_value)

            if pathway_scores_df is not None:
                st.subheader("Visualizations")

                # Drug selection frequency barplot (Plotly)
                if freq_df is not None and not freq_df.empty:
                    try:
                        fig1 = px.bar(freq_df, x="drug", y="frequency_pct", title="Drug selection stability across patients")
                        fig1.update_layout(xaxis_title="Drug", yaxis_title="% patients selected", xaxis_tickangle=-45)
                        st.plotly_chart(fig1, use_container_width=True)
                    except Exception as e:
                        st.write(f"Could not generate frequency barplot: {e}")

                # Co-selection heatmap and dendrogram (Plotly/Matplotlib - dendrogram is complex in Plotly)
                if all_sels and drugs and panel:
                    try:
                        st.subheader("Co-selection Analysis")
                         # --- build co-selection counts ---
                        co_mat = pd.DataFrame(0, index=drugs, columns=drugs, dtype=int)
                        n_pat = max(1, len(all_sels))
                        for sel in all_sels:
                            uniq = list(dict.fromkeys(sel))
                            for i in range(len(uniq)):
                                for j in range(i, len(uniq)):
                                    di, dj = uniq[i], uniq[j]
                                    co_mat.loc[di, dj] += 1
                                    if i != j:
                                        co_mat.loc[dj, di] += 1

                        # normalize to % of patients
                        co_pct = co_mat / n_pat * 100.0
                        st.write("Co-selection matrix (% of patients):")
                        st.dataframe(co_pct.round(1))

                        # heatmap (Plotly)
                        fig2 = px.imshow(co_pct, text_auto=True, color_continuous_scale='Blues',
                                         title='Drug co-selection heatmap',
                                         labels=dict(x="Drug", y="Drug", color="% patients co-selected"))
                        st.plotly_chart(fig2, use_container_width=True)


                        # dendrogram (using Matplotlib for simplicity as Plotly dendrogram is complex)
                        try:
                             corr = np.corrcoef(co_pct.values)
                             corr = np.clip(corr, -1.0, 1.0)
                             dist = 1.0 - corr
                             Z = linkage(dist, method="average")

                             fig3, ax3 = plt.subplots(figsize=(8,5))
                             dendrogram(Z, labels=drugs, leaf_rotation=90, leaf_font_size=10,
                                        color_threshold=0.7 * np.max(Z[:,2]), ax=ax3)
                             ax3.set_title("Clustered dendrogram of drug co-selection")
                             ax3.set_ylabel("Distance (1 - correlation)")
                             plt.tight_layout()
                             st.pyplot(fig3)
                             plt.close(fig3) # Close figure

                             # clustered heatmap (Plotly)
                             order = leaves_list(Z)
                             co_pct_ordered = co_pct.iloc[order, order]

                             fig4 = px.imshow(co_pct_ordered, text_auto=True, color_continuous_scale='Blues',
                                              title='Clustered heatmap of drug co-selection',
                                              labels=dict(x="Drug", y="Drug", color="% patients co-selected"))
                             st.plotly_chart(fig4, use_container_width=True)
                             st.write("Clustered co-selection matrix:")
                             st.dataframe(co_pct_ordered.round(1))

                        except Exception as e:
                             st.write(f"Could not generate clustering plots (dendrogram/clustered heatmap): {e}")


                    except Exception as e:
                        st.write(f"Could not generate co-selection plots: {e}")

                # Selection size distribution histogram (Plotly)
                if all_sels:
                    try:
                        sizes = [len(set(sel)) for sel in all_sels]
                        size_hist = pd.Series(sizes).value_counts().sort_index()
                        size_hist_df = size_hist.reset_index()
                        size_hist_df.columns = ["# drugs selected", "# patients"]
                        fig5 = px.bar(size_hist_df, x="# drugs selected", y="# patients",
                                      title="Selection size distribution")
                        st.plotly_chart(fig5, use_container_width=True)
                    except Exception as e:
                        st.write(f"Could not generate selection size histogram: {e}")

                # Synthetic Classification Validation (Plotly)
                if all_sels and drugs and pathway_scores_df is not None and not pathway_scores_df.empty:
                    try:
                        st.subheader("Synthetic Classification Validation")
                        # 1) Construct patient-drug feature matrix
                        N = len(all_sels)
                        X = np.zeros((N, len(drugs)))
                        for i, sel in enumerate(all_sels):
                            for d in drugs: # Iterate through drugs to ensure correct indexing
                                if d in sel:
                                    X[i, drugs.index(d)] = 1

                        # 2) Synthetic binary labels
                        risk_scores = []
                        # Ensure we only process samples that were successfully processed in run_analysis
                        processed_sample_ids = pathway_scores_df.columns
                        if len(processed_sample_ids) < N:
                             st.warning(f"Only {len(processed_sample_ids)} samples processed for classification.")
                             # Adjust N and X to match processed samples if necessary, or skip classification
                             # For simplicity, skipping if mismatch or no processed samples
                             if len(processed_sample_ids) == 0:
                                  st.warning("No samples processed for synthetic classification.")
                                  raise ValueError("No processed samples") # Trigger exception to skip
                             N = len(processed_sample_ids)
                             # Rebuild X with correct size if needed, but currently X is built from all_sels,
                             # which should align with the number of processed samples if no errors occurred earlier.
                             # The warning/exception above handles the case where processed_sample_ids is empty.
                             # If there's a size mismatch while processed_sample_ids is not empty,
                             # it indicates a more complex issue not easily fixable here, so we'll skip.
                             st.warning("Skipping synthetic classification due to unexpected sample count mismatch.")
                             raise ValueError("Sample count mismatch") # Trigger exception to skip


                        for sid in processed_sample_ids:
                             z_path = patient_vector(pathway_scores_df, sid)
                             risk_scores.append(z_path.mean())
                        risk_scores = np.array(risk_scores)

                        # Handle case where all risk scores are the same
                        if np.std(risk_scores) == 0:
                             st.warning("Cannot generate synthetic labels: all pathway mean scores are the same.")
                             st.write("Synthetic labels (all 0):")
                             st.write(np.zeros(N, dtype=int))
                        else:
                             y = (risk_scores > np.median(risk_scores)).astype(int)
                             st.write("Synthetic labels (0=low risk, 1=high risk):")
                             st.write(np.bincount(y))

                             # 3) Train simple logistic regression
                             if len(np.unique(y)) > 1: # Only train if there's more than one class
                                 clf = LogisticRegression(max_iter=200)
                                 clf.fit(X[:N], y) # Use adjusted N
                                 y_pred = clf.predict_proba(X[:N])[:,1] # Use adjusted N

                                 # 4) Metrics
                                 roc_auc = roc_auc_score(y, y_pred)
                                 pr_auc = average_precision_score(y, y_pred)

                                 st.write(f"ROC-AUC: {roc_auc:.3f}")
                                 st.write(f"PR-AUC: {pr_auc:.3f}")

                                 # Optional barplot of learned coefficients (Plotly)
                                 coefs = pd.Series(clf.coef_[0], index=drugs).sort_values()
                                 coefs_df = coefs.reset_index()
                                 coefs_df.columns = ["Drug", "Weight"]
                                 fig6 = px.bar(coefs_df, y="Drug", x="Weight", orientation='h',
                                               color="Weight", color_continuous_scale=['salmon', 'steelblue'],
                                               title="Drug selection coefficients (synthetic label prediction)")
                                 st.plotly_chart(fig6, use_container_width=True)

                             else:
                                 st.warning("Classification skipped: only one unique synthetic label class found.")

                    except Exception as e:
                        st.write(f"Could not perform synthetic classification validation: {e}")

                # Downloadable reports
                if patient_report_df is not None and not patient_report_df.empty:
                    st.subheader("Download Reports")
                    st.markdown(create_download_link(patient_report_df, "patient_report.csv", "Download Patient Report (CSV)"), unsafe_allow_html=True)

                if freq_df is not None and not freq_df.empty:
                     st.markdown(create_download_link(freq_df, "drug_frequency_report.csv", "Download Drug Frequency Report (CSV)"), unsafe_allow_html=True)

                st.success("Analysis complete!")

else:
    st.info("Please upload a gene expression data file to begin.")

**Reasoning**:
The Matplotlib dendrogram is still being generated. Convert the Matplotlib dendrogram to Plotly for consistency and interactivity.



In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import gzip
import re
from collections import Counter
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
import matplotlib.pyplot as plt
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
import io
import base64
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assume the core logic functions from the notebook are available here.
# For demonstration, I'll include them directly. In a larger project,
# these would be in a separate module.

def norm_ensembl(x): return x.split('.',1)[0]
def norm_symbol(x):  return re.sub(r'[^A-Za-z0-9_-]+','', x)

def detect_row_mode(file_object, scan_rows=50000):
    """Detect row ID type (Ensembl vs symbol) from file-like object."""
    seen = Counter(); total = 0
    # Use a TextIOWrapper to treat the bytes stream as text
    # Ensure file_object is at the beginning
    file_object.seek(0)
    with io.TextIOWrapper(file_object, encoding="utf-8", errors="replace") as f:
        # Read the header line to skip it for row detection
        header = f.readline()
        # Use header=None because we already read the header
        reader = pd.read_csv(f, sep="\t", chunksize=200_000, usecols=[0], dtype=str, header=None)
        for ch in reader:
            v = ch.iloc[:,0].astype(str)
            vals = v.head(min(len(v), scan_rows-total)).tolist()
            total += len(vals)
            seen.update('ENSG' if x.startswith('ENSG') else 'OTHER' for x in vals)
            if total >= scan_rows: break
    ratio = seen['ENSG']/max(1,(seen['ENSG']+seen['OTHER']))
    mode = 'ensembl' if ratio >= 0.6 else 'symbol'
    return mode

def stream_select_rows_columns(file_object, selected_samples, sig_genes, sym2ensg, row_mode, gene_col_name="Name", sep='\t'):
    """Stream-select rows and columns from file-like object."""
    if row_mode == 'ensembl':
        target_rows = set(sym2ensg.get(g) for g in sig_genes if g in sym2ensg)
        normalize = norm_ensembl
    else:
        target_rows = set(sig_genes)
        normalize = norm_symbol

    # Ensure file_object is at the beginning
    file_object.seek(0)

    with io.TextIOWrapper(file_object, encoding="utf-8", errors="replace") as f:
        # Read header to find the actual gene_col_name and sample columns
        header = f.readline().rstrip("\n").split(sep)
        try:
            gene_col_idx = header.index(gene_col_name)
        except ValueError:
             # If gene_col_name not found, assume the first column is the gene column
            gene_col_name = header[0]
            gene_col_idx = 0

        # Filter header to include only the gene column and selected sample columns
        usecols_filter = [gene_col_name] + selected_samples

        # Read data in chunks, selecting only necessary columns
        # Use header=None as we've already read the header
        reader = pd.read_csv(f, sep=sep, chunksize=50_000, dtype=str, header=None)
        kept = []
        for i, ch in enumerate(reader):
            # Assign original header to the chunk
            ch.columns = header
            # Select only the columns we need
            ch = ch[usecols_filter]

            ch = ch.rename(columns={gene_col_name: "row_id"})

            # Ensure 'row_id' column exists after renaming
            if 'row_id' not in ch.columns:
                 st.error(f"Error: Could not find gene identifier column '{gene_col_name}' in the uploaded file.")
                 return pd.DataFrame() # Return empty DataFrame on error

            ids = ch["row_id"].astype(str).map(normalize)
            mask = ids.isin(target_rows)

            if mask.any():
                out = ch.loc[mask].copy()
                out["row_id"] = out["row_id"].map(normalize)
                kept.append(out)

    if not kept:
        st.warning("No signature rows matched the provided gene list.")
        return pd.DataFrame()

    expr_small = pd.concat(kept, axis=0, ignore_index=False).drop_duplicates(subset=["row_id"]).set_index("row_id")

    if row_mode == 'ensembl':
        ensg2sym = {v:k for k,v in SYM2ENSG.items()}
        expr_sym_small = expr_small.copy()
        expr_sym_small.index = [ensg2sym.get(e, e) for e in expr_small.index]
    else:
        expr_sym_small = expr_small.copy()

    expr_sym_small = expr_sym_small[~expr_sym_small.index.duplicated(keep="first")]
    return expr_sym_small


def zscore_by_gene(expr_symbols: pd.DataFrame) -> pd.DataFrame:
    E = expr_symbols.copy()
    E = E.apply(pd.to_numeric, errors="coerce")
    E = E.loc[~E.isna().all(axis=1)]
    gene_means = E.mean(axis=1)
    E = E.apply(lambda col: col.fillna(gene_means), axis=0)
    mu = E.mean(axis=1)
    sd = E.std(axis=1) + 1e-8
    return (E.sub(mu, axis=0)).div(sd, axis=0)

def pathway_scores(expr_symbols: pd.DataFrame, signatures: dict) -> pd.DataFrame:
    """Return pathways x samples (mean z across member genes present)."""
    Z = zscore_by_gene(expr_symbols)
    rows = []
    for pw, genes in signatures.items():
        present = [g for g in genes if g in Z.index]
        if present:
            s = Z.loc[present].mean(axis=0)
        else:
            s = pd.Series([np.nan]*Z.shape[1], index=Z.columns)
        s.name = pw
        rows.append(s)
    return pd.DataFrame(rows)

def example_drug_panel():
    return {
        "EGFRi": ["REACTOME_SIGNALING_BY_EGFR"],
        "ALKi":  ["REACTOME_SIGNALING_BY_ALK"],
        "MEKi":  ["REACTOME_MAPK1_MAPK3_SIGNALING"],
        "PI3Ki": ["REACTOME_PI3K_AKT_SIGNALING"],
        "mTORi": ["REACTOME_MTORC1_MEDIATED_SIGNALLING"],
        "PD1i":  ["REACTOME_PD1_SIGNALING"],
        "VEGFi": ["REACTOME_VEGFA_VEGFR2_SIGNALING_PATHWAY"],
        "FGFRi": ["REACTOME_SIGNALING_BY_FGFR"],
    }

def patient_vector(P: pd.DataFrame, sample_id: str) -> pd.Series:
    """z-normalize pathways across samples; return vector for one patient."""
    z = (P - P.mean(axis=1).values.reshape(-1,1)) / (P.std(axis=1).values.reshape(-1,1) + 1e-8)
    return z[sample_id].fillna(0.0)

def drug_benefit_prior(z_path: pd.Series, panel: dict) -> pd.Series:
    """Aggregate pathway z's per drug (ReLU to emphasize upregulated pathways)."""
    s = pd.Series({d: float(np.sum([max(z_path.get(p, 0.0), 0.0) for p in pws])) for d, pws in panel.items()})
    return s / s.max() if s.max() > 0 else s

def build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10):
    """Pairwise penalties for overlapping mechanisms; diagonal = sparsity."""
    K = len(drugs); R = np.zeros((K, K), dtype=float)
    for i in range(K):
        for j in range(i+1, K):
            overlap = len(set(panel[drugs[i]]) & set(panel[drugs[j]]))
            if overlap > 0:
                R[i, j] = R[j, i] = base_overlap * overlap
    for i in range(K):
        R[i, i] += sparsity
    return R

def build_qubo(b_hat, R, lam=1.0):
    """QUBO: minimize x^T (lam R) x + q^T x  where q = -b̂ + diag(lam R)."""
    Q = lam * R.copy()
    q = -b_hat.copy()
    diag = np.diag(Q).copy()
    np.fill_diagonal(Q, 0.0)  # keep off-diagonal in Q
    q += diag
    return q, np.triu(Q, 1)

def exact_qubo_solve(b_hat: np.ndarray, R: np.ndarray, lam: float = 1.0):
    """
    Exact minimization of QUBO:
      E(x) = x^T (lam R) x + q^T x, with q = -b̂ + diag(lam R)
    We enumerate all bitstrings (2^K). Returns best bitstring (as 0/1 np array).
    """
    K = len(b_hat)
    Q = lam * R.copy()
    q = -b_hat.copy() + np.diag(Q)
    np.fill_diagonal(Q, 0.0)  # keep only off-diagonal in Q

    best_e = np.inf
    best_x = None
    # vectorize partial precomputations
    upper_idx = np.triu_indices(K, 1)
    for mask in range(1 << K):
        # build x from bits
        x = np.fromiter(((mask >> i) & 1 for i in range(K)), dtype=np.int8)
        # E = x^T Q x + q^T x, where Q is strictly upper-triangular mirrored
        e = np.dot(q, x) + 2.0 * np.sum(Q[upper_idx] * (x[upper_idx[0]] * x[upper_idx[1]]))
        if e < best_e:
            best_e, best_x = e, x
    return best_x, float(best_e)


# Placeholder for CUDA-Q function if needed later, currently uses exact solve
def try_cudaq_qaoa(h, J, p=2, shots=2048, max_iters=60, seed=7):
    return None # Not implemented in this web app version

# --- Main analysis function ---
def run_analysis(uploaded_file, n_samples, lam_value):
    """
    Runs the full analysis workflow for the uploaded data.
    Returns pathway scores, drug selection summary, and patient report.
    """
    SIGS = {
        "REACTOME_SIGNALING_BY_EGFR": [
            "EGFR","ERBB2","ERBB3","GRB2","SOS1","SHC1","PTPN11","KRAS","NRAS","HRAS",
            "BRAF","MAP2K1","MAP2K2","MAPK1","MAPK3","PLCG1","PIK3CA","PIK3R1","AKT1","AKT2","AKT3","GAB1"
        ],
        "REACTOME_SIGNALING_BY_ALK": [
            "ALK","EML4","GRB2","SHC1","PIK3CA","PIK3R1","AKT1","AKT2","AKT3","STAT3","MAP2K1","MAPK1","MAPK3"
        ],
        "REACTOME_MAPK1_MAPK3_SIGNALING": [
            "BRAF","RAF1","MAP2K1","MAP2K2","MAPK1","MAPK3","DUSP6","DUSP4","FOS","JUN","EGFR"
        ],
        "REACTOME_PI3K_AKT_SIGNALING": [
            "PIK3CA","PIK3CB","PIK3CD","PIK3R1","PIK3R2","AKT1","AKT2","AKT3","PTEN","MTOR","RHEB"
        ],
        "REACTOME_MTORC1_MEDIATED_SIGNALLING": [
            "MTOR","RPTOR","MLST8","RHEB","TSC1","TSC2","EIF4EBP1","RPS6KB1","RPS6"
        ],
        "REACTOME_PD1_SIGNALING": [
            "PDCD1","CD274","PDCD1LG2","PDCD1LG2","JAK1","JAK2","STAT1","IFNG","GZMB","LAG3","TIGIT","CXCL9","CXCL10"
        ],
        "REACTOME_VEGFA_VEGFR2_SIGNALING_PATHWAY": [
            "VEGFA","KDR","FLT1","FLT4","PTPRB","PLCG1","MAP2K1","MAPK1","NOS3"
        ],
        "REACTOME_SIGNALING_BY_FGFR": [
            "FGFR1","FGFR2","FGFR3","FGFR4","FRS2","PLCG1","PIK3CA","PIK3R1","MAP2K1","MAPK1"
        ]
    }
    SIG_GENES = sorted({g for gs in SIGS.values() for g in gs})
    SYM2ENSG = {
        "EGFR":"ENSG00000146648","ERBB2":"ENSG00000141736","ERBB3":"ENSG00000065361","GRB2":"ENSG00000177885",
        "SOS1":"ENSG00000115904","SHC1":"ENSG00000154639","PTPN11":"ENSG00000179295","KRAS":"ENSG00000133703",
        "NRAS":"ENSG00000213281","HRAS":"ENSG00000174775","BRAF":"ENSG00000157764","MAP2K1":"ENSG00000169032",
        "MAP2K2":"ENSG00000126934","MAPK1":"ENSG00000100030","MAPK3":"ENSG00000102882","PLCG1":"ENSG00000124181",
        "PIK3CA":"ENSG00000121879","PIK3R1":"ENSG00000145675","AKT1":"ENSG00000142208","AKT2":"ENSG00000105221",
        "AKT3":"ENSG00000117020","GAB1":"ENSG00000117676",
        "ALK":"ENSG00000171094","EML4":"ENSG00000143924","STAT3":"ENSG00000168610",
        "DUSP6":"ENSG00000139318","DUSP4":"ENSG00000120875","FOS":"ENSG00000170345","JUN":"ENSG00000177606","RAF1":"ENSG00000132155",
        "PIK3CB":"ENSG00000119402","PIK3CD":"ENSG00000171608","PIK3R2":"ENSG00000189403","PTEN":"ENSG00000171862",
        "MTOR":"ENSG00000198793","RHEB":"ENSG00000106615",
        "RPTOR":"ENSG00000141564","MLST8":"ENSG00000105705","TSC1":"ENSG00000165699","TSC2":"ENSG00000103197",
        "EIF4EBP1":"ENSG00000187840","RPS6KB1":"ENSG00000108443","RPS6":"ENSG00000137154",
        "PDCD1":"ENSG00000276977","CD274":"ENSG00000120217","PDCD1LG2":"ENSG00000197646","JAK1":"ENSG00000162434",
        "JAK2":"ENSG00000096968","STAT1":"ENSG00000115415","IFNG":"ENSG00000111537","GZMB":"ENSG00000100453",
        "LAG3":"ENSG00000089692","TIGIT":"ENSG00000181847","CXCL9":"ENSG00000138755","CXCL10":"ENSG00000169245",
        "VEGFA":"ENSG00000112715","KDR":"ENSG00000128052","FLT1":"ENSG00000102755","FLT4":"ENSG00000037280",
        "PTPRB":"ENSG00000160593","NOS3":"ENSG00000164867",
        "FGFR1":"ENSG00000077782","FGFR2":"ENSG00000066468","FGFR3":"ENSG00000068078","FGFR4":"ENSG00000069535",
        "FRS2":"ENSG00000181873"
    }

    # Data Loading and Initial Processing
    # Handle potential gzip compression and determine delimiter
    file_content = uploaded_file.getvalue()
    if uploaded_file.name.endswith('.gz'):
        try:
            gz_file = gzip.GzipFile(fileobj=io.BytesIO(file_content))
            # Peek at the first line after decompression to determine delimiter
            with io.TextIOWrapper(gz_file, encoding="utf-8", errors="replace") as f:
                header_peek = f.readline().rstrip("\n")
                sep = ',' if ',' in header_peek.split() else '\t'
            # Reset gz_file for actual reading
            gz_file.seek(0)
            file_stream = io.BytesIO(gz_file.read())
        except Exception as e:
            st.error(f"Error reading gzipped file: {e}")
            return None, None, None, None, None, None
    else:
        # Determine delimiter for non-gzipped files
        try:
            header_peek = io.BytesIO(file_content).readline().decode('utf-8').rstrip("\n")
            sep = ',' if ',' in header_peek.split() else '\t'
        except Exception as e:
             st.error(f"Error reading file header: {e}")
             return None, None, None, None, None, None
        file_stream = io.BytesIO(file_content)

    st.write(f"Detected delimiter: '{sep}'")

    # Detect row mode
    # Need a fresh file object for detect_row_mode
    file_stream_for_detect = io.BytesIO(file_stream.getvalue())
    row_mode = detect_row_mode(file_stream_for_detect)
    st.write(f"Detected row mode: {row_mode.upper()}")

    # Get header to identify sample columns and gene column
    file_stream.seek(0) # Ensure stream is at the beginning
    header_line = io.TextIOWrapper(file_stream, encoding="utf-8", errors="replace").readline().rstrip("\n")
    cols = header_line.split(sep)
    # Assuming the first column is the gene ID/symbol column
    gene_col = cols[0]
    # Assuming sample columns start from the second column
    sample_cols_full = cols[1:]

    # Select samples based on n_samples parameter
    if n_samples > len(sample_cols_full):
        st.warning(f"Requested {n_samples} samples, but only {len(sample_cols_full)} available. Using all available samples.")
        selected_samples = sample_cols_full
    else:
        selected_samples = sample_cols_full[:n_samples]

    st.write(f"Processing {len(selected_samples)} samples.")
    st.write(f"Using gene column: '{gene_col}'")

    # Stream-select rows and columns
    # Need a fresh file object for stream_select_rows_columns
    file_stream_for_select = io.BytesIO(file_stream.getvalue())
    expr_sym_small = stream_select_rows_columns(file_stream_for_select, selected_samples, SIG_GENES, SYM2ENSG, row_mode, gene_col_name=gene_col, sep=sep)

    if expr_sym_small.empty:
        st.error("Failed to load expression data. Please check file format and contents.")
        return None, None, None, None, None, None

    st.subheader("Expression Matrix (Subset)")
    st.write("Shape (genes x samples):", expr_sym_small.shape)
    st.dataframe(expr_sym_small.head())

    # Pathway Scoring
    st.subheader("Pathway Activity Scores")
    P = pathway_scores(expr_sym_small, SIGS)
    st.write("Shape (pathways x samples):", P.shape)
    st.dataframe(P)

    # QUBO Formulation and Drug Selection
    st.subheader("Drug Selection Results")
    panel = example_drug_panel()
    drugs = list(panel.keys())

    all_sels = []
    all_bhats = []
    patient_reports_data = []

    # Run analysis for each selected sample
    if P.empty:
        st.warning("No pathway scores computed. Cannot perform drug selection.")
        return P, pd.DataFrame(), pd.DataFrame(), [], drugs, panel

    for sample_id in P.columns:
        z_path = patient_vector(P, sample_id)
        b_series = drug_benefit_prior(z_path, panel).reindex(drugs).fillna(0.0)
        R = build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10)
        b = b_series.to_numpy(float)
        q, Q = build_qubo(b, R, lam=lam_value)

        # Use exact solver for now
        x_star, e_star = exact_qubo_solve(b, R, lam=lam_value)
        sel = [drugs[i] for i, xi in enumerate(x_star) if xi == 1]
        all_sels.append(sel)
        all_bhats.append(b_series)

        # Prepare data for patient report
        top_pw = z_path.abs().sort_values(ascending=False).head(5)
        patient_reports_data.append({
            "patient": sample_id,
            "selected_drugs": ", ".join(sel) if sel else "(none)",
            "top_pathways": "; ".join([f"{p}:{z_path[p]:+.2f}" for p in top_pw.index]),
            **{f"b̂.{d}": float(b_series.get(d,0.0)) for d in drugs}
        })

    # Drug Selection Summary
    st.subheader("Drug Selection Frequency")
    if not all_sels:
        st.info("No drugs were selected for any patient.")
        freq_df = pd.DataFrame({"drug": drugs, "frequency": 0, "frequency_pct": 0.0})
    else:
        flat = [d for sel in all_sels for d in sel]
        freq = Counter(flat)
        freq_df = pd.DataFrame({"drug": drugs, "frequency": [freq[d] for d in drugs]})
        freq_df["frequency_pct"] = 100 * freq_df["frequency"] / len(selected_samples)
        freq_df = freq_df.sort_values("frequency_pct", ascending=False).reset_index(drop=True)
    st.dataframe(freq_df)

    # Patient-level Report
    st.subheader("Patient-level Report")
    patient_report_df = pd.DataFrame(patient_reports_data)
    st.dataframe(patient_report_df)

    return P, freq_df, patient_report_df, all_sels, drugs, panel

# Function to create a download link
def create_download_link(df, filename, text):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    href = f'<a href="data:file/csv;base64,{b64}" download="{filename}">{text}</a>'
    return href


# --- Streamlit UI ---
st.set_page_config(layout="wide")

st.title("Digital Twin Workflow: Gene Expression to Drug Selection")

st.markdown("""
This application demonstrates a workflow for identifying potential drug therapies
for cancer patients based on their gene expression data, using pathway analysis
and QUBO optimization.
""")

st.sidebar.header("Input Data")
uploaded_file = st.sidebar.file_uploader("Upload gene expression data (CSV, TSV, or gzipped)", type=["csv", "tsv", "gz"])

st.sidebar.header("Parameters")
n_samples = st.sidebar.slider("Number of samples to process", min_value=1, max_value=100, value=40, step=1)
lam_value = st.sidebar.slider("Lambda (penalty) value for QUBO", min_value=0.1, max_value=5.0, value=1.0, step=0.1)

st.header("Analysis Results")

if uploaded_file is not None:
    if st.button("Run Analysis"):
        with st.spinner(f"Running analysis for {n_samples} samples with lambda={lam_value}..."):
            pathway_scores_df, freq_df, patient_report_df, all_sels, drugs, panel = run_analysis(uploaded_file, n_samples, lam_value)

            if pathway_scores_df is not None:
                st.subheader("Visualizations")

                # Drug selection frequency barplot (Plotly)
                if freq_df is not None and not freq_df.empty:
                    try:
                        fig1 = px.bar(freq_df, x="drug", y="frequency_pct", title="Drug selection stability across patients")
                        fig1.update_layout(xaxis_title="Drug", yaxis_title="% patients selected", xaxis_tickangle=-45)
                        st.plotly_chart(fig1, use_container_width=True)
                    except Exception as e:
                        st.write(f"Could not generate frequency barplot: {e}")

                # Co-selection heatmap and dendrogram (Plotly)
                if all_sels and drugs and panel:
                    try:
                        st.subheader("Co-selection Analysis")
                         # --- build co-selection counts ---
                        co_mat = pd.DataFrame(0, index=drugs, columns=drugs, dtype=int)
                        n_pat = max(1, len(all_sels))
                        for sel in all_sels:
                            uniq = list(dict.fromkeys(sel))
                            for i in range(len(uniq)):
                                for j in range(i, len(uniq)):
                                    di, dj = uniq[i], uniq[j]
                                    co_mat.loc[di, dj] += 1
                                    if i != j:
                                        co_mat.loc[dj, di] += 1

                        # normalize to % of patients
                        co_pct = co_mat / n_pat * 100.0
                        st.write("Co-selection matrix (% of patients):")
                        st.dataframe(co_pct.round(1))

                        # heatmap (Plotly)
                        fig2 = px.imshow(co_pct, text_auto=True, color_continuous_scale='Blues',
                                         title='Drug co-selection heatmap',
                                         labels=dict(x="Drug", y="Drug", color="% patients co-selected"))
                        st.plotly_chart(fig2, use_container_width=True)


                        # dendrogram (Plotly)
                        try:
                            corr = np.corrcoef(co_pct.values)
                            corr = np.clip(corr, -1.0, 1.0)
                            dist = 1.0 - corr
                            Z = linkage(dist, method="average")

                            # Create Plotly Dendrogram
                            fig3 = go.Figure(data=go.Heatmap(
                                z=co_pct.values,
                                x=co_pct.columns,
                                y=co_pct.index,
                                colorscale='Blues',
                                colorbar=dict(title="% patients co-selected")
                            ))

                            # Add dendrogram using Plotly figure factory (requires plotly-scientific)
                            # from plotly.figure_factory import create_dendrogram
                            # fig3 = create_dendrogram(co_pct.values, orientation='bottom', labels=co_pct.index)
                            # fig3.update_layout(title='Clustered dendrogram of drug co-selection',
                            #                    xaxis_title="Drug", yaxis_title="Distance (1 - correlation)")
                            # st.plotly_chart(fig3, use_container_width=True)

                            # Manual Plotly Dendrogram creation (more complex, using Matplotlib for now)
                            # Reverting to Matplotlib dendrogram as Plotly dendrogram requires specific data format or fig_factory
                            fig3_mpl, ax3_mpl = plt.subplots(figsize=(8,5))
                            dendrogram(Z, labels=drugs, leaf_rotation=90, leaf_font_size=10,
                                       color_threshold=0.7 * np.max(Z[:,2]), ax=ax3_mpl)
                            ax3_mpl.set_title("Clustered dendrogram of drug co-selection (Matplotlib)")
                            ax3_mpl.set_ylabel("Distance (1 - correlation)")
                            plt.tight_layout()
                            st.pyplot(fig3_mpl)
                            plt.close(fig3_mpl) # Close figure


                            # clustered heatmap (Plotly)
                            order = leaves_list(Z)
                            co_pct_ordered = co_pct.iloc[order, order]

                            fig4 = px.imshow(co_pct_ordered, text_auto=True, color_continuous_scale='Blues',
                                              title='Clustered heatmap of drug co-selection',
                                              labels=dict(x="Drug", y="Drug", color="% patients co-selected"))
                            st.plotly_chart(fig4, use_container_width=True)
                            st.write("Clustered co-selection matrix:")
                            st.dataframe(co_pct_ordered.round(1))

                        except Exception as e:
                             st.write(f"Could not generate clustering plots (dendrogram/clustered heatmap): {e}")


                    except Exception as e:
                        st.write(f"Could not generate co-selection plots: {e}")

                # Selection size distribution histogram (Plotly)
                if all_sels:
                    try:
                        sizes = [len(set(sel)) for sel in all_sels]
                        size_hist = pd.Series(sizes).value_counts().sort_index()
                        size_hist_df = size_hist.reset_index()
                        size_hist_df.columns = ["# drugs selected", "# patients"]
                        fig5 = px.bar(size_hist_df, x="# drugs selected", y="# patients",
                                      title="Selection size distribution")
                        fig5.update_layout(xaxis=dict(tickmode='linear')) # Ensure all integer ticks are shown
                        st.plotly_chart(fig5, use_container_width=True)
                    except Exception as e:
                        st.write(f"Could not generate selection size histogram: {e}")

                # Synthetic Classification Validation (Plotly)
                if all_sels and drugs and pathway_scores_df is not None and not pathway_scores_df.empty:
                    try:
                        st.subheader("Synthetic Classification Validation")
                        # 1) Construct patient-drug feature matrix
                        N = len(all_sels)
                        X = np.zeros((N, len(drugs)))
                        for i, sel in enumerate(all_sels):
                            for d in drugs: # Iterate through drugs to ensure correct indexing
                                if d in sel:
                                    X[i, drugs.index(d)] = 1

                        # 2) Synthetic binary labels
                        risk_scores = []
                        # Ensure we only process samples that were successfully processed in run_analysis
                        processed_sample_ids = pathway_scores_df.columns
                        if len(processed_sample_ids) < N:
                             st.warning(f"Only {len(processed_sample_ids)} samples processed for classification.")
                             # Adjust N and X to match processed samples if necessary, or skip classification
                             # For simplicity, skipping if mismatch or no processed samples
                             if len(processed_sample_ids) == 0:
                                  st.warning("No samples processed for synthetic classification.")
                                  raise ValueError("No processed samples") # Trigger exception to skip
                             N = len(processed_sample_ids)
                             # Rebuild X with correct size if needed, but currently X is built from all_sels,
                             # which should align with the number of processed samples if no errors occurred earlier.
                             # The warning/exception above handles the case where processed_sample_ids is empty.
                             # If there's a size mismatch while processed_sample_ids is not empty,
                             # it indicates a more complex issue not easily fixable here, so we'll skip.
                             st.warning("Skipping synthetic classification due to unexpected sample count mismatch.")
                             raise ValueError("Sample count mismatch") # Trigger exception to skip


                        for sid in processed_sample_ids:
                             z_path = patient_vector(pathway_scores_df, sid)
                             risk_scores.append(z_path.mean())
                        risk_scores = np.array(risk_scores)

                        # Handle case where all risk scores are the same
                        if np.std(risk_scores) == 0:
                             st.warning("Cannot generate synthetic labels: all pathway mean scores are the same.")
                             st.write("Synthetic labels (all 0):")
                             st.write(np.zeros(N, dtype=int))
                        else:
                             y = (risk_scores > np.median(risk_scores)).astype(int)
                             st.write("Synthetic labels (0=low risk, 1=high risk):")
                             st.write(np.bincount(y))

                             # 3) Train simple logistic regression
                             if len(np.unique(y)) > 1: # Only train if there's more than one class
                                 clf = LogisticRegression(max_iter=200)
                                 clf.fit(X[:N], y) # Use adjusted N
                                 y_pred = clf.predict_proba(X[:N])[:,1] # Use adjusted N

                                 # 4) Metrics
                                 roc_auc = roc_auc_score(y, y_pred)
                                 pr_auc = average_precision_score(y, y_pred)

                                 st.write(f"ROC-AUC: {roc_auc:.3f}")
                                 st.write(f"PR-AUC: {pr_auc:.3f}")

                                 # Optional barplot of learned coefficients (Plotly)
                                 coefs = pd.Series(clf.coef_[0], index=drugs).sort_values()
                                 coefs_df = coefs.reset_index()
                                 coefs_df.columns = ["Drug", "Weight"]
                                 fig6 = px.bar(coefs_df, y="Drug", x="Weight", orientation='h',
                                               color="Weight", color_continuous_scale=['salmon', 'steelblue'],
                                               title="Drug selection coefficients (synthetic label prediction)")
                                 st.plotly_chart(fig6, use_container_width=True)

                             else:
                                 st.warning("Classification skipped: only one unique synthetic label class found.")

                    except Exception as e:
                        st.write(f"Could not perform synthetic classification validation: {e}")

                # Downloadable reports
                if patient_report_df is not None and not patient_report_df.empty:
                    st.subheader("Download Reports")
                    st.markdown(create_download_link(patient_report_df, "patient_report.csv", "Download Patient Report (CSV)"), unsafe_allow_html=True)

                if freq_df is not None and not freq_df.empty:
                     st.markdown(create_download_link(freq_df, "drug_frequency_report.csv", "Download Drug Frequency Report (CSV)"), unsafe_allow_html=True)

                st.success("Analysis complete!")

else:
    st.info("Please upload a gene expression data file to begin.")

## Consider scalability and performance

### Subtask:
Evaluate the current implementation for scalability and performance limitations and propose potential optimizations or alternative approaches.


**Reasoning**:
Evaluate the current implementation for scalability and performance limitations and propose potential optimizations or alternative approaches by reviewing the code and identifying bottlenecks.



In [ ]:
# Reviewing the code for scalability and performance limitations.

# 1. Data Loading and Initial Processing:
# - Reading the entire file into memory with uploaded_file.getvalue() can be a bottleneck for very large files.
# - io.BytesIO(gz_file.read()) for gzipped files also reads the entire decompressed content into memory.
# - The chunking in stream_select_rows_columns helps with row selection but still processes chunks in memory.
# - The approach of reading the header twice (once for mode detection, once for column names) involves redundant file reads.

# 2. Pathway Scoring:
# - zscore_by_gene copies the DataFrame. For large DataFrames, this can be memory intensive.
# - Mean imputation and z-scoring are done in memory.
# - pathway_scores iterates through pathways and genes, which is efficient for the current small SIGS but could be slower with many pathways or very large gene lists per pathway.

# 3. QUBO Formulation and Solving:
# - exact_qubo_solve enumerates all 2^K possible drug combinations. K is the number of drugs.
# - For K=8 (current panel size), 2^8 = 256 combinations, which is very fast.
# - If the drug panel size increases significantly (e.g., K=25), 2^25 is huge, making the exact solver infeasible. This is a major scalability limitation.
# - The QUBO matrix R and vector q are built in memory. For a very large number of drugs, these could become large, but the primary bottleneck is the 2^K enumeration.

# 4. Multi-patient Analysis:
# - The analysis is run sequentially for each patient. This is fine for a small number of samples but can be slow for thousands or millions of samples.
# - Storing all_sels and all_bhats in memory for all patients can consume significant memory for a large cohort.

# 5. Visualizations:
# - Generating plots using Plotly or Matplotlib is generally efficient for the current scale.
# - Creating intermediate DataFrames for plotting (e.g., for co-selection) adds memory overhead.

# Proposed Optimizations and Alternative Approaches:

# 1. Data Loading:
# - For very large files, consider using Dask DataFrames or iterating over chunks without reading the entire file into memory first.
# - Optimize header reading to minimize file pointer manipulation or redundant reads.

# 2. Pathway Scoring:
# - Explore libraries like Dask or Vaex for out-of-core processing if the expression matrix becomes too large for memory.
# - If using pandas, ensure operations are vectorized where possible (already done for z-scoring).

# 3. QUBO Solving:
# - For larger drug panels (K > ~20-25), the exact solver is not scalable.
# - Alternative solvers are needed:
#     - Approximate classical solvers (e.g., simulated annealing, genetic algorithms, specialized QUBO solvers like those in dimod or PyQUBO).
#     - Quantum-inspired optimization (QIO) algorithms on classical hardware (e.g., Fujitsu Digital Annealer, Hitachi CMOS Annealing).
#     - Quantum computing (QAOA on actual quantum hardware or simulators, as hinted in the original notebook, but currently not used in the web app).
# - The choice of solver depends on the desired scale of the drug panel and available hardware/software.

# 4. Multi-patient Analysis:
# - Parallelize the patient analysis loop using libraries like concurrent.futures or joblib.
# - If processing a massive cohort, consider distributed computing frameworks like Spark or Dask.
# - Process patient results incrementally or store them in a database instead of keeping all results in memory.

# 5. Memory Management:
# - Explicitly delete large objects (DataFrames, arrays) that are no longer needed.
# - Use more memory-efficient data types if possible (e.g., lower precision floats if appropriate).

# 6. Code Optimization:
# - Profile the code to identify the exact bottlenecks for typical dataset sizes.
# - Optimize specific functions identified as slow (e.g., the exact_qubo_solve loop for slightly larger K before it becomes completely infeasible).

# Documenting limitations and solutions:
print("\n--- Scalability and Performance Limitations ---")
print("1. Data Loading: Reading entire large files into memory (especially gzipped) can cause OutOfMemory errors.")
print("2. Exact QUBO Solver: Scales exponentially with the number of drugs (2^K). Infeasible for K > ~20-25.")
print("3. Multi-patient Processing: Sequential processing can be slow for large patient cohorts.")
print("4. Memory Usage: Storing full expression matrix, pathway scores, and all patient results in memory can be limiting.")

print("\n--- Proposed Optimizations and Alternative Approaches ---")
print("1. Data Loading: Use Dask or chunked processing without full in-memory read for large files.")
print("2. QUBO Solver: Implement approximate classical solvers (simulated annealing, genetic algorithms) or QIO solvers for larger drug panels.")
print("3. Multi-patient Processing: Parallelize the patient loop (concurrent.futures, joblib) or use distributed computing (Spark, Dask) for large cohorts.")
print("4. Memory Management: Use Dask/Vaex for out-of-core processing, process results incrementally, explicitly delete objects.")
print("5. Code Optimization: Profile and optimize specific functions; use vectorized operations.")

## Deploy the web application

### Subtask:
Choose a hosting platform (e.g., Heroku, AWS, Google Cloud Platform) and deploy the web application so that it can be accessed by users.


**Reasoning**:
The current subtask is to deploy the Streamlit application. Streamlit Community Cloud is the simplest option for deploying a Streamlit app directly from a GitHub repository. This involves creating a `requirements.txt` file, ensuring the Streamlit code is in a Python file (e.g., `app.py`), setting up a Git repository, and deploying via the Streamlit Community Cloud dashboard. I will create the `requirements.txt` and `app.py` files in the current environment.



In [ ]:
# Save the Streamlit application code to app.py
# Regenerate the string content, being careful with escapes and quotes.
streamlit_code = """
import streamlit as st
import pandas as pd
import numpy as np
import gzip
import re
from collections import Counter
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
import matplotlib.pyplot as plt
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
import io
import base64
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assume the core logic functions from the notebook are available here.
# For demonstration, I'll include them directly. In a larger project,
# these would be in a separate module.

def norm_ensembl(x): return x.split('.',1)[0]
def norm_symbol(x):  return re.sub(r'[^A-Za-z0-9_-]+','', x)

def detect_row_mode(file_object, scan_rows=50000):
    \\"\\"\\"Detect row ID type (Ensembl vs symbol) from file-like object.\\"\\"\\"
    seen = Counter(); total = 0
    # Use a TextIOWrapper to treat the bytes stream as text
    # Ensure file_object is at the beginning
    file_object.seek(0)
    with io.TextIOWrapper(file_object, encoding="utf-8", errors="replace") as f:
        # Read the header line to skip it for row detection
        header = f.readline()
        # Use header=None because we already read the header
        reader = pd.read_csv(f, sep="\\t", chunksize=200_000, usecols=[0], dtype=str, header=None)
        for ch in reader:
            v = ch.iloc[:,0].astype(str)
            vals = v.head(min(len(v), scan_rows-total)).tolist()
            total += len(vals)
            seen.update('ENSG' if x.startswith('ENSG') else 'OTHER' for x in vals)
            if total >= scan_rows: break
    ratio = seen['ENSG']/max(1,(seen['ENSG']+seen['OTHER']))
    mode = 'ensembl' if ratio >= 0.6 else 'symbol'
    return mode

def stream_select_rows_columns(file_object, selected_samples, sig_genes, sym2ensg, row_mode, gene_col_name="Name", sep='\\t'):
    \\"\\"\\"Stream-select rows and columns from file-like object.\\"\\"\\"
    if row_mode == 'ensembl':
        target_rows = set(sym2ensg.get(g) for g in sig_genes if g in sym2ensg)
        normalize = norm_ensembl
    else:
        target_rows = set(sig_genes)
        normalize = norm_symbol

    # Ensure file_object is at the beginning
    file_object.seek(0)

    with io.TextIOWrapper(file_object, encoding="utf-8", errors="replace") as f:
        # Read header to find the actual gene_col_name and sample columns
        header = f.readline().rstrip("\\\\n").split(sep)
        try:
            gene_col_idx = header.index(gene_col_name)
        except ValueError:
             # If gene_col_name not found, assume the first column is the gene column
            gene_col_name = header[0]
            gene_col_idx = 0

        # Filter header to include only the gene column and selected sample columns
        usecols_filter = [gene_col_name] + selected_samples

        # Read data in chunks, selecting only necessary columns
        # Use header=None as we've already read the header
        reader = pd.read_csv(f, sep=sep, chunksize=50_000, dtype=str, header=None)
        kept = []
        for i, ch in enumerate(reader):
            # Assign original header to the chunk
            ch.columns = header
            # Select only the columns we need
            ch = ch[usecols_filter]

            ch = ch.rename(columns={gene_col_name: "row_id"})

            # Ensure 'row_id' column exists after renaming
            if 'row_id' not in ch.columns:
                 st.error(f"Error: Could not find gene identifier column '{gene_col_name}' in the uploaded file.")
                 return pd.DataFrame() # Return empty DataFrame on error

            ids = ch["row_id"].astype(str).map(normalize)
            mask = ids.isin(target_rows)

            if mask.any():
                out = ch.loc[mask].copy()
                out["row_id"] = out["row_id"].map(normalize)
                kept.append(out)

    if not kept:
        st.warning("No signature rows matched the provided gene list.")
        return pd.DataFrame()

    expr_small = pd.concat(kept, axis=0, ignore_index=False).drop_duplicates(subset=["row_id"]).set_index("row_id")

    if row_mode == 'ensembl':
        ensg2sym = {v:k for k,v in sym2ensg.items()}
        expr_sym_small = expr_small.copy()
        expr_sym_small.index = [ensg2sym.get(e, e) for e in expr_sym_small.index]
    else:
        expr_sym_small = expr_small.copy()

    expr_sym_small = expr_sym_small[~expr_sym_small.index.duplicated(keep="first")]
    return expr_sym_small


def zscore_by_gene(expr_symbols: pd.DataFrame) -> pd.DataFrame:
    E = expr_symbols.copy()
    E = E.apply(pd.to_numeric, errors="coerce")
    E = E.loc[~E.isna().all(axis=1)]
    gene_means = E.mean(axis=1)
    E = E.apply(lambda col: col.fillna(gene_means), axis=0)
    mu = E.mean(axis=1)
    sd = E.std(axis=1) + 1e-8
    return (E.sub(mu, axis=0)).div(sd, axis=0)

def pathway_scores(expr_symbols: pd.DataFrame, signatures: dict) -> pd.DataFrame:
    \"\"\"Return pathways x samples (mean z across member genes present).\"\"\"
    Z = zscore_by_gene(expr_symbols)
    rows = []
    for pw, genes in signatures.items():
        present = [g for g in genes if g in Z.index]
        if present:
            s = Z.loc[present].mean(axis=0)
        else:
            s = pd.Series([np.nan]*Z.shape[1], index=Z.columns)
        s.name = pw
        rows.append(s)
    return pd.DataFrame(rows)

def example_drug_panel():
    return {
        "EGFRi": ["REACTOME_SIGNALING_BY_EGFR"],
        "ALKi":  ["REACTOME_SIGNALING_BY_ALK"],
        "MEKi":  ["REACTOME_MAPK1_MAPK3_SIGNALING"],
        "PI3Ki": ["REACTOME_PI3K_AKT_SIGNALING"],
        "mTORi": ["REACTOME_MTORC1_MEDIATED_SIGNALLING"],
        "PD1i":  ["REACTOME_PD1_SIGNALING"],
        "VEGFi": ["REACTOME_VEGFA_VEGFR2_SIGNALING_PATHWAY"],
        "FGFRi": ["REACTOME_SIGNALING_BY_FGFR"],
    }

def patient_vector(P: pd.DataFrame, sample_id: str) -> pd.Series:
    \"\"\"z-normalize pathways across samples; return vector for one patient.\"\"\"
    z = (P - P.mean(axis=1).values.reshape(-1,1)) / (P.std(axis=1).values.reshape(-1,1) + 1e-8)
    return z[sample_id].fillna(0.0)

def drug_benefit_prior(z_path: pd.Series, panel: dict) -> pd.Series:
    \"\"\"Aggregate pathway z's per drug (ReLU to emphasize upregulated pathways).\"\"\"
    s = pd.Series({d: float(np.sum([max(z_path.get(p, 0.0), 0.0) for p in pws])) for d, pws in panel.items()})
    return s / s.max() if s.max() > 0 else s

def build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10):
    \"\"\"Pairwise penalties for overlapping mechanisms; diagonal = sparsity.\"\"\"
    K = len(drugs); R = np.zeros((K, K), dtype=float)
    for i in range(K):
        for j in range(i+1, K):
            overlap = len(set(panel[drugs[i]]) & set(panel[drugs[j]]))
            if overlap > 0:
                R[i, j] = R[j, i] = base_overlap * overlap
    for i in range(K):
        R[i, i] += sparsity
    return R

def build_qubo(b_hat, R, lam=1.0):
    \"\"\"QUBO: minimize x^T (lam R) x + q^T x  where q = -b̂ + diag(lam R).\"\"\"
    Q = lam * R.copy()
    q = -b_hat.copy()
    diag = np.diag(Q).copy()
    np.fill_diagonal(Q, 0.0)  # keep off-diagonal in Q
    q += diag
    return q, np.triu(Q, 1)

def exact_qubo_solve(b_hat: np.ndarray, R: np.ndarray, lam: float = 1.0):
    \"\"\"
    Exact minimization of QUBO:
      E(x) = x^T (lam R) x + q^T x, with q = -b̂ + diag(lam R)
    We enumerate all bitstrings (2^K). Returns best bitstring (as 0/1 np array).
    \"\"\"
    K = len(b_hat)
    Q = lam * R.copy()
    q = -b_hat.copy() + np.diag(Q)
    np.fill_diagonal(Q, 0.0)  # keep only off-diagonal in Q

    best_e = np.inf
    best_x = None
    # vectorize partial precomputations
    upper_idx = np.triu_indices(K, 1)
    for mask in range(1 << K):
        # build x from bits
        x = np.fromiter(((mask >> i) & 1 for i in range(K)), dtype=np.int8)
        # E = x^T Q x + q^T x, where Q is strictly upper-triangular mirrored
        e = np.dot(q, x) + 2.0 * np.sum(Q[upper_idx] * (x[upper_idx[0]] * x[upper_idx[1]]))
        if e < best_e:
            best_e, best_x = e, x
    return best_x, float(best_e)


# Placeholder for CUDA-Q function if needed later, currently uses exact solve
def try_cudaq_qaoa(h, J, p=2, shots=2048, max_iters=60, seed=7):
    return None # Not implemented in this web app version

# --- Main analysis function ---
def run_analysis(uploaded_file, n_samples, lam_value):
    \"\"\"
    Runs the full analysis workflow for the uploaded data.
    Returns pathway scores, drug selection summary, and patient report.
    \"\"\"
    SIGS = {
        "REACTOME_SIGNALING_BY_EGFR": [
            "EGFR","ERBB2","ERBB3","GRB2","SOS1","SHC1","PTPN11","KRAS","NRAS","HRAS",
            "BRAF","MAP2K1","MAP2K2","MAPK1","MAPK3","PLCG1","PIK3CA","PIK3R1","AKT1","AKT2","AKT3","GAB1"
        ],
        "REACTOME_SIGNALING_BY_ALK": [
            "ALK","EML4","GRB2","SHC1","PIK3CA","PIK3R1","AKT1","AKT2","AKT3","STAT3","MAP2K1","MAPK1","MAPK3"
        ],
        "REACTOME_MAPK1_MAPK3_SIGNALING": [
            "BRAF","RAF1","MAP2K1","MAP2K2","MAPK1","MAPK3","DUSP6","DUSP4","FOS","JUN","EGFR"
        ],
        "REACTOME_PI3K_AKT_SIGNALING": [
            "PIK3CA","PIK3CB","PIK3CD","PIK3R1","PIK3R2","AKT1","AKT2","AKT3","PTEN","MTOR","RHEB"
        ],
        "REACTOME_MTORC1_MEDIATED_SIGNALLING": [
            "MTOR","RPTOR","MLST8","RHEB","TSC1","TSC2","EIF4EBP1","RPS6KB1","RPS6"
        ],
        "REACTOME_PD1_SIGNALING": [
            "PDCD1","CD274","PDCD1LG2","PDCD1LG2","JAK1","JAK2","STAT1","IFNG","GZMB","LAG3","TIGIT","CXCL9","CXCL10"
        ],
        "REACTOME_VEGFA_VEGFR2_SIGNALING_PATHWAY": [
            "VEGFA","KDR","FLT1","FLT4","PTPRB","PLCG1","MAP2K1","MAPK1","NOS3"
        ],
        "REACTOME_SIGNALING_BY_FGFR": [
            "FGFR1","FGFR2","FGFR3","FGFR4","FRS2","PLCG1","PIK3CA","PIK3R1","MAP2K1","MAPK1"
        ]
    }
    SIG_GENES = sorted({g for gs in SIGS.values() for g in gs})
    SYM2ENSG = {
        "EGFR":"ENSG00000146648","ERBB2":"ENSG00000141736","ERBB3":"ENSG00000065361","GRB2":"ENSG00000177885",
        "SOS1":"ENSG00000115904","SHC1":"ENSG00000154639","PTPN11":"ENSG00000179295","KRAS":"ENSG00000133703",
        "NRAS":"ENSG00000213281","HRAS":"ENSG00000174775","BRAF":"ENSG00000157764","MAP2K1":"ENSG00000169032",
        "MAP2K2":"ENSG00000126934","MAPK1":"ENSG00000100030","MAPK3":"ENSG00000102882","PLCG1":"ENSG00000124181",
        "PIK3CA":"ENSG00000121879","PIK3R1":"ENSG00000145675","AKT1":"ENSG00000142208","AKT2":"ENSG00000105221",
        "AKT3":"ENSG00000117020","GAB1":"ENSG00000117676",
        "ALK":"ENSG00000171094","EML4":"ENSG00000143924","STAT3":"ENSG00000168610",
        "DUSP6":"ENSG00000139318","DUSP4":"ENSG00000120875","FOS":"ENSG00000170345","JUN":"ENSG00000177606","RAF1":"ENSG00000132155",
        "PIK3CB":"ENSG00000119402","PIK3CD":"ENSG00000171608","PIK3R2":"ENSG00000189403","PTEN":"ENSG00000171862",
        "MTOR":"ENSG00000198793","RHEB":"ENSG00000106615",
        "RPTOR":"ENSG00000141564","MLST8":"ENSG00000105705","TSC1":"ENSG00000165699","TSC2":"ENSG00000103197",
        "EIF4EBP1":"ENSG00000187840","RPS6KB1":"ENSG00000108443","RPS6":"ENSG00000137154",
        "PDCD1":"ENSG00000276977","CD274":"ENSG00000120217","PDCD1LG2":"ENSG00000197646","JAK1":"ENSG00000162434",
        "JAK2":"ENSG00000096968","STAT1":"ENSG00000115415","IFNG":"ENSG00000111537","GZMB":"ENSG00000100453",
        "LAG3":"ENSG00000089692","TIGIT":"ENSG00000181847","CXCL9":"ENSG00000138755","CXCL10":"ENSG00000169245",
        "VEGFA":"ENSG00000112715","KDR":"ENSG00000128052","FLT1":"ENSG00000102755","FLT4":"ENSG00000037280",
        "PTPRB":"ENSG00000160593","NOS3":"ENSG00000164867",
        "FGFR1":"ENSG00000077782","FGFR2":"ENSG00000066468","FGFR3":"ENSG00000068078","FGFR4":"ENSG00000069535",
        "FRS2":"ENSG00000181873"
    }

    # Data Loading and Initial Processing
    # Handle potential gzip compression and determine delimiter
    file_content = uploaded_file.getvalue()
    if uploaded_file.name.endswith('.gz'):
        try:
            gz_file = gzip.GzipFile(fileobj=io.BytesIO(file_content))
            # Peek at the first line after decompression to determine delimiter
            with io.TextIOWrapper(gz_file, encoding="utf-8", errors="replace") as f:
                header_peek = f.readline().rstrip("\\\\n")
                sep = ',' if ',' in header_peek.split() else '\\t'
            # Reset gz_file for actual reading
            gz_file.seek(0)
            file_stream = io.BytesIO(gz_file.read())
        except Exception as e:
            st.error(f"Error reading gzipped file: {e}")
            return None, None, None, None, None, None
    else:
        # Determine delimiter for non-gzipped files
        try:
            header_peek = io.BytesIO(file_content).readline().decode('utf-8').rstrip("\\\\n")
            sep = ',' if ',' in header_peek.split() else '\\t'
        except Exception as e:
             st.error(f"Error reading file header: {e}")
             return None, None, None, None, None, None
        file_stream = io.BytesIO(file_content)

    st.write(f"Detected delimiter: '{sep}'")

    # Detect row mode
    # Need a fresh file object for detect_row_mode
    file_stream_for_detect = io.BytesIO(file_stream.getvalue())
    row_mode = detect_row_mode(file_stream_for_detect)
    st.write(f"Detected row mode: {row_mode.upper()}")

    # Get header to identify sample columns and gene column
    file_stream.seek(0) # Ensure stream is at the beginning
    header_line = io.TextIOWrapper(file_stream, encoding="utf-8", errors="replace").readline().rstrip("\\\\n")
    cols = header_line.split(sep)
    # Assuming the first column is the gene ID/symbol column
    gene_col = cols[0]
    # Assuming sample columns start from the second column
    sample_cols_full = cols[1:]

    # Select samples based on n_samples parameter
    if n_samples > len(sample_cols_full):
        st.warning(f"Requested {n_samples} samples, but only {len(sample_cols_full)} available. Using all available samples.")
        selected_samples = sample_cols_full
    else:
        selected_samples = sample_cols_full[:n_samples]

    st.write(f"Processing {len(selected_samples)} samples.")
    st.write(f"Using gene column: '{gene_col}'")

    # Stream-select rows and columns
    # Need a fresh file object for stream_select_rows_columns
    file_stream_for_select = io.BytesIO(file_stream.getvalue())
    expr_sym_small = stream_select_rows_columns(file_stream_for_select, selected_samples, SIG_GENES, SYM2ENSG, row_mode, gene_col_name=gene_col, sep=sep)

    if expr_sym_small.empty:
        st.error("Failed to load expression data. Please check file format and contents.")
        return None, None, None, None, None, None

    st.subheader("Expression Matrix (Subset)")
    st.write("Shape (genes x samples):", expr_sym_small.shape)
    st.dataframe(expr_sym_small.head())

    # Pathway Scoring
    st.subheader("Pathway Activity Scores")
    P = pathway_scores(expr_sym_small, SIGS)
    st.write("Shape (pathways x samples):", P.shape)
    st.dataframe(P)

    # QUBO Formulation and Drug Selection
    st.subheader("Drug Selection Results")
    panel = example_drug_panel()
    drugs = list(panel.keys())

    all_sels = []
    all_bhats = []
    patient_reports_data = []

    # Run analysis for each selected sample
    if P.empty:
        st.warning("No pathway scores computed. Cannot perform drug selection.")
        return P, pd.DataFrame(), pd.DataFrame(), [], drugs, panel

    for sample_id in P.columns:
        z_path = patient_vector(P, sample_id)
        b_series = drug_benefit_prior(z_path, panel).reindex(drugs).fillna(0.0)
        R = build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10)
        b = b_series.to_numpy(float)
        q, Q = build_qubo(b, R, lam=lam_value)

        # Use exact solver for now
        x_star, e_star = exact_qubo_solve(b, R, lam=lam_value)
        sel = [drugs[i] for i, xi in enumerate(x_star) if xi == 1]
        all_sels.append(sel)
        all_bhats.append(b_series)

        # Prepare data for patient report
        top_pw = z_path.abs().sort_values(ascending=False).head(5)
        patient_reports_data.append({
            "patient": sample_id,
            "selected_drugs": ", ".join(sel) if sel else "(none)",
            "top_pathways": "; ".join([f"{p}:{z_path[p]:+.2f}" for p in top_pw.index]),
            **{f"b̂.{d}": float(b_series.get(d,0.0)) for d in drugs}
        })

    # Drug Selection Summary
    st.subheader("Drug Selection Frequency")
    if not all_sels:
        st.info("No drugs were selected for any patient.")
        freq_df = pd.DataFrame({"drug": drugs, "frequency": 0, "frequency_pct": 0.0})
    else:
        flat = [d for sel in all_sels for d in sel]
        freq = Counter(flat)
        freq_df = pd.DataFrame({"drug": drugs, "frequency": [freq[d] for d in drugs]})
        freq_df["frequency_pct"] = 100 * freq_df["frequency"] / len(selected_samples)
        freq_df = freq_df.sort_values("frequency_pct", ascending=False).reset_index(drop=True)
    st.dataframe(freq_df)

    # Patient-level Report
    st.subheader("Patient-level Report")
    patient_report_df = pd.DataFrame(patient_reports_data)
    st.dataframe(patient_report_df)

    return P, freq_df, patient_report_df, all_sels, drugs, panel

# Function to create a download link
def create_download_link(df, filename, text):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    href = f'<a href="data:file/csv;base64,{b64}" download="{filename}">{text}</a>'
    return href


# --- Streamlit UI ---
st.set_page_config(layout="wide")

st.title("Digital Twin Workflow: Gene Expression to Drug Selection")

st.markdown(\"\"\"
This application demonstrates a workflow for identifying potential drug therapies
for cancer patients based on their gene expression data, using pathway analysis
and QUBO optimization.
\"\"\")

st.sidebar.header("Input Data")
uploaded_file = st.sidebar.file_uploader("Upload gene expression data (CSV, TSV, or gzipped)", type=["csv", "tsv", "gz"])

st.sidebar.header("Parameters")
n_samples = st.sidebar.slider("Number of samples to process", min_value=1, max_value=100, value=40, step=1)
lam_value = st.sidebar.slider("Lambda (penalty) value for QUBO", min_value=0.1, max_value=5.0, value=1.0, step=0.1)

st.header("Analysis Results")

if uploaded_file is not None:
    if st.button("Run Analysis"):
        with st.spinner(f"Running analysis for {n_samples} samples with lambda={lam_value}..."):
            pathway_scores_df, freq_df, patient_report_df, all_sels, drugs, panel = run_analysis(uploaded_file, n_samples, lam_value)

            if pathway_scores_df is not None:
                st.subheader("Visualizations")

                # Drug selection frequency barplot (Plotly)
                if freq_df is not None and not freq_df.empty:
                    try:
                        fig1 = px.bar(freq_df, x="drug", y="frequency_pct", title="Drug selection stability across patients")
                        fig1.update_layout(xaxis_title="Drug", yaxis_title="% patients selected", xaxis_tickangle=-45)
                        st.plotly_chart(fig1, use_container_width=True)
                    except Exception as e:
                        st.write(f"Could not generate frequency barplot: {e}")

                # Co-selection heatmap and dendrogram (Plotly & Matplotlib)
                if all_sels and drugs and panel:
                    try:
                        st.subheader("Co-selection Analysis")
                         # --- build co-selection counts ---
                        co_mat = pd.DataFrame(0, index=drugs, columns=drugs, dtype=int)
                        n_pat = max(1, len(all_sels))
                        for sel in all_sels:
                            uniq = list(dict.fromkeys(sel))
                            for i in range(len(uniq)):
                                for j in range(i, len(uniq)):
                                    di, dj = uniq[i], uniq[j]
                                    co_mat.loc[di, dj] += 1
                                    if i != j:
                                        co_mat.loc[dj, di] += 1

                        # normalize to % of patients
                        co_pct = co_mat / n_pat * 100.0
                        st.write("Co-selection matrix (% of patients):")
                        st.dataframe(co_pct.round(1))

                        # heatmap (Plotly)
                        fig2 = px.imshow(co_pct, text_auto=False, color_continuous_scale='Blues', # text_auto=True can make it crowded
                                         title='Drug co-selection heatmap',
                                         labels=dict(x="Drug", y="Drug", color="% patients co-selected"))
                        st.plotly_chart(fig2, use_container_width=True)


                        # dendrogram (Matplotlib) - Using Matplotlib for dendrogram due to Plotly complexity
                        try:
                            corr = np.corrcoef(co_pct.values)
                            corr = np.clip(corr, -1.0, 1.0)
                            dist = 1.0 - corr
                            Z = linkage(dist, method="average")

                            fig3_mpl, ax3_mpl = plt.subplots(figsize=(8,5))
                            dendrogram(Z, labels=drugs, leaf_rotation=90, leaf_font_size=10,
                                       color_threshold=0.7 * np.max(Z[:,2]), ax=ax3_mpl)
                            ax3_mpl.set_title("Clustered dendrogram of drug co-selection (Matplotlib)")
                            ax3_mpl.set_ylabel("Distance (1 - correlation)")
                            plt.tight_layout()
                            st.pyplot(fig3_mpl)
                            plt.close(fig3_mpl) # Close figure

                            # clustered heatmap (Plotly)
                            order = leaves_list(Z)
                            co_pct_ordered = co_pct.iloc[order, order]

                            fig4 = px.imshow(co_pct_ordered, text_auto=False, color_continuous_scale='Blues', # text_auto=True can make it crowded
                                              title='Clustered heatmap of drug co-selection (Ordered)',
                                              labels=dict(x="Drug", y="Drug", color="% patients co-selected"))
                            st.plotly_chart(fig4, use_container_width=True)
                            st.write("Clustered co-selection matrix:")
                            st.dataframe(co_pct_ordered.round(1))

                        except Exception as e:
                             st.write(f"Could not generate clustering plots (dendrogram/clustered heatmap): {e}")


                    except Exception as e:
                        st.write(f"Could not perform co-selection analysis: {e}")

                # Selection size distribution histogram (Plotly)
                if all_sels:
                    try:
                        sizes = [len(set(sel)) for sel in all_sels]
                        if sizes: # Check if sizes list is not empty
                            size_hist = pd.Series(sizes).value_counts().sort_index()
                            size_hist_df = size_hist.reset_index()
                            size_hist_df.columns = ["# drugs selected", "# patients"]
                            fig5 = px.bar(size_hist_df, x="# drugs selected", y="# patients",
                                          title="Selection size distribution")
                            fig5.update_layout(xaxis=dict(tickmode='linear')) # Ensure all integer ticks are shown
                            st.plotly_chart(fig5, use_container_width=True)
                        else:
                             st.info("No drug selections were made to plot selection size distribution.")

                    except Exception as e:
                        st.write(f"Could not generate selection size histogram: {e}")

                # Synthetic Classification Validation (Plotly)
                if all_sels and drugs and pathway_scores_df is not None and not pathway_scores_df.empty:
                    try:
                        st.subheader("Synthetic Classification Validation")
                        # 1) Construct patient-drug feature matrix
                        N = len(all_sels)
                        X = np.zeros((N, len(drugs)))
                        for i, sel in enumerate(all_sels):
                            for d in drugs: # Iterate through drugs to ensure correct indexing
                                if d in sel:
                                    X[i, drugs.index(d)] = 1

                        # 2) Synthetic binary labels
                        risk_scores = []
                        # Ensure we only process samples that were successfully processed in run_analysis
                        processed_sample_ids = pathway_scores_df.columns
                        if len(processed_sample_ids) == 0:
                            st.warning("No samples processed for synthetic classification.")
                            raise ValueError("No processed samples") # Trigger exception to skip
                        if len(processed_sample_ids) < N:
                             st.warning(f"Only {len(processed_sample_ids)} samples processed for classification (less than requested {N}). Using processed samples.")
                             # Adjust N and X to match processed samples
                             N_processed = len(processed_sample_ids)
                             # Need to map all_sels back to processed_sample_ids order if necessary
                             # Assuming all_sels corresponds to P.columns order
                             X_processed = np.zeros((N_processed, len(drugs)))
                             processed_sels = all_sels[:N_processed] # Assuming order matches
                             for i, sel in enumerate(processed_sels):
                                  for d in drugs:
                                       if d in sel:
                                            X_processed[i, drugs.index(d)] = 1
                             X = X_processed # Use the adjusted feature matrix
                             N = N_processed # Use the adjusted sample count


                        for sid in processed_sample_ids:
                             z_path = patient_vector(pathway_scores_df, sid)
                             risk_scores.append(z_path.mean())
                        risk_scores = np.array(risk_scores)

                        # Handle case where all risk scores are the same
                        if np.std(risk_scores) == 0:
                             st.warning("Cannot generate synthetic labels: all pathway mean scores are the same.")
                             st.write("Synthetic labels (all 0):")
                             st.write(np.zeros(N, dtype=int))
                        else:
                             y = (risk_scores > np.median(risk_scores)).astype(int)
                             st.write("Synthetic labels (0=low risk, 1=high risk):")
                             st.write(np.bincount(y))

                             # 3) Train simple logistic regression
                             if len(np.unique(y)) > 1: # Only train if there's more than one class
                                 clf = LogisticRegression(max_iter=200)
                                 clf.fit(X, y)
                                 y_pred = clf.predict_proba(X)[:,1]

                                 # 4) Metrics
                                 roc_auc = roc_auc_score(y, y_pred)
                                 pr_auc = average_precision_score(y, y_pred)

                                 st.write(f"ROC-AUC: {roc_auc:.3f}")
                                 st.write(f"PR-AUC: {pr_auc:.3f}")

                                 # Optional barplot of learned coefficients (Plotly)
                                 coefs = pd.Series(clf.coef_[0], index=drugs).sort_values()
                                 coefs_df = coefs.reset_index()
                                 coefs_df.columns = ["Drug", "Weight"]
                                 fig6 = px.bar(coefs_df, y="Drug", x="Weight", orientation='h',
                                               color="Weight", color_continuous_scale=['salmon', 'steelblue'],
                                               title="Drug selection coefficients (synthetic label prediction)")
                                 st.plotly_chart(fig6, use_container_width=True)

                             else:
                                 st.warning("Classification skipped: only one unique synthetic label class found.")

                    except ValueError as ve:
                         # Catch the specific ValueError raised for skipping classification
                         st.warning(f"Synthetic classification skipped: {ve}")
                    except Exception as e:
                        st.write(f"Could not perform synthetic classification validation: {e}")


                # Downloadable reports
                if patient_report_df is not None and not patient_report_df.empty:
                    st.subheader("Download Reports")
                    st.markdown(create_download_link(patient_report_df, "patient_report.csv", "Download Patient Report (CSV)"), unsafe_allow_html=True)

                if freq_df is not None and not freq_df.empty:
                     st.markdown(create_download_link(freq_df, "drug_frequency_report.csv", "Download Drug Frequency Report (CSV)"), unsafe_allow_html=True)

                st.success("Analysis complete!")

else:
    st.info("Please upload a gene expression data file to begin.")
"""

with open("app.py", "w") as f:
    f.write(streamlit_code)

print("Created app.py")

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

## Add documentation and examples

### Subtask:
Add documentation and examples to the web application to provide clear instructions on how to use it and include example data or use cases.


**Reasoning**:
Add documentation and examples to the Streamlit application code in `app.py` by including markdown sections for overview, data format, parameter explanations, results description, and download options. Include a note about example data sources.



In [ ]:
# Save the Streamlit application code to app.py
streamlit_code = """
import streamlit as st
import pandas as pd
import numpy as np
import gzip
import re
from collections import Counter
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
import matplotlib.pyplot as plt
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
import io
import base64
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assume the core logic functions from the notebook are available here.
# For demonstration, I'll include them directly. In a larger project,
# these would be in a separate module.

def norm_ensembl(x): return x.split('.',1)[0]
def norm_symbol(x):  return re.sub(r'[^A-Za-z0-9_-]+','', x)

def detect_row_mode(file_object, scan_rows=50000):
    \\"\\"\\"Detect row ID type (Ensembl vs symbol) from file-like object.\\"\\"\\"
    seen = Counter(); total = 0
    # Use a TextIOWrapper to treat the bytes stream as text
    # Ensure file_object is at the beginning
    file_object.seek(0)
    with io.TextIOWrapper(file_object, encoding="utf-8", errors="replace") as f:
        # Read the header line to skip it for row detection
        header = f.readline()
        # Use header=None because we already read the header
        reader = pd.read_csv(f, sep="\\t", chunksize=200_000, usecols=[0], dtype=str, header=None)
        for ch in reader:
            v = ch.iloc[:,0].astype(str)
            vals = v.head(min(len(v), scan_rows-total)).tolist()
            total += len(vals)
            seen.update('ENSG' if x.startswith('ENSG') else 'OTHER' for x in vals)
            if total >= scan_rows: break
    ratio = seen['ENSG']/max(1,(seen['ENSG']+seen['OTHER']))
    mode = 'ensembl' if ratio >= 0.6 else 'symbol'
    return mode

def stream_select_rows_columns(file_object, selected_samples, sig_genes, sym2ensg, row_mode, gene_col_name="Name", sep='\\t'):
    \\"\\"\\"Stream-select rows and columns from file-like object.\\"\\"\\"
    if row_mode == 'ensembl':
        target_rows = set(sym2ensg.get(g) for g in sig_genes if g in sym2ensg)
        normalize = norm_ensembl
    else:
        target_rows = set(sig_genes)
        normalize = norm_symbol

    # Ensure file_object is at the beginning
    file_object.seek(0)

    with io.TextIOWrapper(file_object, encoding="utf-8", errors="replace") as f:
        # Read header to find the actual gene_col_name and sample columns
        header = f.readline().rstrip("\\\\n").split(sep)
        try:
            gene_col_idx = header.index(gene_col_name)
        except ValueError:
             # If gene_col_name not found, assume the first column is the gene column
            gene_col_name = header[0]
            gene_col_idx = 0

        # Filter header to include only the gene column and selected sample columns
        usecols_filter = [gene_col_name] + selected_samples

        # Read data in chunks, selecting only necessary columns
        # Use header=None as we've already read the header
        reader = pd.read_csv(f, sep=sep, chunksize=50_000, dtype=str, header=None)
        kept = []
        for i, ch in enumerate(reader):
            # Assign original header to the chunk
            ch.columns = header
            # Select only the columns we need
            ch = ch[usecols_filter]

            ch = ch.rename(columns={gene_col_name: "row_id"})

            # Ensure 'row_id' column exists after renaming
            if 'row_id' not in ch.columns:
                 st.error(f"Error: Could not find gene identifier column '{gene_col_name}' in the uploaded file.")
                 return pd.DataFrame() # Return empty DataFrame on error

            ids = ch["row_id"].astype(str).map(normalize)
            mask = ids.isin(target_rows)

            if mask.any():
                out = ch.loc[mask].copy()
                out["row_id"] = out["row_id"].map(normalize)
                kept.append(out)

    if not kept:
        st.warning("No signature rows matched the provided gene list.")
        return pd.DataFrame()

    expr_small = pd.concat(kept, axis=0, ignore_index=False).drop_duplicates(subset=["row_id"]).set_index("row_id")

    if row_mode == 'ensembl':
        ensg2sym = {v:k for k,v in SYM2ENSG.items()}
        expr_sym_small = expr_small.copy()
        expr_sym_small.index = [ensg2sym.get(e, e) for e in expr_sym_small.index]
    else:
        expr_sym_small = expr_small.copy()

    expr_sym_small = expr_sym_small[~expr_sym_small.index.duplicated(keep="first")]
    return expr_sym_small


def zscore_by_gene(expr_symbols: pd.DataFrame) -> pd.DataFrame:
    E = expr_symbols.copy()
    E = E.apply(pd.to_numeric, errors="coerce")
    E = E.loc[~E.isna().all(axis=1)]
    gene_means = E.mean(axis=1)
    E = E.apply(lambda col: col.fillna(gene_means), axis=0)
    mu = E.mean(axis=1)
    sd = E.std(axis=1) + 1e-8
    return (E.sub(mu, axis=0)).div(sd, axis=0)

def pathway_scores(expr_symbols: pd.DataFrame, signatures: dict) -> pd.DataFrame:
    \"\"\"Return pathways x samples (mean z across member genes present).\"\"\"
    Z = zscore_by_gene(expr_symbols)
    rows = []
    for pw, genes in signatures.items():
        present = [g for g in genes if g in Z.index]
        if present:
            s = Z.loc[present].mean(axis=0)
        else:
            s = pd.Series([np.nan]*Z.shape[1], index=Z.columns)
        s.name = pw
        rows.append(s)
    return pd.DataFrame(rows)

def example_drug_panel():
    return {
        "EGFRi": ["REACTOME_SIGNALING_BY_EGFR"],
        "ALKi":  ["REACTOME_SIGNALING_BY_ALK"],
        "MEKi":  ["REACTOME_MAPK1_MAPK3_SIGNALING"],
        "PI3Ki": ["REACTOME_PI3K_AKT_SIGNALING"],
        "mTORi": ["REACTOME_MTORC1_MEDIATED_SIGNALLING"],
        "PD1i":  ["REACTOME_PD1_SIGNALING"],
        "VEGFi": ["REACTOME_VEGFA_VEGFR2_SIGNALING_PATHWAY"],
        "FGFRi": ["REACTOME_SIGNALING_BY_FGFR"],
    }

def patient_vector(P: pd.DataFrame, sample_id: str) -> pd.Series:
    \"\"\"z-normalize pathways across samples; return vector for one patient.\"\"\"
    z = (P - P.mean(axis=1).values.reshape(-1,1)) / (P.std(axis=1).values.reshape(-1,1) + 1e-8)
    return z[sample_id].fillna(0.0)

def drug_benefit_prior(z_path: pd.Series, panel: dict) -> pd.Series:
    \"\"\"Aggregate pathway z's per drug (ReLU to emphasize upregulated pathways).\"\"\"
    s = pd.Series({d: float(np.sum([max(z_path.get(p, 0.0), 0.0) for p in pws])) for d, pws in panel.items()})
    return s / s.max() if s.max() > 0 else s

def build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10):
    \"\"\"Pairwise penalties for overlapping mechanisms; diagonal = sparsity.\"\"\"
    K = len(drugs); R = np.zeros((K, K), dtype=float)
    for i in range(K):
        for j in range(i+1, K):
            overlap = len(set(panel[drugs[i]]) & set(panel[drugs[j]]))
            if overlap > 0:
                R[i, j] = R[j, i] = base_overlap * overlap
    for i in range(K):
        R[i, i] += sparsity
    return R

def build_qubo(b_hat, R, lam=1.0):
    \"\"\"QUBO: minimize x^T (lam R) x + q^T x  where q = -b̂ + diag(lam R).\"\"\"
    Q = lam * R.copy()
    q = -b_hat.copy()
    diag = np.diag(Q).copy()
    np.fill_diagonal(Q, 0.0)  # keep off-diagonal in Q
    q += diag
    return q, np.triu(Q, 1)

def exact_qubo_solve(b_hat: np.ndarray, R: np.ndarray, lam: float = 1.0):
    \"\"\"
    Exact minimization of QUBO:
      E(x) = x^T (lam R) x + q^T x, with q = -b̂ + diag(lam R)
    We enumerate all bitstrings (2^K). Returns best bitstring (as 0/1 np array).
    \"\"\"
    K = len(b_hat)
    Q = lam * R.copy()
    q = -b_hat.copy() + np.diag(Q)
    np.fill_diagonal(Q, 0.0)  # keep only off-diagonal in Q

    best_e = np.inf
    best_x = None
    # vectorize partial precomputations
    upper_idx = np.triu_indices(K, 1)
    for mask in range(1 << K):
        # build x from bits
        x = np.fromiter(((mask >> i) & 1 for i in range(K)), dtype=np.int8)
        # E = x^T Q x + q^T x, where Q is strictly upper-triangular mirrored
        e = np.dot(q, x) + 2.0 * np.sum(Q[upper_idx] * (x[upper_idx[0]] * x[upper_idx[1]]))
        if e < best_e:
            best_e, best_x = e, x
    return best_x, float(best_e)


# Placeholder for CUDA-Q function if needed later, currently uses exact solve
def try_cudaq_qaoa(h, J, p=2, shots=2048, max_iters=60, seed=7):
    return None # Not implemented in this web app version

# --- Main analysis function ---
def run_analysis(uploaded_file, n_samples, lam_value):
    \"\"\"
    Runs the full analysis workflow for the uploaded data.
    Returns pathway scores, drug selection summary, and patient report.
    \"\"\"
    SIGS = {
        "REACTOME_SIGNALING_BY_EGFR": [
            "EGFR","ERBB2","ERBB3","GRB2","SOS1","SHC1","PTPN11","KRAS","NRAS","HRAS",
            "BRAF","MAP2K1","MAP2K2","MAPK1","MAPK3","PLCG1","PIK3CA","PIK3R1","AKT1","AKT2","AKT3","GAB1"
        ],
        "REACTOME_SIGNALING_BY_ALK": [
            "ALK","EML4","GRB2","SHC1","PIK3CA","PIK3R1","AKT1","AKT2","AKT3","STAT3","MAP2K1","MAPK1","MAPK3"
        ],
        "REACTOME_MAPK1_MAPK3_SIGNALING": [
            "BRAF","RAF1","MAP2K1","MAP2K2","MAPK1","MAPK3","DUSP6","DUSP4","FOS","JUN","EGFR"
        ],
        "REACTOME_PI3K_AKT_SIGNALING": [
            "PIK3CA","PIK3CB","PIK3CD","PIK3R1","PIK3R2","AKT1","AKT2","AKT3","PTEN","MTOR","RHEB"
        ],
        "REACTOME_MTORC1_MEDIATED_SIGNALLING": [
            "MTOR","RPTOR","MLST8","RHEB","TSC1","TSC2","EIF4EBP1","RPS6KB1","RPS6"
        ],
        "REACTOME_PD1_SIGNALING": [
            "PDCD1","CD274","PDCD1LG2","PDCD1LG2","JAK1","JAK2","STAT1","IFNG","GZMB","LAG3","TIGIT","CXCL9","CXCL10"
        ],
        "REACTOME_VEGFA_VEGFR2_SIGNALING_PATHWAY": [
            "VEGFA","KDR","FLT1","FLT4","PTPRB","PLCG1","MAP2K1","MAPK1","NOS3"
        ],
        "REACTOME_SIGNALING_BY_FGFR": [
            "FGFR1","FGFR2","FGFR3","FGFR4","FRS2","PLCG1","PIK3CA","PIK3R1","MAP2K1","MAPK1"
        ]
    }
    SIG_GENES = sorted({g for gs in SIGS.values() for g in gs})
    SYM2ENSG = {
        "EGFR":"ENSG00000146648","ERBB2":"ENSG00000141736","ERBB3":"ENSG00000065361","GRB2":"ENSG00000177885",
        "SOS1":"ENSG00000115904","SHC1":"ENSG00000154639","PTPN11":"ENSG00000179295","KRAS":"ENSG00000133703",
        "NRAS":"ENSG00000213281","HRAS":"ENSG00000174775","BRAF":"ENSG00000157764","MAP2K1":"ENSG00000169032",
        "MAP2K2":"ENSG00000126934","MAPK1":"ENSG00000100030","MAPK3":"ENSG00000102882","PLCG1":"ENSG00000124181",
        "PIK3CA":"ENSG00000121879","PIK3R1":"ENSG00000145675","AKT1":"ENSG00000142208","AKT2":"ENSG00000105221",
        "AKT3":"ENSG00000117020","GAB1":"ENSG00000117676",
        "ALK":"ENSG00000171094","EML4":"ENSG00000143924","STAT3":"ENSG00000168610",
        "DUSP6":"ENSG00000139318","DUSP4":"ENSG00000120875","FOS":"ENSG00000170345","JUN":"ENSG00000177606","RAF1":"ENSG00000132155",
        "PIK3CB":"ENSG00000119402","PIK3CD":"ENSG00000171608","PIK3R2":"ENSG00000189403","PTEN":"ENSG00000171862",
        "MTOR":"ENSG00000198793","RHEB":"ENSG00000106615",
        "RPTOR":"ENSG00000141564","MLST8":"ENSG00000105705","TSC1":"ENSG00000165699","TSC2":"ENSG00000103197",
        "EIF4EBP1":"ENSG00000187840","RPS6KB1":"ENSG00000108443","RPS6":"ENSG00000137154",
        "PDCD1":"ENSG00000276977","CD274":"ENSG00000120217","PDCD1LG2":"ENSG00000197646","JAK1":"ENSG00000162434",
        "JAK2":"ENSG00000096968","STAT1":"ENSG00000115415","IFNG":"ENSG00000111537","GZMB":"ENSG00000100453",
        "LAG3":"ENSG00000089692","TIGIT":"ENSG00000181847","CXCL9":"ENSG00000138755","CXCL10":"ENSG00000169245",
        "VEGFA":"ENSG00000112715","KDR":"ENSG00000128052","FLT1":"ENSG00000102755","FLT4":"ENSG00000037280",
        "PTPRB":"ENSG00000160593","NOS3":"ENSG00000164867",
        "FGFR1":"ENSG00000077782","FGFR2":"ENSG00000066468","FGFR3":"ENSG00000068078","FGFR4":"ENSG00000069535",
        "FRS2":"ENSG00000181873"
    }

    # Data Loading and Initial Processing
    # Handle potential gzip compression and determine delimiter
    file_content = uploaded_file.getvalue()
    if uploaded_file.name.endswith('.gz'):
        try:
            gz_file = gzip.GzipFile(fileobj=io.BytesIO(file_content))
            # Peek at the first line after decompression to determine delimiter
            with io.TextIOWrapper(gz_file, encoding="utf-8", errors="replace") as f:
                header_peek = f.readline().rstrip("\\\\n")
                sep = ',' if ',' in header_peek.split() else '\\t'
            # Reset gz_file for actual reading
            gz_file.seek(0)
            file_stream = io.BytesIO(gz_file.read())
        except Exception as e:
            st.error(f"Error reading gzipped file: {e}")
            return None, None, None, None, None, None
    else:
        # Determine delimiter for non-gzipped files
        try:
            header_peek = io.BytesIO(file_content).readline().decode('utf-8').rstrip("\\\\n")
            sep = ',' if ',' in header_peek.split() else '\\t'
        except Exception as e:
             st.error(f"Error reading file header: {e}")
             return None, None, None, None, None, None
        file_stream = io.BytesIO(file_content)

    st.write(f"Detected delimiter: '{sep}'")

    # Detect row mode
    # Need a fresh file object for detect_row_mode
    file_stream_for_detect = io.BytesIO(file_stream.getvalue())
    row_mode = detect_row_mode(file_stream_for_detect)
    st.write(f"Detected row mode: {row_mode.upper()}")

    # Get header to identify sample columns and gene column
    file_stream.seek(0) # Ensure stream is at the beginning
    header_line = io.TextIOWrapper(file_stream, encoding="utf-8", errors="replace").readline().rstrip("\\\\n")
    cols = header_line.split(sep)
    # Assuming the first column is the gene ID/symbol column
    gene_col = cols[0]
    # Assuming sample columns start from the second column
    sample_cols_full = cols[1:]

    # Select samples based on n_samples parameter
    if n_samples > len(sample_cols_full):
        st.warning(f"Requested {n_samples} samples, but only {len(sample_cols_full)} available. Using all available samples.")
        selected_samples = sample_cols_full
    else:
        selected_samples = sample_cols_full[:n_samples]

    st.write(f"Processing {len(selected_samples)} samples.")
    st.write(f"Using gene column: '{gene_col}'")

    # Stream-select rows and columns
    # Need a fresh file object for stream_select_rows_columns
    file_stream_for_select = io.BytesIO(file_stream.getvalue())
    expr_sym_small = stream_select_rows_columns(file_stream_for_select, selected_samples, SIG_GENES, SYM2ENSG, row_mode, gene_col_name=gene_col, sep=sep)

    if expr_sym_small.empty:
        st.error("Failed to load expression data. Please check file format and contents.")
        return None, None, None, None, None, None

    st.subheader("Expression Matrix (Subset)")
    st.write("Shape (genes x samples):", expr_sym_small.shape)
    st.dataframe(expr_sym_small.head())

    # Pathway Scoring
    st.subheader("Pathway Activity Scores")
    P = pathway_scores(expr_sym_small, SIGS)
    st.write("Shape (pathways x samples):", P.shape)
    st.dataframe(P)

    # QUBO Formulation and Drug Selection
    st.subheader("Drug Selection Results")
    panel = example_drug_panel()
    drugs = list(panel.keys())

    all_sels = []
    all_bhats = []
    patient_reports_data = []

    # Run analysis for each selected sample
    if P.empty:
        st.warning("No pathway scores computed. Cannot perform drug selection.")
        return P, pd.DataFrame(), pd.DataFrame(), [], drugs, panel

    for sample_id in P.columns:
        z_path = patient_vector(P, sample_id)
        b_series = drug_benefit_prior(z_path, panel).reindex(drugs).fillna(0.0)
        R = build_penalty_matrix(drugs, panel, base_overlap=0.25, sparsity=0.10)
        b = b_series.to_numpy(float)
        q, Q = build_qubo(b, R, lam=lam_value)

        # Use exact solver for now
        x_star, e_star = exact_qubo_solve(b, R, lam=lam_value)
        sel = [drugs[i] for i, xi in enumerate(x_star) if xi == 1]
        all_sels.append(sel)
        all_bhats.append(b_series)

        # Prepare data for patient report
        top_pw = z_path.abs().sort_values(ascending=False).head(5)
        patient_reports_data.append({
            "patient": sample_id,
            "selected_drugs": ", ".join(sel) if sel else "(none)",
            "top_pathways": "; ".join([f"{p}:{z_path[p]:+.2f}" for p in top_pw.index]),
            **{f"b̂.{d}": float(b_series.get(d,0.0)) for d in drugs}
        })

    # Drug Selection Summary
    st.subheader("Drug Selection Frequency")
    if not all_sels:
        st.info("No drugs were selected for any patient.")
        freq_df = pd.DataFrame({"drug": drugs, "frequency": 0, "frequency_pct": 0.0})
    else:
        flat = [d for sel in all_sels for d in sel]
        freq = Counter(flat)
        freq_df = pd.DataFrame({"drug": drugs, "frequency": [freq[d] for d in drugs]})
        freq_df["frequency_pct"] = 100 * freq_df["frequency"] / len(selected_samples)
        freq_df = freq_df.sort_values("frequency_pct", ascending=False).reset_index(drop=True)
    st.dataframe(freq_df)

    # Patient-level Report
    st.subheader("Patient-level Report")
    patient_report_df = pd.DataFrame(patient_reports_data)
    st.dataframe(patient_report_df)

    return P, freq_df, patient_report_df, all_sels, drugs, panel

# Function to create a download link
def create_download_link(df, filename, text):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    #href = f'<a href="data:file/csv;base64,{b64}" download="{filename}">{text}</a>'
    # Using st.download_button instead of markdown link for better UX
    st.download_button(
        label=text,
        data=csv,
        file_name=filename,
        mime='text/csv',
    )


# --- Streamlit UI ---
st.set_page_config(layout="wide")

st.title("Digital Twin Workflow: Gene Expression to Drug Selection")

st.markdown(\"\"\"
This application demonstrates a workflow for identifying potential drug therapies
for cancer patients based on their gene expression data. It utilizes pathway
analysis to score biological activity and formulates a Quadratic Unconstrained
Binary Optimization (QUBO) problem to select a combination of drugs.

**How to Use:**

1.  **Upload Data:** Use the file uploader in the sidebar to upload your gene
    expression data. The expected format is a tab-separated or comma-separated
    file (CSV or TSV), with genes in rows and samples in columns. The first
    column should contain gene identifiers (either gene symbols or Ensembl IDs).
    The file can be optionally gzipped (`.gz`).
2.  **Set Parameters:** Adjust the "Number of samples to process" slider to
    specify how many samples from your uploaded file should be included in the
    analysis. Modify the "Lambda (penalty) value for QUBO" slider to control
    the trade-off between maximizing potential drug benefit and minimizing drug
    overlap in the optimization step.
3.  **Run Analysis:** Click the "Run Analysis" button to start the workflow.
    The application will process your data, calculate pathway scores, perform
    drug selection using the QUBO model, and generate various results and
    visualizations.
4.  **Explore Results:** Scroll down to view the analysis results, including
    subsets of the processed expression matrix and pathway scores, a summary of
    drug selection frequencies across the cohort, a patient-level report detailing
    selected drugs and pathway context for each sample, and several visualizations.
5.  **Download Reports:** Download the generated reports (Patient Report and
    Drug Frequency Report) as CSV files for further analysis or record-keeping.

**Example Data:**

You can obtain example gene expression data in the expected format from public
repositories like the TCGA (The Cancer Genome Atlas) or GTEx (Genotype-Tissue
Expression) projects, available through resources like the UCSC Xena Browser
(e.g., the Toil RNA-seq recompute data). Ensure the data is in a matrix format
(genes as rows, samples as columns) and saved as a tab-separated or comma-separated
file.

*Note: The current analysis focuses on a predefined set of signaling pathways
and a simplified drug panel. The synthetic classification validation is for
demonstration purposes only and does not represent a real-world clinical prediction.*
\"\"\")

st.sidebar.header("Input Data")
uploaded_file = st.sidebar.file_uploader("Upload gene expression data (CSV, TSV, or gzipped)", type=["csv", "tsv", "gz"])

st.sidebar.header("Parameters")
n_samples = st.sidebar.slider("Number of samples to process", min_value=1, max_value=100, value=40, step=1)
lam_value = st.sidebar.slider("Lambda (penalty) value for QUBO", min_value=0.1, max_value=5.0, value=1.0, step=0.1)

st.header("Analysis Results")

if uploaded_file is not None:
    if st.button("Run Analysis"):
        with st.spinner(f"Running analysis for {n_samples} samples with lambda={lam_value}..."):
            pathway_scores_df, freq_df, patient_report_df, all_sels, drugs, panel = run_analysis(uploaded_file, n_samples, lam_value)

            if pathway_scores_df is not None:
                st.subheader("Visualizations")

                # Drug selection frequency barplot (Plotly)
                if freq_df is not None and not freq_df.empty:
                    try:
                        fig1 = px.bar(freq_df, x="drug", y="frequency_pct", title="Drug selection stability across patients")
                        fig1.update_layout(xaxis_title="Drug", yaxis_title="% patients selected", xaxis_tickangle=-45)
                        st.plotly_chart(fig1, use_container_width=True)
                    except Exception as e:
                        st.write(f"Could not generate frequency barplot: {e}")

                # Co-selection heatmap and dendrogram (Plotly & Matplotlib)
                if all_sels and drugs and panel:
                    try:
                        st.subheader("Co-selection Analysis")
                         # --- build co-selection counts ---
                        co_mat = pd.DataFrame(0, index=drugs, columns=drugs, dtype=int)
                        n_pat = max(1, len(all_sels))
                        if n_pat > 0:
                            for sel in all_sels:
                                uniq = list(dict.fromkeys(sel))
                                for i in range(len(uniq)):
                                    for j in range(i, len(uniq)):
                                        di, dj = uniq[i], uniq[j]
                                        co_mat.loc[di, dj] += 1
                                        if i != j:
                                            co_mat.loc[dj, di] += 1

                            # normalize to % of patients
                            co_pct = co_mat / n_pat * 100.0
                            st.write("Co-selection matrix (% of patients):")
                            st.dataframe(co_pct.round(1))

                            # heatmap (Plotly)
                            fig2 = px.imshow(co_pct, text_auto=False, color_continuous_scale='Blues', # text_auto=True can make it crowded
                                             title='Drug co-selection heatmap',
                                             labels=dict(x="Drug", y="Drug", color="% patients co-selected"))
                            st.plotly_chart(fig2, use_container_width=True)


                            # dendrogram (Matplotlib) - Using Matplotlib for dendrogram due to Plotly complexity
                            try:
                                corr = np.corrcoef(co_pct.values)
                                corr = np.clip(corr, -1.0, 1.0)
                                dist = 1.0 - corr
                                # Check if distance matrix is valid for linkage
                                if np.isfinite(dist).all() and dist.shape[0] > 1:
                                    Z = linkage(dist, method="average")

                                    fig3_mpl, ax3_mpl = plt.subplots(figsize=(8,5))
                                    dendrogram(Z, labels=drugs, leaf_rotation=90, leaf_font_size=10,
                                               color_threshold=0.7 * np.max(Z[:,2]), ax=ax3_mpl)
                                    ax3_mpl.set_title("Clustered dendrogram of drug co-selection (Matplotlib)")
                                    ax3_mpl.set_ylabel("Distance (1 - correlation)")
                                    plt.tight_layout()
                                    st.pyplot(fig3_mpl)
                                    plt.close(fig3_mpl) # Close figure

                                    # clustered heatmap (Plotly)
                                    order = leaves_list(Z)
                                    co_pct_ordered = co_pct.iloc[order, order]

                                    fig4 = px.imshow(co_pct_ordered, text_auto=False, color_continuous_scale='Blues', # text_auto=True can make it crowded
                                                      title='Clustered heatmap of drug co-selection (Ordered)',
                                                      labels=dict(x="Drug", y="Drug", color="% patients co-selected"))
                                    st.plotly_chart(fig4, use_container_width=True)
                                    st.write("Clustered co-selection matrix:")
                                    st.dataframe(co_pct_ordered.round(1))
                                else:
                                     st.info("Could not generate clustering plots (dendrogram/clustered heatmap): Insufficient data or invalid distance matrix.")

                            except Exception as e:
                                 st.write(f"Could not generate clustering plots (dendrogram/clustered heatmap): {e}")
                        else:
                             st.info("Not enough patient data to perform co-selection analysis.")


                    except Exception as e:
                        st.write(f"Could not perform co-selection analysis: {e}")

                # Selection size distribution histogram (Plotly)
                if all_sels:
                    try:
                        sizes = [len(set(sel)) for sel in all_sels]
                        if sizes: # Check if sizes list is not empty
                            size_hist = pd.Series(sizes).value_counts().sort_index()
                            size_hist_df = size_hist.reset_index()
                            size_hist_df.columns = ["# drugs selected", "# patients"]
                            fig5 = px.bar(size_hist_df, x="# drugs selected", y="# patients",
                                          title="Selection size distribution")
                            fig5.update_layout(xaxis=dict(tickmode='linear')) # Ensure all integer ticks are shown
                            st.plotly_chart(fig5, use_container_width=True)
                        else:
                             st.info("No drug selections were made to plot selection size distribution.")

                    except Exception as e:
                        st.write(f"Could not generate selection size histogram: {e}")

                # Synthetic Classification Validation (Plotly)
                if all_sels and drugs and pathway_scores_df is not None and not pathway_scores_df.empty:
                    try:
                        st.subheader("Synthetic Classification Validation")
                        # 1) Construct patient-drug feature matrix
                        N = len(all_sels)
                        X = np.zeros((N, len(drugs)))
                        for i, sel in enumerate(all_sels):
                            for d in drugs: # Iterate through drugs to ensure correct indexing
                                if d in sel:
                                    X[i, drugs.index(d)] = 1

                        # 2) Synthetic binary labels
                        risk_scores = []
                        # Ensure we only process samples that were successfully processed in run_analysis
                        processed_sample_ids = pathway_scores_df.columns
                        if len(processed_sample_ids) == 0:
                            st.warning("No samples processed for synthetic classification.")
                            raise ValueError("No processed samples") # Trigger exception to skip
                        if len(processed_sample_ids) < N:
                             st.warning(f"Only {len(processed_sample_ids)} samples processed for classification (less than requested {N}). Using processed samples.")
                             # Adjust N and X to match processed samples
                             N_processed = len(processed_sample_ids)
                             # Need to map all_sels back to processed_sample_ids order if necessary
                             # Assuming all_sels corresponds to P.columns order
                             X_processed = np.zeros((N_processed, len(drugs)))
                             processed_sels = all_sels[:N_processed] # Assuming order matches
                             for i, sel in enumerate(processed_sels):
                                  for d in drugs:
                                       if d in sel:
                                            X_processed[i, drugs.index(d)] = 1
                             X = X_processed # Use the adjusted feature matrix
                             N = N_processed # Use the adjusted sample count


                        for sid in processed_sample_ids:
                             z_path = patient_vector(pathway_scores_df, sid)
                             risk_scores.append(z_path.mean())
                        risk_scores = np.array(risk_scores)

                        # Handle case where all risk scores are the same
                        if np.std(risk_scores) == 0:
                             st.warning("Cannot generate synthetic labels: all pathway mean scores are the same.")
                             st.write("Synthetic labels (all 0):")
                             st.write(np.zeros(N, dtype=int))
                        else:
                             y = (risk_scores > np.median(risk_scores)).astype(int)
                             st.write("Synthetic labels (0=low risk, 1=high risk):")
                             st.write(np.bincount(y))

                             # 3) Train simple logistic regression
                             if len(np.unique(y)) > 1: # Only train if there's more than one class
                                 clf = LogisticRegression(max_iter=200)
                                 clf.fit(X, y)
                                 y_pred = clf.predict_proba(X)[:,1]

                                 # 4) Metrics
                                 roc_auc = roc_auc_score(y, y_pred)
                                 pr_auc = average_precision_score(y, y_pred)

                                 st.write(f"ROC-AUC: {roc_auc:.3f}")
                                 st.write(f"PR-AUC: {pr_auc:.3f}")

                                 # Optional barplot of learned coefficients (Plotly)
                                 coefs = pd.Series(clf.coef_[0], index=drugs).sort_values()
                                 coefs_df = coefs.reset_index()
                                 coefs_df.columns = ["Drug", "Weight"]
                                 fig6 = px.bar(coefs_df, y="Drug", x="Weight", orientation='h',
                                               color="Weight", color_continuous_scale=['salmon', 'steelblue'],
                                               title="Drug selection coefficients (synthetic label prediction)")
                                 st.plotly_chart(fig6, use_container_width=True)

                             else:
                                 st.warning("Classification skipped: only one unique synthetic label class found.")

                    except ValueError as ve:
                         # Catch the specific ValueError raised for skipping classification
                         st.warning(f"Synthetic classification skipped: {ve}")
                    except Exception as e:
                        st.write(f"Could not perform synthetic classification validation: {e}")

                # Downloadable reports
                if patient_report_df is not None and not patient_report_df.empty:
                    st.subheader("Download Reports")
                    # Using st.download_button
                    create_download_link(patient_report_df, "patient_report.csv", "Download Patient Report (CSV)")


                if freq_df is not None and not freq_df.empty:
                     # Using st.download_button
                     create_download_link(freq_df, "drug_frequency_report.csv", "Download Drug Frequency Report (CSV)")


                st.success("Analysis complete!")

else:
    st.info("Please upload a gene expression data file to begin.")
"""

with open("app.py", "w") as f:
    f.write(streamlit_code)

print("Updated app.py with documentation and download buttons.")

## Summary:

### Data Analysis Key Findings

*   The core logic from the original Jupyter notebook, including functions for data normalization (`norm_ensembl`, `norm_symbol`), data mode detection (`detect_row_mode`), data selection and streaming (`stream_select_rows_columns`), gene-level z-scoring (`zscore_by_gene`), pathway scoring (`pathway_scores`), example drug panel definition (`example_drug_panel`), patient vector creation (`patient_vector`), drug benefit prior calculation (`drug_benefit_prior`), penalty matrix building (`build_penalty_matrix`), QUBO building (`build_qubo`), and exact QUBO solving (`exact_qubo_solve`), was successfully extracted and integrated into the web application's backend logic.
*   Streamlit was selected as the web framework for its ease of use in building data science applications.
*   A user interface was successfully implemented in Streamlit, featuring a file uploader for gene expression data, sliders for setting the number of samples and lambda parameter, and dedicated sections for displaying results in tables and plots.
*   The backend analysis workflow was successfully connected to the Streamlit frontend, allowing the "Run Analysis" button to trigger the full data processing and drug selection pipeline using uploaded data and user-defined parameters.
*   The application successfully handles data input from CSV/TSV/gzipped files, detects gene ID format (Ensembl vs. Symbol), and outputs results as tables (subset expression, pathway scores, drug frequency, patient report) and visualizations.
*   Matplotlib plots were largely converted to interactive Plotly charts (frequency barplot, co-selection heatmaps, selection size histogram, classification coefficients), enhancing the user experience, while the co-selection dendrogram remained a Matplotlib plot embedded in the app.
*   Downloadable reports for patient-level results and drug frequency were implemented using `st.download_button`.
*   An evaluation of scalability highlighted limitations in memory usage for large files/cohorts and, critically, the exponential scaling of the exact QUBO solver with the number of drugs (infeasible beyond ~20-25 drugs).
*   Potential optimizations proposed include using out-of-core processing (Dask, Vaex), implementing approximate classical or QIO solvers for QUBO, parallelizing multi-patient analysis, and improving memory management.
*   Comprehensive documentation on application usage, data format, parameters, and results was added to the Streamlit app's main page using markdown.
*   The necessary files (`app.py` and `requirements.txt`) were prepared for deployment on a platform like Streamlit Community Cloud.

### Insights or Next Steps

*   The current exact QUBO solver severely limits the scalability of the drug panel size. Implementing an approximate solver (e.g., simulated annealing) is crucial for applying this workflow to larger, more realistic drug panels.
*   For very large gene expression datasets or patient cohorts, incorporating out-of-core processing libraries (like Dask or Vaex) or parallelizing the patient analysis loop will be necessary to handle memory constraints and improve processing time.
